#Importing libraries

In [ ]:
import os
import shutil
import numpy as np
from google.colab import drive

# Clean the mount directory if it exists
mount_point = '/content/gdrive'
if os.path.exists(mount_point):
    # Remove only if it's a mount point (contains specific Google Drive files)
    try:
        for item in os.listdir(mount_point):
            item_path = os.path.join(mount_point, item)
            if os.path.isfile(item_path):
                os.remove(item_path)
            else:
                shutil.rmtree(item_path)
    except:
        pass

# Now mount
drive.mount('/content/gdrive')
os.chdir('/content/gdrive/MyDrive')

In [ ]:
os.chdir('/content/gdrive/MyDrive/DatasetsEvaluations')

In [ ]:
ls

arxivAcademicPapers/       EnronEmailDataset/       NewArticleCorpus/
arxivAcademicPapersStego/  EnronEmailDatasetStego/  NewArticleCorpusStego/


In [ ]:
pip install colorama rich

In [ ]:
pip install python-docx

#Evaluations NewArticleCorpus

In [ ]:
import math
from itertools import combinations, permutations
import os
from docx import Document
from docx.shared import RGBColor
from docx.enum.text import WD_COLOR_INDEX

class WordColor:
    """Color mapping for Word documents"""
    COLORS = {
        'red': RGBColor(255, 0, 0),
        'blue': RGBColor(0, 0, 255),
        'green': RGBColor(0, 255, 0),
        'yellow': RGBColor(255, 255, 0),
        'cyan': RGBColor(0, 255, 255),
        'magenta': RGBColor(255, 0, 255),
        'orange': RGBColor(255, 165, 0),
        'purple': RGBColor(128, 0, 128),
        'brown': RGBColor(165, 42, 42),
        'gray': RGBColor(128, 128, 128),
        'teal': RGBColor(0, 128, 128),
        'violet': RGBColor(238, 130, 238),
        'pink': RGBColor(255, 192, 203),
        'olive': RGBColor(128, 128, 0),
        'lime': RGBColor(0, 255, 0),
        'navy': RGBColor(0, 0, 128),
        'maroon': RGBColor(128, 0, 0),
        'coral': RGBColor(255, 127, 80),
        'turquoise': RGBColor(64, 224, 208),
        'gold': RGBColor(255, 215, 0),
        'silver': RGBColor(192, 192, 192),
        'indigo': RGBColor(75, 0, 130),
        'crimson': RGBColor(220, 20, 60),
        'beige': RGBColor(245, 245, 220)
    }

def binary_to_integer(bitstream):
    return int(bitstream, 2)

def select_color_combination(alpha, n, all_colors):
    combs = list(combinations(all_colors, n))
    return combs[alpha % len(combs)]

def unrank_permutation(n, beta, pi):
    perms = list(permutations(pi[:n]))
    return perms[beta % len(perms)]

def embed_k_block(M, cover_text, n, pi):
    # Predefined color palette
    palette = [
        'red', 'blue', 'green', 'yellow', 'cyan', 'magenta',
        'orange', 'purple', 'brown', 'gray', 'teal', 'violet',
        'pink', 'olive', 'lime','navy', 'maroon', 'coral', 'turquoise',
        'gold', 'silver', 'indigo', 'crimson', 'beige'
    ]

    # Calculate block capacities
    B_color = math.comb(len(palette), n)
    B_perm = math.factorial(n)
    BitsPerBlock = math.floor(math.log2(B_color * B_perm)) if B_color * B_perm > 0 else 0

    # Convert message to binary
    binary_msg = ''.join(format(ord(c), '08b') for c in M)
    k = math.ceil(len(binary_msg) / BitsPerBlock) if BitsPerBlock > 0 else 0
    padded_msg = binary_msg.ljust(k * BitsPerBlock, '0') if k > 0 else ''

    # Structure to store text with colors
    colored_chars = []
    cover_chars = list(cover_text)

    for block in range(k):
        start = block * BitsPerBlock
        chunk = padded_msg[start: start + BitsPerBlock]
        m = binary_to_integer(chunk)

        alpha = m // B_perm
        beta = m % B_perm

        # Get color combination and permutation
        color_comb = select_color_combination(alpha, n, palette)
        perm = unrank_permutation(n, beta, pi)

        # Apply colors to cover text
        for i in range(n):
            pos = block * n + i
            if pos < len(cover_chars):
                color = color_comb[perm[i]]
                colored_chars.append((cover_chars[pos], color))
            else:
                break

    # Add remaining uncolored characters
    remaining_pos = k * n
    if remaining_pos < len(cover_chars):
        for i in range(remaining_pos, len(cover_chars)):
            colored_chars.append((cover_chars[i], None))  # None = no color

    return colored_chars

def create_colored_word_document(colored_chars, output_path):
    """Creates a Word document with colored characters"""
    doc = Document()
    paragraph = doc.add_paragraph()

    current_run = paragraph.add_run()
    current_color = None

    for char, color in colored_chars:
        # If color changes, create a new run
        if color != current_color:
            if current_run.text:  # Save previous run if it contains text
                if current_color:
                    current_run.font.color.rgb = WordColor.COLORS[current_color]

            # Create a new run
            current_run = paragraph.add_run()
            current_color = color

        current_run.text += char

    # Apply color to the last run
    if current_color and current_run.text:
        current_run.font.color.rgb = WordColor.COLORS[current_color]

    doc.save(output_path)

def process_text_files_in_directory(input_dir, output_dir, secret_message, n=10, pi=None):
    """
    Processes all text files in a directory and generates colored Word files

    Args:
        input_dir: Directory containing text files
        output_dir: Output directory for Word files
        secret_message: Secret message to hide
        n: Block size for encoding
        pi: Permutation key (if None, uses range(20))
    """

    if pi is None:
        pi = list(range(20))

    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)

    # List all text files in the directory
    text_files = [f for f in os.listdir(input_dir) if f.endswith('.txt')]

    if not text_files:
        print(f"No .txt files found in directory '{input_dir}'")
        return

    print(f"Processing {len(text_files)} text files...")

    for filename in text_files:
        input_path = os.path.join(input_dir, filename)
        output_filename = os.path.splitext(filename)[0] + '_colored.docx'
        output_path = os.path.join(output_dir, output_filename)

        try:
            # Read text file
            with open(input_path, 'r', encoding='utf-8') as f:
                content = f.read()

            print(f"Processing: {filename} ({len(content)} characters)")

            # Apply steganography
            colored_chars = embed_k_block(secret_message, content, n, pi)

            # Create colored Word document
            create_colored_word_document(colored_chars, output_path)

            # Count colored characters
            colored_count = sum(1 for char, color in colored_chars if color is not None)
            total_count = len(colored_chars)

            print(f"  ✓ Word file generated: {output_filename}")
            print(f"  ✓ Colored characters: {colored_count}/{total_count} ({(colored_count/total_count)*100:.1f}%)")

        except Exception as e:
            print(f"  ✗ Error with {filename}: {e}")

def main():
    # Parameters
    secret_message = "Coding late into the night, fueled by coffee and a dream to build something amazing that changes everything for good."
    input_directory = '/content/gdrive/MyDrive/DatasetsEvaluations/NewArticleCorpus'  # Directory containing text files
    output_directory = '/content/gdrive/MyDrive/DatasetsEvaluations/NewArticleCorpusStego'  # Output directory for Word files
    n = 10
    pi = list(range(20))

    print("=== COLOR-BASED STEGANOGRAPHY IN TEXT FILES ===")
    print(f"Secret message: '{secret_message}'")
    print(f"Block size: {n}")
    print(f"Input directory: {input_directory}")
    print(f"Output directory: {output_directory}")
    print("-" * 60)

    # Process files
    process_text_files_in_directory(input_directory, output_directory, secret_message, n, pi)

    print("-" * 60)
    print("Processing completed!")
    print(f"Colored Word files are in directory: {output_directory}")

# Example function to create sample text files
def create_sample_text_files():
    """Creates sample text files for testing"""
    sample_dir = "text_files"
    os.makedirs(sample_dir, exist_ok=True)

    samples = {
        "sample1.txt": """Only boats catch connotes of the islands sober wines
only ships wrap the slips on the cleats of twining lines
only flags flap in tags with color that assigns
only passage on vessels""",

        "sample2.txt": """The quick brown fox jumps over the lazy dog.
This sentence contains all letters of the English alphabet.
Perfect for testing text processing algorithms.""",

        "sample3.txt": """Lorem ipsum dolor sit amet, consectetur adipiscing elit.
Sed do eiusmod tempor incididunt ut labore et dolore magna aliqua.
Ut enim ad minim veniam, quis nostrud exercitation ullamco."""
    }

    for filename, content in samples.items():
        with open(os.path.join(sample_dir, filename), 'w', encoding='utf-8') as f:
            f.write(content)

    print(f"Sample files created in directory '{sample_dir}'")

if __name__ == "__main__":
    # Install required dependency: pip install python-docx

    # Create sample files (uncomment if needed)
    # create_sample_text_files()

    # Execute main processing
    main()

In [ ]:
pip install xlsxwriter

#Evaluations Nazario.csv

In [ ]:
import pandas as pd
import math
from itertools import combinations, permutations
import xlsxwriter

def binary_to_integer(bitstream):
    return int(bitstream, 2)

def select_color_combination(alpha, n, all_colors):
    combs = list(combinations(all_colors, n))
    return combs[alpha % len(combs)]

def unrank_permutation(n, beta, pi):
    perms = list(permutations(pi[:n]))
    return perms[beta % len(perms)]

def embed_k_block(M, cover_text, n, pi):
    # Predefined color palette with basic Excel-compatible color names only
    palette = [
        ('red', (255, 0, 0)),
        ('blue', (0, 0, 255)),
        ('green', (0, 128, 0)),
        ('yellow', (255, 255, 0)),
        ('magenta', (255, 0, 255)),
        ('orange', (255, 165, 0)),
        ('purple', (128, 0, 128)),
        ('brown', (165, 42, 42)),
        ('gray', (128, 128, 128)),
        ('pink', (255, 192, 203))
    ]
    color_names = [c[0] for c in palette]

    # Calculate block capacities
    B_color = math.comb(len(color_names), n)
    B_perm = math.factorial(n)
    BitsPerBlock = math.floor(math.log2(B_color * B_perm)) if B_color * B_perm > 0 else 0

    if BitsPerBlock == 0:
        return []

    # Convert message to binary
    binary_msg = ''.join(format(ord(c), '08b') for c in M)
    k = math.ceil(len(binary_msg) / BitsPerBlock) if BitsPerBlock > 0 else 0

    if k == 0:
        return []

    padded_msg = binary_msg.ljust(k * BitsPerBlock, '0')

    stego_chars = []
    cover_chars = list(cover_text)

    for block in range(k):
        start = block * BitsPerBlock
        chunk = padded_msg[start : start + BitsPerBlock]
        m = binary_to_integer(chunk)

        alpha = m // B_perm
        beta = m % B_perm

        # Get color combination and permutation
        color_comb = select_color_combination(alpha, n, color_names)
        perm = unrank_permutation(n, beta, list(range(n)))

        # Apply colors to cover text
        for i in range(n):
            pos = block * n + i
            if pos < len(cover_chars):
                color_idx = perm[i]
                color_name = color_comb[color_idx]
                stego_chars.append({
                    'char': cover_chars[pos],
                    'color': color_name
                })
            else:
                break

    # Add remaining uncolored characters
    remaining_pos = k * n
    if remaining_pos < len(cover_chars):
        for char in cover_chars[remaining_pos:]:
            stego_chars.append({
                'char': char,
                'color': 'black'
            })

    return stego_chars

def process_email_body(body, secret_message, n=3):
    """Process email body and return colored character information"""
    if pd.isna(body) or body == "":
        return []

    body_str = str(body)
    # Use first few characters of body as permutation base
    pi = list(range(min(n, len(body_str))))

    # Embed secret message using coloration
    colored_chars = embed_k_block(secret_message, body_str, n, pi)
    return colored_chars

def create_colored_excel(input_csv, output_excel, secret_message="SECRET"):
    """Create Excel file with colored email bodies"""

    # Read the CSV file
    df = pd.read_csv(input_csv)

    # Create Excel writer
    with pd.ExcelWriter(output_excel, engine='xlsxwriter') as writer:
        # Write original data to first sheet
        df.to_excel(writer, sheet_name='Original Data', index=False)

        # Create workbook and formats
        workbook = writer.book

        # Define color formats with ONLY basic Excel-compatible colors
        color_formats = {
            'red': workbook.add_format({'color': 'red', 'font_size': 10}),
            'blue': workbook.add_format({'color': 'blue', 'font_size': 10}),
            'green': workbook.add_format({'color': 'green', 'font_size': 10}),
            'yellow': workbook.add_format({'color': 'yellow', 'font_size': 10}),
            'magenta': workbook.add_format({'color': 'magenta', 'font_size': 10}),
            'orange': workbook.add_format({'color': 'orange', 'font_size': 10}),
            'purple': workbook.add_format({'color': 'purple', 'font_size': 10}),
            'brown': workbook.add_format({'color': 'brown', 'font_size': 10}),
            'gray': workbook.add_format({'color': 'gray', 'font_size': 10}),
            'pink': workbook.add_format({'color': 'pink', 'font_size': 10}),
            'black': workbook.add_format({'color': 'black', 'font_size': 10})
        }

        # Create sheet for colored data
        worksheet = workbook.add_worksheet('Colored Email Bodies')

        # Write headers
        headers = list(df.columns)
        for col_num, header in enumerate(headers):
            worksheet.write(0, col_num, header)

        # Process each row
        for row_num in range(len(df)):
            # Copy all original data
            for col_num, col_name in enumerate(headers):
                if col_name != 'body':
                    value = df.iloc[row_num][col_name]
                    if pd.isna(value):
                        worksheet.write(row_num + 1, col_num, "")
                    else:
                        worksheet.write(row_num + 1, col_num, str(value))

            # Process body column with coloration
            body_content = df.iloc[row_num]['body']
            if pd.isna(body_content) or body_content == "":
                worksheet.write(row_num + 1, headers.index('body'), "")
                continue

            # Apply coloration to body (limit to first 200 chars for performance)
            body_preview = str(body_content)[:200]  # Further limit for performance
            colored_chars = process_email_body(body_preview, secret_message, n=2)  # Reduce n to 2

            if not colored_chars:
                worksheet.write(row_num + 1, headers.index('body'), body_preview)
                continue

            # Write colored body using rich string
            col_idx = headers.index('body')

            # Prepare rich string format
            rich_string_parts = []
            current_color = colored_chars[0]['color']
            current_text = ""

            for char_info in colored_chars:
                if char_info['color'] == current_color:
                    current_text += char_info['char']
                else:
                    # Add the accumulated text with current color
                    if current_text:
                        rich_string_parts.append(color_formats[current_color])
                        rich_string_parts.append(current_text)
                    # Start new color group
                    current_color = char_info['color']
                    current_text = char_info['char']

            # Add the last group
            if current_text:
                rich_string_parts.append(color_formats[current_color])
                rich_string_parts.append(current_text)

            # Write the rich string
            if rich_string_parts:
                try:
                    worksheet.write_rich_string(row_num + 1, col_idx, *rich_string_parts)
                except Exception as e:
                    # Fallback: write as plain text if rich string fails
                    print(f"Warning: Rich string failed for row {row_num}, using plain text: {e}")
                    plain_text = ''.join([char_info['char'] for char_info in colored_chars])
                    worksheet.write(row_num + 1, col_idx, plain_text)

        # Auto-adjust column widths
        for col_num, col_name in enumerate(headers):
            if col_name == 'body':
                worksheet.set_column(col_num, col_num, 50)  # Wider for body
            else:
                max_len = df[col_name].astype(str).str.len().max()
                worksheet.set_column(col_num, col_num, min(max_len + 2, 30))

        print("✅ Excel file created successfully!")

def main():
    input_file = '/content/gdrive/MyDrive/DatasetsEvaluations/EnronEmailDataset/Nazario.csv'
    output_file = '/content/gdrive/MyDrive/DatasetsEvaluations/EnronEmailDatasetStego/Nazario_Colored.xlsx'
    secret_message = "Coding late into the night, fueled by coffee and a dream to build something amazing that changes everything for good."  # Message to hide

    print("Processing email bodies with coloration steganography...")
    try:
        create_colored_excel(input_file, output_file, secret_message)
        print(f"✅ Colored Excel file created successfully: {output_file}")
        print(f"🔒 Secret message embedded: '{secret_message}'")
        print("📊 Check the 'Colored Email Bodies' sheet to see the colored text")
    except Exception as e:
        print(f"❌ Error: {e}")
        print("Please make sure the input file exists and is a valid CSV file.")

if __name__ == "__main__":
    main()

#Evaluations CEAS

In [ ]:
import pandas as pd
import math
from itertools import combinations, permutations
import xlsxwriter

def binary_to_integer(bitstream):
    return int(bitstream, 2)

def select_color_combination(alpha, n, all_colors):
    combs = list(combinations(all_colors, n))
    return combs[alpha % len(combs)]

def unrank_permutation(n, beta, pi):
    perms = list(permutations(pi[:n]))
    return perms[beta % len(perms)]

def embed_k_block(M, cover_text, n, pi):
    # Predefined color palette with basic Excel-compatible color names only
    palette = [
        ('red', (255, 0, 0)),
        ('blue', (0, 0, 255)),
        ('green', (0, 128, 0)),
        ('yellow', (255, 255, 0)),
        ('magenta', (255, 0, 255)),
        ('orange', (255, 165, 0)),
        ('purple', (128, 0, 128)),
        ('brown', (165, 42, 42)),
        ('gray', (128, 128, 128)),
        ('pink', (255, 192, 203))
    ]
    color_names = [c[0] for c in palette]

    # Calculate block capacities
    B_color = math.comb(len(color_names), n)
    B_perm = math.factorial(n)
    BitsPerBlock = math.floor(math.log2(B_color * B_perm)) if B_color * B_perm > 0 else 0

    if BitsPerBlock == 0:
        return []

    # Convert message to binary
    binary_msg = ''.join(format(ord(c), '08b') for c in M)
    k = math.ceil(len(binary_msg) / BitsPerBlock) if BitsPerBlock > 0 else 0

    if k == 0:
        return []

    padded_msg = binary_msg.ljust(k * BitsPerBlock, '0')

    stego_chars = []
    cover_chars = list(cover_text)

    for block in range(k):
        start = block * BitsPerBlock
        chunk = padded_msg[start : start + BitsPerBlock]
        m = binary_to_integer(chunk)

        alpha = m // B_perm
        beta = m % B_perm

        # Get color combination and permutation
        color_comb = select_color_combination(alpha, n, color_names)
        perm = unrank_permutation(n, beta, list(range(n)))

        # Apply colors to cover text
        for i in range(n):
            pos = block * n + i
            if pos < len(cover_chars):
                color_idx = perm[i]
                color_name = color_comb[color_idx]
                stego_chars.append({
                    'char': cover_chars[pos],
                    'color': color_name
                })
            else:
                break

    # Add remaining uncolored characters
    remaining_pos = k * n
    if remaining_pos < len(cover_chars):
        for char in cover_chars[remaining_pos:]:
            stego_chars.append({
                'char': char,
                'color': 'black'
            })

    return stego_chars

def process_email_body(body, secret_message, n=3):
    """Process email body and return colored character information"""
    if pd.isna(body) or body == "":
        return []

    body_str = str(body)
    # Use first few characters of body as permutation base
    pi = list(range(min(n, len(body_str))))

    # Embed secret message using coloration
    colored_chars = embed_k_block(secret_message, body_str, n, pi)
    return colored_chars

def create_colored_excel(input_csv, output_excel, secret_message="SECRET"):
    """Create Excel file with colored email bodies"""

    # Read the CSV file
    df = pd.read_csv(input_csv)

    # Create Excel writer
    with pd.ExcelWriter(output_excel, engine='xlsxwriter') as writer:
        # Write original data to first sheet
        df.to_excel(writer, sheet_name='Original Data', index=False)

        # Create workbook and formats
        workbook = writer.book

        # Define color formats with ONLY basic Excel-compatible colors
        color_formats = {
            'red': workbook.add_format({'color': 'red', 'font_size': 10}),
            'blue': workbook.add_format({'color': 'blue', 'font_size': 10}),
            'green': workbook.add_format({'color': 'green', 'font_size': 10}),
            'yellow': workbook.add_format({'color': 'yellow', 'font_size': 10}),
            'magenta': workbook.add_format({'color': 'magenta', 'font_size': 10}),
            'orange': workbook.add_format({'color': 'orange', 'font_size': 10}),
            'purple': workbook.add_format({'color': 'purple', 'font_size': 10}),
            'brown': workbook.add_format({'color': 'brown', 'font_size': 10}),
            'gray': workbook.add_format({'color': 'gray', 'font_size': 10}),
            'pink': workbook.add_format({'color': 'pink', 'font_size': 10}),
            'black': workbook.add_format({'color': 'black', 'font_size': 10})
        }

        # Create sheet for colored data
        worksheet = workbook.add_worksheet('Colored Email Bodies')

        # Write headers
        headers = list(df.columns)
        for col_num, header in enumerate(headers):
            worksheet.write(0, col_num, header)

        # Process each row
        for row_num in range(len(df)):
            # Copy all original data
            for col_num, col_name in enumerate(headers):
                if col_name != 'body':
                    value = df.iloc[row_num][col_name]
                    if pd.isna(value):
                        worksheet.write(row_num + 1, col_num, "")
                    else:
                        worksheet.write(row_num + 1, col_num, str(value))

            # Process body column with coloration
            body_content = df.iloc[row_num]['body']
            if pd.isna(body_content) or body_content == "":
                worksheet.write(row_num + 1, headers.index('body'), "")
                continue

            # Apply coloration to body (limit to first 200 chars for performance)
            body_preview = str(body_content)[:200]  # Further limit for performance
            colored_chars = process_email_body(body_preview, secret_message, n=2)  # Reduce n to 2

            if not colored_chars:
                worksheet.write(row_num + 1, headers.index('body'), body_preview)
                continue

            # Write colored body using rich string
            col_idx = headers.index('body')

            # Prepare rich string format
            rich_string_parts = []
            current_color = colored_chars[0]['color']
            current_text = ""

            for char_info in colored_chars:
                if char_info['color'] == current_color:
                    current_text += char_info['char']
                else:
                    # Add the accumulated text with current color
                    if current_text:
                        rich_string_parts.append(color_formats[current_color])
                        rich_string_parts.append(current_text)
                    # Start new color group
                    current_color = char_info['color']
                    current_text = char_info['char']

            # Add the last group
            if current_text:
                rich_string_parts.append(color_formats[current_color])
                rich_string_parts.append(current_text)

            # Write the rich string
            if rich_string_parts:
                try:
                    worksheet.write_rich_string(row_num + 1, col_idx, *rich_string_parts)
                except Exception as e:
                    # Fallback: write as plain text if rich string fails
                    print(f"Warning: Rich string failed for row {row_num}, using plain text: {e}")
                    plain_text = ''.join([char_info['char'] for char_info in colored_chars])
                    worksheet.write(row_num + 1, col_idx, plain_text)

        # Auto-adjust column widths
        for col_num, col_name in enumerate(headers):
            if col_name == 'body':
                worksheet.set_column(col_num, col_num, 50)  # Wider for body
            else:
                max_len = df[col_name].astype(str).str.len().max()
                worksheet.set_column(col_num, col_num, min(max_len + 2, 30))

        print("✅ Excel file created successfully!")

def main():
    input_file = '/content/gdrive/MyDrive/DatasetsEvaluations/EnronEmailDataset/CEAS.csv'
    output_file = '/content/gdrive/MyDrive/DatasetsEvaluations/EnronEmailDatasetStego/CEAS_Colored.xlsx'
    secret_message = "Coding late into the night, fueled by coffee and a dream to build something amazing that changes everything for good."  # Message to hide

    print("Processing email bodies with coloration steganography...")
    try:
        create_colored_excel(input_file, output_file, secret_message)
        print(f"✅ Colored Excel file created successfully: {output_file}")
        print(f"🔒 Secret message embedded: '{secret_message}'")
        print("📊 Check the 'Colored Email Bodies' sheet to see the colored text")
    except Exception as e:
        print(f"❌ Error: {e}")
        print("Please make sure the input file exists and is a valid CSV file.")

if __name__ == "__main__":
    main()

#Evaluations Enron

In [ ]:
import pandas as pd
import math
from itertools import combinations, permutations
import xlsxwriter

def binary_to_integer(bitstream):
    return int(bitstream, 2)

def select_color_combination(alpha, n, all_colors):
    combs = list(combinations(all_colors, n))
    return combs[alpha % len(combs)]

def unrank_permutation(n, beta, pi):
    perms = list(permutations(pi[:n]))
    return perms[beta % len(perms)]

def embed_k_block(M, cover_text, n, pi):
    # Predefined color palette with basic Excel-compatible color names only
    palette = [
        ('red', (255, 0, 0)),
        ('blue', (0, 0, 255)),
        ('green', (0, 128, 0)),
        ('yellow', (255, 255, 0)),
        ('magenta', (255, 0, 255)),
        ('orange', (255, 165, 0)),
        ('purple', (128, 0, 128)),
        ('brown', (165, 42, 42)),
        ('gray', (128, 128, 128)),
        ('pink', (255, 192, 203))
    ]
    color_names = [c[0] for c in palette]

    # Calculate block capacities
    B_color = math.comb(len(color_names), n)
    B_perm = math.factorial(n)
    BitsPerBlock = math.floor(math.log2(B_color * B_perm)) if B_color * B_perm > 0 else 0

    if BitsPerBlock == 0:
        return []

    # Convert message to binary
    binary_msg = ''.join(format(ord(c), '08b') for c in M)
    k = math.ceil(len(binary_msg) / BitsPerBlock) if BitsPerBlock > 0 else 0

    if k == 0:
        return []

    padded_msg = binary_msg.ljust(k * BitsPerBlock, '0')

    stego_chars = []
    cover_chars = list(cover_text)

    for block in range(k):
        start = block * BitsPerBlock
        chunk = padded_msg[start : start + BitsPerBlock]
        m = binary_to_integer(chunk)

        alpha = m // B_perm
        beta = m % B_perm

        # Get color combination and permutation
        color_comb = select_color_combination(alpha, n, color_names)
        perm = unrank_permutation(n, beta, list(range(n)))

        # Apply colors to cover text
        for i in range(n):
            pos = block * n + i
            if pos < len(cover_chars):
                color_idx = perm[i]
                color_name = color_comb[color_idx]
                stego_chars.append({
                    'char': cover_chars[pos],
                    'color': color_name
                })
            else:
                break

    # Add remaining uncolored characters
    remaining_pos = k * n
    if remaining_pos < len(cover_chars):
        for char in cover_chars[remaining_pos:]:
            stego_chars.append({
                'char': char,
                'color': 'black'
            })

    return stego_chars

def process_email_body(body, secret_message, n=3):
    """Process email body and return colored character information"""
    if pd.isna(body) or body == "":
        return []

    body_str = str(body)
    # Use first few characters of body as permutation base
    pi = list(range(min(n, len(body_str))))

    # Embed secret message using coloration
    colored_chars = embed_k_block(secret_message, body_str, n, pi)
    return colored_chars

def create_colored_excel(input_csv, output_excel, secret_message="SECRET"):
    """Create Excel file with colored email bodies"""

    # Read the CSV file
    df = pd.read_csv(input_csv)

    # Create Excel writer
    with pd.ExcelWriter(output_excel, engine='xlsxwriter') as writer:
        # Write original data to first sheet
        df.to_excel(writer, sheet_name='Original Data', index=False)

        # Create workbook and formats
        workbook = writer.book

        # Define color formats with ONLY basic Excel-compatible colors
        color_formats = {
            'red': workbook.add_format({'color': 'red', 'font_size': 10}),
            'blue': workbook.add_format({'color': 'blue', 'font_size': 10}),
            'green': workbook.add_format({'color': 'green', 'font_size': 10}),
            'yellow': workbook.add_format({'color': 'yellow', 'font_size': 10}),
            'magenta': workbook.add_format({'color': 'magenta', 'font_size': 10}),
            'orange': workbook.add_format({'color': 'orange', 'font_size': 10}),
            'purple': workbook.add_format({'color': 'purple', 'font_size': 10}),
            'brown': workbook.add_format({'color': 'brown', 'font_size': 10}),
            'gray': workbook.add_format({'color': 'gray', 'font_size': 10}),
            'pink': workbook.add_format({'color': 'pink', 'font_size': 10}),
            'black': workbook.add_format({'color': 'black', 'font_size': 10})
        }

        # Create sheet for colored data
        worksheet = workbook.add_worksheet('Colored Email Bodies')

        # Write headers
        headers = list(df.columns)
        for col_num, header in enumerate(headers):
            worksheet.write(0, col_num, header)

        # Process each row
        for row_num in range(len(df)):
            # Copy all original data
            for col_num, col_name in enumerate(headers):
                if col_name != 'body':
                    value = df.iloc[row_num][col_name]
                    if pd.isna(value):
                        worksheet.write(row_num + 1, col_num, "")
                    else:
                        worksheet.write(row_num + 1, col_num, str(value))

            # Process body column with coloration
            body_content = df.iloc[row_num]['body']
            if pd.isna(body_content) or body_content == "":
                worksheet.write(row_num + 1, headers.index('body'), "")
                continue

            # Apply coloration to body (limit to first 200 chars for performance)
            body_preview = str(body_content)[:200]  # Further limit for performance
            colored_chars = process_email_body(body_preview, secret_message, n=2)  # Reduce n to 2

            if not colored_chars:
                worksheet.write(row_num + 1, headers.index('body'), body_preview)
                continue

            # Write colored body using rich string
            col_idx = headers.index('body')

            # Prepare rich string format
            rich_string_parts = []
            current_color = colored_chars[0]['color']
            current_text = ""

            for char_info in colored_chars:
                if char_info['color'] == current_color:
                    current_text += char_info['char']
                else:
                    # Add the accumulated text with current color
                    if current_text:
                        rich_string_parts.append(color_formats[current_color])
                        rich_string_parts.append(current_text)
                    # Start new color group
                    current_color = char_info['color']
                    current_text = char_info['char']

            # Add the last group
            if current_text:
                rich_string_parts.append(color_formats[current_color])
                rich_string_parts.append(current_text)

            # Write the rich string
            if rich_string_parts:
                try:
                    worksheet.write_rich_string(row_num + 1, col_idx, *rich_string_parts)
                except Exception as e:
                    # Fallback: write as plain text if rich string fails
                    print(f"Warning: Rich string failed for row {row_num}, using plain text: {e}")
                    plain_text = ''.join([char_info['char'] for char_info in colored_chars])
                    worksheet.write(row_num + 1, col_idx, plain_text)

        # Auto-adjust column widths
        for col_num, col_name in enumerate(headers):
            if col_name == 'body':
                worksheet.set_column(col_num, col_num, 50)  # Wider for body
            else:
                max_len = df[col_name].astype(str).str.len().max()
                worksheet.set_column(col_num, col_num, min(max_len + 2, 30))

        print("✅ Excel file created successfully!")

def main():
    input_file = '/content/gdrive/MyDrive/DatasetsEvaluations/EnronEmailDataset/Enron.csv'
    output_file = '/content/gdrive/MyDrive/DatasetsEvaluations/EnronEmailDatasetStego/Enron_Colored.xlsx'
    secret_message = "Coding late into the night, fueled by coffee and a dream to build something amazing that changes everything for good."  # Message to hide

    print("Processing email bodies with coloration steganography...")
    try:
        create_colored_excel(input_file, output_file, secret_message)
        print(f"✅ Colored Excel file created successfully: {output_file}")
        print(f"🔒 Secret message embedded: '{secret_message}'")
        print("📊 Check the 'Colored Email Bodies' sheet to see the colored text")
    except Exception as e:
        print(f"❌ Error: {e}")
        print("Please make sure the input file exists and is a valid CSV file.")

if __name__ == "__main__":
    main()

#Evaluations Ling

In [ ]:
import pandas as pd
import math
from itertools import combinations, permutations
import xlsxwriter

def binary_to_integer(bitstream):
    return int(bitstream, 2)

def select_color_combination(alpha, n, all_colors):
    combs = list(combinations(all_colors, n))
    return combs[alpha % len(combs)]

def unrank_permutation(n, beta, pi):
    perms = list(permutations(pi[:n]))
    return perms[beta % len(perms)]

def embed_k_block(M, cover_text, n, pi):
    # Predefined color palette with basic Excel-compatible color names only
    palette = [
        ('red', (255, 0, 0)),
        ('blue', (0, 0, 255)),
        ('green', (0, 128, 0)),
        ('yellow', (255, 255, 0)),
        ('magenta', (255, 0, 255)),
        ('orange', (255, 165, 0)),
        ('purple', (128, 0, 128)),
        ('brown', (165, 42, 42)),
        ('gray', (128, 128, 128)),
        ('pink', (255, 192, 203))
    ]
    color_names = [c[0] for c in palette]

    # Calculate block capacities
    B_color = math.comb(len(color_names), n)
    B_perm = math.factorial(n)
    BitsPerBlock = math.floor(math.log2(B_color * B_perm)) if B_color * B_perm > 0 else 0

    if BitsPerBlock == 0:
        return []

    # Convert message to binary
    binary_msg = ''.join(format(ord(c), '08b') for c in M)
    k = math.ceil(len(binary_msg) / BitsPerBlock) if BitsPerBlock > 0 else 0

    if k == 0:
        return []

    padded_msg = binary_msg.ljust(k * BitsPerBlock, '0')

    stego_chars = []
    cover_chars = list(cover_text)

    for block in range(k):
        start = block * BitsPerBlock
        chunk = padded_msg[start : start + BitsPerBlock]
        m = binary_to_integer(chunk)

        alpha = m // B_perm
        beta = m % B_perm

        # Get color combination and permutation
        color_comb = select_color_combination(alpha, n, color_names)
        perm = unrank_permutation(n, beta, list(range(n)))

        # Apply colors to cover text
        for i in range(n):
            pos = block * n + i
            if pos < len(cover_chars):
                color_idx = perm[i]
                color_name = color_comb[color_idx]
                stego_chars.append({
                    'char': cover_chars[pos],
                    'color': color_name
                })
            else:
                break

    # Add remaining uncolored characters
    remaining_pos = k * n
    if remaining_pos < len(cover_chars):
        for char in cover_chars[remaining_pos:]:
            stego_chars.append({
                'char': char,
                'color': 'black'
            })

    return stego_chars

def process_email_body(body, secret_message, n=3):
    """Process email body and return colored character information"""
    if pd.isna(body) or body == "":
        return []

    body_str = str(body)
    # Use first few characters of body as permutation base
    pi = list(range(min(n, len(body_str))))

    # Embed secret message using coloration
    colored_chars = embed_k_block(secret_message, body_str, n, pi)
    return colored_chars

def create_colored_excel(input_csv, output_excel, secret_message="SECRET"):
    """Create Excel file with colored email bodies"""

    # Read the CSV file
    df = pd.read_csv(input_csv)

    # Create Excel writer
    with pd.ExcelWriter(output_excel, engine='xlsxwriter') as writer:
        # Write original data to first sheet
        df.to_excel(writer, sheet_name='Original Data', index=False)

        # Create workbook and formats
        workbook = writer.book

        # Define color formats with ONLY basic Excel-compatible colors
        color_formats = {
            'red': workbook.add_format({'color': 'red', 'font_size': 10}),
            'blue': workbook.add_format({'color': 'blue', 'font_size': 10}),
            'green': workbook.add_format({'color': 'green', 'font_size': 10}),
            'yellow': workbook.add_format({'color': 'yellow', 'font_size': 10}),
            'magenta': workbook.add_format({'color': 'magenta', 'font_size': 10}),
            'orange': workbook.add_format({'color': 'orange', 'font_size': 10}),
            'purple': workbook.add_format({'color': 'purple', 'font_size': 10}),
            'brown': workbook.add_format({'color': 'brown', 'font_size': 10}),
            'gray': workbook.add_format({'color': 'gray', 'font_size': 10}),
            'pink': workbook.add_format({'color': 'pink', 'font_size': 10}),
            'black': workbook.add_format({'color': 'black', 'font_size': 10})
        }

        # Create sheet for colored data
        worksheet = workbook.add_worksheet('Colored Email Bodies')

        # Write headers
        headers = list(df.columns)
        for col_num, header in enumerate(headers):
            worksheet.write(0, col_num, header)

        # Process each row
        for row_num in range(len(df)):
            # Copy all original data
            for col_num, col_name in enumerate(headers):
                if col_name != 'body':
                    value = df.iloc[row_num][col_name]
                    if pd.isna(value):
                        worksheet.write(row_num + 1, col_num, "")
                    else:
                        worksheet.write(row_num + 1, col_num, str(value))

            # Process body column with coloration
            body_content = df.iloc[row_num]['body']
            if pd.isna(body_content) or body_content == "":
                worksheet.write(row_num + 1, headers.index('body'), "")
                continue

            # Apply coloration to body (limit to first 200 chars for performance)
            body_preview = str(body_content)[:200]  # Further limit for performance
            colored_chars = process_email_body(body_preview, secret_message, n=2)  # Reduce n to 2

            if not colored_chars:
                worksheet.write(row_num + 1, headers.index('body'), body_preview)
                continue

            # Write colored body using rich string
            col_idx = headers.index('body')

            # Prepare rich string format
            rich_string_parts = []
            current_color = colored_chars[0]['color']
            current_text = ""

            for char_info in colored_chars:
                if char_info['color'] == current_color:
                    current_text += char_info['char']
                else:
                    # Add the accumulated text with current color
                    if current_text:
                        rich_string_parts.append(color_formats[current_color])
                        rich_string_parts.append(current_text)
                    # Start new color group
                    current_color = char_info['color']
                    current_text = char_info['char']

            # Add the last group
            if current_text:
                rich_string_parts.append(color_formats[current_color])
                rich_string_parts.append(current_text)

            # Write the rich string
            if rich_string_parts:
                try:
                    worksheet.write_rich_string(row_num + 1, col_idx, *rich_string_parts)
                except Exception as e:
                    # Fallback: write as plain text if rich string fails
                    print(f"Warning: Rich string failed for row {row_num}, using plain text: {e}")
                    plain_text = ''.join([char_info['char'] for char_info in colored_chars])
                    worksheet.write(row_num + 1, col_idx, plain_text)

        # Auto-adjust column widths
        for col_num, col_name in enumerate(headers):
            if col_name == 'body':
                worksheet.set_column(col_num, col_num, 50)  # Wider for body
            else:
                max_len = df[col_name].astype(str).str.len().max()
                worksheet.set_column(col_num, col_num, min(max_len + 2, 30))

        print("✅ Excel file created successfully!")

def main():
    input_file = '/content/gdrive/MyDrive/DatasetsEvaluations/EnronEmailDataset/Ling.csv'
    output_file = '/content/gdrive/MyDrive/DatasetsEvaluations/EnronEmailDatasetStego/Ling_Colored.xlsx'
    secret_message = "Coding late into the night, fueled by coffee and a dream to build something amazing that changes everything for good."  # Message to hide

    print("Processing email bodies with coloration steganography...")
    try:
        create_colored_excel(input_file, output_file, secret_message)
        print(f"✅ Colored Excel file created successfully: {output_file}")
        print(f"🔒 Secret message embedded: '{secret_message}'")
        print("📊 Check the 'Colored Email Bodies' sheet to see the colored text")
    except Exception as e:
        print(f"❌ Error: {e}")
        print("Please make sure the input file exists and is a valid CSV file.")

if __name__ == "__main__":
    main()

#Evaluations NigerianFraud

In [ ]:
import pandas as pd
import math
from itertools import combinations, permutations
import xlsxwriter

def binary_to_integer(bitstream):
    return int(bitstream, 2)

def select_color_combination(alpha, n, all_colors):
    combs = list(combinations(all_colors, n))
    return combs[alpha % len(combs)]

def unrank_permutation(n, beta, pi):
    perms = list(permutations(pi[:n]))
    return perms[beta % len(perms)]

def embed_k_block(M, cover_text, n, pi):
    # Predefined color palette with basic Excel-compatible color names only
    palette = [
        ('red', (255, 0, 0)),
        ('blue', (0, 0, 255)),
        ('green', (0, 128, 0)),
        ('yellow', (255, 255, 0)),
        ('magenta', (255, 0, 255)),
        ('orange', (255, 165, 0)),
        ('purple', (128, 0, 128)),
        ('brown', (165, 42, 42)),
        ('gray', (128, 128, 128)),
        ('pink', (255, 192, 203))
    ]
    color_names = [c[0] for c in palette]

    # Calculate block capacities
    B_color = math.comb(len(color_names), n)
    B_perm = math.factorial(n)
    BitsPerBlock = math.floor(math.log2(B_color * B_perm)) if B_color * B_perm > 0 else 0

    if BitsPerBlock == 0:
        return []

    # Convert message to binary
    binary_msg = ''.join(format(ord(c), '08b') for c in M)
    k = math.ceil(len(binary_msg) / BitsPerBlock) if BitsPerBlock > 0 else 0

    if k == 0:
        return []

    padded_msg = binary_msg.ljust(k * BitsPerBlock, '0')

    stego_chars = []
    cover_chars = list(cover_text)

    for block in range(k):
        start = block * BitsPerBlock
        chunk = padded_msg[start : start + BitsPerBlock]
        m = binary_to_integer(chunk)

        alpha = m // B_perm
        beta = m % B_perm

        # Get color combination and permutation
        color_comb = select_color_combination(alpha, n, color_names)
        perm = unrank_permutation(n, beta, list(range(n)))

        # Apply colors to cover text
        for i in range(n):
            pos = block * n + i
            if pos < len(cover_chars):
                color_idx = perm[i]
                color_name = color_comb[color_idx]
                stego_chars.append({
                    'char': cover_chars[pos],
                    'color': color_name
                })
            else:
                break

    # Add remaining uncolored characters
    remaining_pos = k * n
    if remaining_pos < len(cover_chars):
        for char in cover_chars[remaining_pos:]:
            stego_chars.append({
                'char': char,
                'color': 'black'
            })

    return stego_chars

def process_email_body(body, secret_message, n=3):
    """Process email body and return colored character information"""
    if pd.isna(body) or body == "":
        return []

    body_str = str(body)
    # Use first few characters of body as permutation base
    pi = list(range(min(n, len(body_str))))

    # Embed secret message using coloration
    colored_chars = embed_k_block(secret_message, body_str, n, pi)
    return colored_chars

def create_colored_excel(input_csv, output_excel, secret_message="SECRET"):
    """Create Excel file with colored email bodies"""

    # Read the CSV file
    df = pd.read_csv(input_csv)

    # Create Excel writer
    with pd.ExcelWriter(output_excel, engine='xlsxwriter') as writer:
        # Write original data to first sheet
        df.to_excel(writer, sheet_name='Original Data', index=False)

        # Create workbook and formats
        workbook = writer.book

        # Define color formats with ONLY basic Excel-compatible colors
        color_formats = {
            'red': workbook.add_format({'color': 'red', 'font_size': 10}),
            'blue': workbook.add_format({'color': 'blue', 'font_size': 10}),
            'green': workbook.add_format({'color': 'green', 'font_size': 10}),
            'yellow': workbook.add_format({'color': 'yellow', 'font_size': 10}),
            'magenta': workbook.add_format({'color': 'magenta', 'font_size': 10}),
            'orange': workbook.add_format({'color': 'orange', 'font_size': 10}),
            'purple': workbook.add_format({'color': 'purple', 'font_size': 10}),
            'brown': workbook.add_format({'color': 'brown', 'font_size': 10}),
            'gray': workbook.add_format({'color': 'gray', 'font_size': 10}),
            'pink': workbook.add_format({'color': 'pink', 'font_size': 10}),
            'black': workbook.add_format({'color': 'black', 'font_size': 10})
        }

        # Create sheet for colored data
        worksheet = workbook.add_worksheet('Colored Email Bodies')

        # Write headers
        headers = list(df.columns)
        for col_num, header in enumerate(headers):
            worksheet.write(0, col_num, header)

        # Process each row
        for row_num in range(len(df)):
            # Copy all original data
            for col_num, col_name in enumerate(headers):
                if col_name != 'body':
                    value = df.iloc[row_num][col_name]
                    if pd.isna(value):
                        worksheet.write(row_num + 1, col_num, "")
                    else:
                        worksheet.write(row_num + 1, col_num, str(value))

            # Process body column with coloration
            body_content = df.iloc[row_num]['body']
            if pd.isna(body_content) or body_content == "":
                worksheet.write(row_num + 1, headers.index('body'), "")
                continue

            # Apply coloration to body (limit to first 200 chars for performance)
            body_preview = str(body_content)[:200]  # Further limit for performance
            colored_chars = process_email_body(body_preview, secret_message, n=2)  # Reduce n to 2

            if not colored_chars:
                worksheet.write(row_num + 1, headers.index('body'), body_preview)
                continue

            # Write colored body using rich string
            col_idx = headers.index('body')

            # Prepare rich string format
            rich_string_parts = []
            current_color = colored_chars[0]['color']
            current_text = ""

            for char_info in colored_chars:
                if char_info['color'] == current_color:
                    current_text += char_info['char']
                else:
                    # Add the accumulated text with current color
                    if current_text:
                        rich_string_parts.append(color_formats[current_color])
                        rich_string_parts.append(current_text)
                    # Start new color group
                    current_color = char_info['color']
                    current_text = char_info['char']

            # Add the last group
            if current_text:
                rich_string_parts.append(color_formats[current_color])
                rich_string_parts.append(current_text)

            # Write the rich string
            if rich_string_parts:
                try:
                    worksheet.write_rich_string(row_num + 1, col_idx, *rich_string_parts)
                except Exception as e:
                    # Fallback: write as plain text if rich string fails
                    print(f"Warning: Rich string failed for row {row_num}, using plain text: {e}")
                    plain_text = ''.join([char_info['char'] for char_info in colored_chars])
                    worksheet.write(row_num + 1, col_idx, plain_text)

        # Auto-adjust column widths
        for col_num, col_name in enumerate(headers):
            if col_name == 'body':
                worksheet.set_column(col_num, col_num, 50)  # Wider for body
            else:
                max_len = df[col_name].astype(str).str.len().max()
                worksheet.set_column(col_num, col_num, min(max_len + 2, 30))

        print("✅ Excel file created successfully!")

def main():
    input_file = '/content/gdrive/MyDrive/DatasetsEvaluations/EnronEmailDataset/NigerianFraud.csv'
    output_file = '/content/gdrive/MyDrive/DatasetsEvaluations/EnronEmailDatasetStego/NigerianFraud_Colored.xlsx'
    secret_message = "Coding late into the night, fueled by coffee and a dream to build something amazing that changes everything for good."  # Message to hide

    print("Processing email bodies with coloration steganography...")
    try:
        create_colored_excel(input_file, output_file, secret_message)
        print(f"✅ Colored Excel file created successfully: {output_file}")
        print(f"🔒 Secret message embedded: '{secret_message}'")
        print("📊 Check the 'Colored Email Bodies' sheet to see the colored text")
    except Exception as e:
        print(f"❌ Error: {e}")
        print("Please make sure the input file exists and is a valid CSV file.")

if __name__ == "__main__":
    main()

#Evaluations Spams

In [ ]:
import pandas as pd
import math
from itertools import combinations, permutations
import xlsxwriter

def binary_to_integer(bitstream):
    return int(bitstream, 2)

def select_color_combination(alpha, n, all_colors):
    combs = list(combinations(all_colors, n))
    return combs[alpha % len(combs)]

def unrank_permutation(n, beta, pi):
    perms = list(permutations(pi[:n]))
    return perms[beta % len(perms)]

def embed_k_block(M, cover_text, n, pi):
    # Predefined color palette with basic Excel-compatible color names only
    palette = [
        ('red', (255, 0, 0)),
        ('blue', (0, 0, 255)),
        ('green', (0, 128, 0)),
        ('yellow', (255, 255, 0)),
        ('magenta', (255, 0, 255)),
        ('orange', (255, 165, 0)),
        ('purple', (128, 0, 128)),
        ('brown', (165, 42, 42)),
        ('gray', (128, 128, 128)),
        ('pink', (255, 192, 203))
    ]
    color_names = [c[0] for c in palette]

    # Calculate block capacities
    B_color = math.comb(len(color_names), n)
    B_perm = math.factorial(n)
    BitsPerBlock = math.floor(math.log2(B_color * B_perm)) if B_color * B_perm > 0 else 0

    if BitsPerBlock == 0:
        return []

    # Convert message to binary
    binary_msg = ''.join(format(ord(c), '08b') for c in M)
    k = math.ceil(len(binary_msg) / BitsPerBlock) if BitsPerBlock > 0 else 0

    if k == 0:
        return []

    padded_msg = binary_msg.ljust(k * BitsPerBlock, '0')

    stego_chars = []
    cover_chars = list(cover_text)

    for block in range(k):
        start = block * BitsPerBlock
        chunk = padded_msg[start : start + BitsPerBlock]
        m = binary_to_integer(chunk)

        alpha = m // B_perm
        beta = m % B_perm

        # Get color combination and permutation
        color_comb = select_color_combination(alpha, n, color_names)
        perm = unrank_permutation(n, beta, list(range(n)))

        # Apply colors to cover text
        for i in range(n):
            pos = block * n + i
            if pos < len(cover_chars):
                color_idx = perm[i]
                color_name = color_comb[color_idx]
                stego_chars.append({
                    'char': cover_chars[pos],
                    'color': color_name
                })
            else:
                break

    # Add remaining uncolored characters
    remaining_pos = k * n
    if remaining_pos < len(cover_chars):
        for char in cover_chars[remaining_pos:]:
            stego_chars.append({
                'char': char,
                'color': 'black'
            })

    return stego_chars

def process_email_body(body, secret_message, n=3):
    """Process email body and return colored character information"""
    if pd.isna(body) or body == "":
        return []

    body_str = str(body)
    # Use first few characters of body as permutation base
    pi = list(range(min(n, len(body_str))))

    # Embed secret message using coloration
    colored_chars = embed_k_block(secret_message, body_str, n, pi)
    return colored_chars

def create_colored_excel(input_csv, output_excel, secret_message="SECRET"):
    """Create Excel file with colored email bodies"""

    # Read the CSV file
    df = pd.read_csv(input_csv)

    # Create Excel writer
    with pd.ExcelWriter(output_excel, engine='xlsxwriter') as writer:
        # Write original data to first sheet
        df.to_excel(writer, sheet_name='Original Data', index=False)

        # Create workbook and formats
        workbook = writer.book

        # Define color formats with ONLY basic Excel-compatible colors
        color_formats = {
            'red': workbook.add_format({'color': 'red', 'font_size': 10}),
            'blue': workbook.add_format({'color': 'blue', 'font_size': 10}),
            'green': workbook.add_format({'color': 'green', 'font_size': 10}),
            'yellow': workbook.add_format({'color': 'yellow', 'font_size': 10}),
            'magenta': workbook.add_format({'color': 'magenta', 'font_size': 10}),
            'orange': workbook.add_format({'color': 'orange', 'font_size': 10}),
            'purple': workbook.add_format({'color': 'purple', 'font_size': 10}),
            'brown': workbook.add_format({'color': 'brown', 'font_size': 10}),
            'gray': workbook.add_format({'color': 'gray', 'font_size': 10}),
            'pink': workbook.add_format({'color': 'pink', 'font_size': 10}),
            'black': workbook.add_format({'color': 'black', 'font_size': 10})
        }

        # Create sheet for colored data
        worksheet = workbook.add_worksheet('Colored Email Bodies')

        # Write headers
        headers = list(df.columns)
        for col_num, header in enumerate(headers):
            worksheet.write(0, col_num, header)

        # Process each row
        for row_num in range(len(df)):
            # Copy all original data
            for col_num, col_name in enumerate(headers):
                if col_name != 'body':
                    value = df.iloc[row_num][col_name]
                    if pd.isna(value):
                        worksheet.write(row_num + 1, col_num, "")
                    else:
                        worksheet.write(row_num + 1, col_num, str(value))

            # Process body column with coloration
            body_content = df.iloc[row_num]['body']
            if pd.isna(body_content) or body_content == "":
                worksheet.write(row_num + 1, headers.index('body'), "")
                continue

            # Apply coloration to body (limit to first 200 chars for performance)
            body_preview = str(body_content)[:200]  # Further limit for performance
            colored_chars = process_email_body(body_preview, secret_message, n=2)  # Reduce n to 2

            if not colored_chars:
                worksheet.write(row_num + 1, headers.index('body'), body_preview)
                continue

            # Write colored body using rich string
            col_idx = headers.index('body')

            # Prepare rich string format
            rich_string_parts = []
            current_color = colored_chars[0]['color']
            current_text = ""

            for char_info in colored_chars:
                if char_info['color'] == current_color:
                    current_text += char_info['char']
                else:
                    # Add the accumulated text with current color
                    if current_text:
                        rich_string_parts.append(color_formats[current_color])
                        rich_string_parts.append(current_text)
                    # Start new color group
                    current_color = char_info['color']
                    current_text = char_info['char']

            # Add the last group
            if current_text:
                rich_string_parts.append(color_formats[current_color])
                rich_string_parts.append(current_text)

            # Write the rich string
            if rich_string_parts:
                try:
                    worksheet.write_rich_string(row_num + 1, col_idx, *rich_string_parts)
                except Exception as e:
                    # Fallback: write as plain text if rich string fails
                    print(f"Warning: Rich string failed for row {row_num}, using plain text: {e}")
                    plain_text = ''.join([char_info['char'] for char_info in colored_chars])
                    worksheet.write(row_num + 1, col_idx, plain_text)

        # Auto-adjust column widths
        for col_num, col_name in enumerate(headers):
            if col_name == 'body':
                worksheet.set_column(col_num, col_num, 50)  # Wider for body
            else:
                max_len = df[col_name].astype(str).str.len().max()
                worksheet.set_column(col_num, col_num, min(max_len + 2, 30))

        print("✅ Excel file created successfully!")

def main():
    input_file = '/content/gdrive/MyDrive/DatasetsEvaluations/EnronEmailDataset/Spams.csv'
    output_file = '/content/gdrive/MyDrive/DatasetsEvaluations/EnronEmailDatasetStego/Spams_Colored.xlsx'
    secret_message = "Coding late into the night, fueled by coffee and a dream to build something amazing that changes everything for good."  # Message to hide

    print("Processing email bodies with coloration steganography...")
    try:
        create_colored_excel(input_file, output_file, secret_message)
        print(f"✅ Colored Excel file created successfully: {output_file}")
        print(f"🔒 Secret message embedded: '{secret_message}'")
        print("📊 Check the 'Colored Email Bodies' sheet to see the colored text")
    except Exception as e:
        print(f"❌ Error: {e}")
        print("Please make sure the input file exists and is a valid CSV file.")

if __name__ == "__main__":
    main()

#Evaluations arxivData

In [ ]:
import pandas as pd
import math
from itertools import combinations, permutations
import xlsxwriter

def binary_to_integer(bitstream):
    return int(bitstream, 2)

def select_color_combination(alpha, n, all_colors):
    combs = list(combinations(all_colors, n))
    return combs[alpha % len(combs)]

def unrank_permutation(n, beta, pi):
    perms = list(permutations(pi[:n]))
    return perms[beta % len(perms)]

def embed_k_block(M, cover_text, n, pi):
    # Predefined color palette with basic Excel-compatible color names only
    palette = [
        ('red', (255, 0, 0)),
        ('blue', (0, 0, 255)),
        ('green', (0, 128, 0)),
        ('yellow', (255, 255, 0)),
        ('magenta', (255, 0, 255)),
        ('orange', (255, 165, 0)),
        ('purple', (128, 0, 128)),
        ('brown', (165, 42, 42)),
        ('gray', (128, 128, 128)),
        ('pink', (255, 192, 203))
    ]
    color_names = [c[0] for c in palette]

    # Calculate block capacities
    B_color = math.comb(len(color_names), n)
    B_perm = math.factorial(n)
    BitsPerBlock = math.floor(math.log2(B_color * B_perm)) if B_color * B_perm > 0 else 0

    if BitsPerBlock == 0:
        return []

    # Convert message to binary
    binary_msg = ''.join(format(ord(c), '08b') for c in M)
    k = math.ceil(len(binary_msg) / BitsPerBlock) if BitsPerBlock > 0 else 0

    if k == 0:
        return []

    padded_msg = binary_msg.ljust(k * BitsPerBlock, '0')

    stego_chars = []
    cover_chars = list(cover_text)

    for block in range(k):
        start = block * BitsPerBlock
        chunk = padded_msg[start : start + BitsPerBlock]
        m = binary_to_integer(chunk)

        alpha = m // B_perm
        beta = m % B_perm

        # Get color combination and permutation
        color_comb = select_color_combination(alpha, n, color_names)
        perm = unrank_permutation(n, beta, list(range(n)))

        # Apply colors to cover text
        for i in range(n):
            pos = block * n + i
            if pos < len(cover_chars):
                color_idx = perm[i]
                color_name = color_comb[color_idx]
                stego_chars.append({
                    'char': cover_chars[pos],
                    'color': color_name
                })
            else:
                break

    # Add remaining uncolored characters
    remaining_pos = k * n
    if remaining_pos < len(cover_chars):
        for char in cover_chars[remaining_pos:]:
            stego_chars.append({
                'char': char,
                'color': 'black'
            })

    return stego_chars

def process_email_body(body, secret_message, n=3):
    """Process email body and return colored character information"""
    if pd.isna(body) or body == "":
        return []

    body_str = str(body)
    # Use first few characters of body as permutation base
    pi = list(range(min(n, len(body_str))))

    # Embed secret message using coloration
    colored_chars = embed_k_block(secret_message, body_str, n, pi)
    return colored_chars

def create_colored_excel(input_csv, output_excel, secret_message="SECRET"):
    """Create Excel file with colored email bodies"""

    # Read the CSV file
    df = pd.read_csv(input_csv)

    # Create Excel writer
    with pd.ExcelWriter(output_excel, engine='xlsxwriter') as writer:
        # Write original data to first sheet
        df.to_excel(writer, sheet_name='Original Data', index=False)

        # Create workbook and formats
        workbook = writer.book

        # Define color formats with ONLY basic Excel-compatible colors
        color_formats = {
            'red': workbook.add_format({'color': 'red', 'font_size': 10}),
            'blue': workbook.add_format({'color': 'blue', 'font_size': 10}),
            'green': workbook.add_format({'color': 'green', 'font_size': 10}),
            'yellow': workbook.add_format({'color': 'yellow', 'font_size': 10}),
            'magenta': workbook.add_format({'color': 'magenta', 'font_size': 10}),
            'orange': workbook.add_format({'color': 'orange', 'font_size': 10}),
            'purple': workbook.add_format({'color': 'purple', 'font_size': 10}),
            'brown': workbook.add_format({'color': 'brown', 'font_size': 10}),
            'gray': workbook.add_format({'color': 'gray', 'font_size': 10}),
            'pink': workbook.add_format({'color': 'pink', 'font_size': 10}),
            'black': workbook.add_format({'color': 'black', 'font_size': 10})
        }

        # Create sheet for colored data
        worksheet = workbook.add_worksheet('Colored Email summaries')

        # Write headers
        headers = list(df.columns)
        for col_num, header in enumerate(headers):
            worksheet.write(0, col_num, header)

        # Process each row
        for row_num in range(len(df)):
            # Copy all original data
            for col_num, col_name in enumerate(headers):
                if col_name != 'summaries':
                    value = df.iloc[row_num][col_name]
                    if pd.isna(value):
                        worksheet.write(row_num + 1, col_num, "")
                    else:
                        worksheet.write(row_num + 1, col_num, str(value))

            # Process body column with coloration
            body_content = df.iloc[row_num]['summaries']
            if pd.isna(body_content) or body_content == "":
                worksheet.write(row_num + 1, headers.index('summaries'), "")
                continue

            # Apply coloration to body (limit to first 200 chars for performance)
            body_preview = str(body_content)[:200]  # Further limit for performance
            colored_chars = process_email_body(body_preview, secret_message, n=2)  # Reduce n to 2

            if not colored_chars:
                worksheet.write(row_num + 1, headers.index('summaries'), body_preview)
                continue

            # Write colored body using rich string
            col_idx = headers.index('summaries')

            # Prepare rich string format
            rich_string_parts = []
            current_color = colored_chars[0]['color']
            current_text = ""

            for char_info in colored_chars:
                if char_info['color'] == current_color:
                    current_text += char_info['char']
                else:
                    # Add the accumulated text with current color
                    if current_text:
                        rich_string_parts.append(color_formats[current_color])
                        rich_string_parts.append(current_text)
                    # Start new color group
                    current_color = char_info['color']
                    current_text = char_info['char']

            # Add the last group
            if current_text:
                rich_string_parts.append(color_formats[current_color])
                rich_string_parts.append(current_text)

            # Write the rich string
            if rich_string_parts:
                try:
                    worksheet.write_rich_string(row_num + 1, col_idx, *rich_string_parts)
                except Exception as e:
                    # Fallback: write as plain text if rich string fails
                    print(f"Warning: Rich string failed for row {row_num}, using plain text: {e}")
                    plain_text = ''.join([char_info['char'] for char_info in colored_chars])
                    worksheet.write(row_num + 1, col_idx, plain_text)

        # Auto-adjust column widths
        for col_num, col_name in enumerate(headers):
            if col_name == 'summaries':
                worksheet.set_column(col_num, col_num, 50)  # Wider for body
            else:
                max_len = df[col_name].astype(str).str.len().max()
                worksheet.set_column(col_num, col_num, min(max_len + 2, 30))

        print("✅ Excel file created successfully!")

def main():
    input_file = '/content/gdrive/MyDrive/DatasetsEvaluations/arxivAcademicPapers/arxivData.csv'
    output_file = '/content/gdrive/MyDrive/DatasetsEvaluations/arxivAcademicPapersStego/arxivData_Colored.xlsx'
    secret_message = "Coding late into the night, fueled by coffee and a dream to build something amazing that changes everything for good."  # Message to hide

    print("Processing email summaries with coloration steganography...")
    try:
        create_colored_excel(input_file, output_file, secret_message)
        print(f"✅ Colored Excel file created successfully: {output_file}")
        print(f"🔒 Secret message embedded: '{secret_message}'")
        print("📊 Check the 'Colored Email summaries' sheet to see the colored text")
    except Exception as e:
        print(f"❌ Error: {e}")
        print("Please make sure the input file exists and is a valid CSV file.")

if __name__ == "__main__":
    main()

#Evaluations arxivPapers

In [ ]:
import pandas as pd
import math
from itertools import combinations, permutations
import xlsxwriter
import os

def binary_to_integer(bitstream):
    return int(bitstream, 2)

def select_color_combination(alpha, n, all_colors):
    combs = list(combinations(all_colors, n))
    return combs[alpha % len(combs)]

def unrank_permutation(n, beta, pi):
    perms = list(permutations(pi[:n]))
    return perms[beta % len(perms)]

def embed_k_block(M, cover_text, n, pi):
    # Predefined color palette with basic Excel-compatible color names only
    palette = [
        ('red', (255, 0, 0)),
        ('blue', (0, 0, 255)),
        ('green', (0, 128, 0)),
        ('yellow', (255, 255, 0)),
        ('magenta', (255, 0, 255)),
        ('orange', (255, 165, 0)),
        ('purple', (128, 0, 128)),
        ('brown', (165, 42, 42)),
        ('gray', (128, 128, 128)),
        ('pink', (255, 192, 203))
    ]
    color_names = [c[0] for c in palette]

    # Calculate block capacities
    B_color = math.comb(len(color_names), n)
    B_perm = math.factorial(n)
    BitsPerBlock = math.floor(math.log2(B_color * B_perm)) if B_color * B_perm > 0 else 0

    if BitsPerBlock == 0:
        return []

    # Convert message to binary
    binary_msg = ''.join(format(ord(c), '08b') for c in M)
    k = math.ceil(len(binary_msg) / BitsPerBlock) if BitsPerBlock > 0 else 0

    if k == 0:
        return []

    padded_msg = binary_msg.ljust(k * BitsPerBlock, '0')

    stego_chars = []
    cover_chars = list(cover_text)

    for block in range(k):
        start = block * BitsPerBlock
        chunk = padded_msg[start : start + BitsPerBlock]
        m = binary_to_integer(chunk)

        alpha = m // B_perm
        beta = m % B_perm

        # Get color combination and permutation
        color_comb = select_color_combination(alpha, n, color_names)
        perm = unrank_permutation(n, beta, list(range(n)))

        # Apply colors to cover text
        for i in range(n):
            pos = block * n + i
            if pos < len(cover_chars):
                color_idx = perm[i]
                color_name = color_comb[color_idx]
                stego_chars.append({
                    'char': cover_chars[pos],
                    'color': color_name
                })
            else:
                break

    # Add remaining uncolored characters
    remaining_pos = k * n
    if remaining_pos < len(cover_chars):
        for char in cover_chars[remaining_pos:]:
            stego_chars.append({
                'char': char,
                'color': 'black'
            })

    return stego_chars

def process_email_body(body, secret_message, n=3):
    """Process email body and return colored character information"""
    if pd.isna(body) or body == "":
        return []

    body_str = str(body)
    # Use first few characters of body as permutation base
    pi = list(range(min(n, len(body_str))))

    # Embed secret message using coloration
    colored_chars = embed_k_block(secret_message, body_str, n, pi)
    return colored_chars

def create_colored_excel(input_csv, output_excel, secret_message="SECRET"):
    """Create Excel file with colored email bodies"""

    # Read the CSV file
    df = pd.read_csv(input_csv)

    # Limit the number of rows for large datasets
    max_rows = 1000
    if len(df) > max_rows:
        print(f"⚠️  Dataset too large. Limiting to first {max_rows} rows.")
        df = df.head(max_rows)

    print(f"📊 Processing {len(df)} rows...")

    # Create Excel writer with ZIP64 enabled
    workbook = xlsxwriter.Workbook(output_excel, {'use_zip64': True})

    try:
        # Create sheets
        original_sheet = workbook.add_worksheet('Original Data')
        colored_sheet = workbook.add_worksheet('Colored Email summary')

        # Write headers for both sheets
        headers = list(df.columns)
        for col_num, header in enumerate(headers):
            original_sheet.write(0, col_num, header)
            colored_sheet.write(0, col_num, header)

        # Define color formats with ONLY basic Excel-compatible colors
        color_formats = {
            'red': workbook.add_format({'color': 'red', 'font_size': 10}),
            'blue': workbook.add_format({'color': 'blue', 'font_size': 10}),
            'green': workbook.add_format({'color': 'green', 'font_size': 10}),
            'yellow': workbook.add_format({'color': 'yellow', 'font_size': 10}),
            'magenta': workbook.add_format({'color': 'magenta', 'font_size': 10}),
            'orange': workbook.add_format({'color': 'orange', 'font_size': 10}),
            'purple': workbook.add_format({'color': 'purple', 'font_size': 10}),
            'brown': workbook.add_format({'color': 'brown', 'font_size': 10}),
            'gray': workbook.add_format({'color': 'gray', 'font_size': 10}),
            'pink': workbook.add_format({'color': 'pink', 'font_size': 10}),
            'black': workbook.add_format({'color': 'black', 'font_size': 10})
        }

        # Add a header format
        header_format = workbook.add_format({'bold': True, 'bg_color': '#D3D3D3'})
        for col_num, header in enumerate(headers):
            original_sheet.write(0, col_num, header, header_format)
            colored_sheet.write(0, col_num, header, header_format)

        # Statistics
        colored_rows = 0
        total_chars_colored = 0

        # Process each row
        for row_num in range(len(df)):
            if row_num % 100 == 0:
                print(f"📝 Processing row {row_num}/{len(df)}")

            # Write original data to both sheets
            for col_num, col_name in enumerate(headers):
                value = df.iloc[row_num][col_name]
                if pd.isna(value):
                    cell_value = ""
                else:
                    cell_value = str(value)

                # Write to original sheet
                original_sheet.write(row_num + 1, col_num, cell_value)

                # Write to colored sheet (without colors for non-summary columns)
                if col_name != 'summary':
                    colored_sheet.write(row_num + 1, col_num, cell_value)

            # Process body column with coloration for colored sheet only
            body_content = df.iloc[row_num]['summary']
            if pd.isna(body_content) or body_content == "":
                colored_sheet.write(row_num + 1, headers.index('summary'), "")
                continue

            # Apply coloration to body
            body_preview = str(body_content)[:100]  # Limit to 100 characters
            colored_chars = process_email_body(body_preview, secret_message, n=2)

            if not colored_chars:
                colored_sheet.write(row_num + 1, headers.index('summary'), body_preview)
                continue

            # Update statistics
            colored_rows += 1
            total_chars_colored += len([c for c in colored_chars if c['color'] != 'black'])

            # Write colored body using rich string
            col_idx = headers.index('summary')

            # Prepare rich string format
            rich_string_parts = []
            current_color = colored_chars[0]['color']
            current_text = ""

            for char_info in colored_chars:
                if char_info['color'] == current_color:
                    current_text += char_info['char']
                else:
                    # Add the accumulated text with current color
                    if current_text:
                        rich_string_parts.append(color_formats[current_color])
                        rich_string_parts.append(current_text)
                    # Start new color group
                    current_color = char_info['color']
                    current_text = char_info['char']

            # Add the last group
            if current_text:
                rich_string_parts.append(color_formats[current_color])
                rich_string_parts.append(current_text)

            # Write the rich string
            if rich_string_parts:
                try:
                    colored_sheet.write_rich_string(row_num + 1, col_idx, *rich_string_parts)
                except Exception as e:
                    # Fallback: write as plain text if rich string fails
                    print(f"⚠️  Rich string failed for row {row_num}, using plain text")
                    plain_text = ''.join([char_info['char'] for char_info in colored_chars])
                    colored_sheet.write(row_num + 1, col_idx, plain_text)

        # Auto-adjust column widths for both sheets
        for col_num, col_name in enumerate(headers):
            if col_name == 'summary':
                colored_sheet.set_column(col_num, col_num, 50)
                original_sheet.set_column(col_num, col_num, 50)
            else:
                max_len = df[col_name].astype(str).str.len().max()
                width = min(max_len + 2, 30)
                colored_sheet.set_column(col_num, col_num, width)
                original_sheet.set_column(col_num, col_num, width)

        # Create a summary sheet
        summary_sheet = workbook.add_worksheet('Steganography Summary')
        summary_sheet.write(0, 0, 'Steganography Statistics', header_format)
        summary_sheet.write(1, 0, 'Total Rows Processed:')
        summary_sheet.write(1, 1, len(df))
        summary_sheet.write(2, 0, 'Rows with Colored Text:')
        summary_sheet.write(2, 1, colored_rows)
        summary_sheet.write(3, 0, 'Total Characters Colored:')
        summary_sheet.write(3, 1, total_chars_colored)
        summary_sheet.write(4, 0, 'Secret Message:')
        summary_sheet.write(4, 1, secret_message)
        summary_sheet.write(5, 0, 'Steganography Method:')
        summary_sheet.write(5, 1, 'Color Encoding (n=2)')

        summary_sheet.set_column(0, 0, 25)
        summary_sheet.set_column(1, 1, 50)

        workbook.close()

        # Verify file was created
        if os.path.exists(output_excel):
            file_size = os.path.getsize(output_excel)
            print(f"✅ Excel file created successfully!")
            print(f"📁 File size: {file_size / (1024*1024):.2f} MB")
            print(f"🎨 Rows with colored text: {colored_rows}/{len(df)}")
            print(f"🔤 Total characters colored: {total_chars_colored}")
        else:
            print("❌ Error: Output file was not created")

    except Exception as e:
        workbook.close()
        raise e

def main():
    input_file = '/content/gdrive/MyDrive/DatasetsEvaluations/arxivAcademicPapers/arxivPapers.csv'
    output_file = '/content/gdrive/MyDrive/DatasetsEvaluations/arxivAcademicPapersStego/arxivPapers_Colored.xlsx'
    secret_message = "Coding late into the night, fueled by coffee and a dream to build something amazing that changes everything for good."

    print("🔒 Processing email summary with coloration steganography...")
    print(f"💬 Secret message: '{secret_message}'")
    print(f"📥 Input: {input_file}")
    print(f"📤 Output: {output_file}")

    try:
        create_colored_excel(input_file, output_file, secret_message)
        print(f"\n🎉 Success! Colored Excel file created: {output_file}")
        print("📊 Sheets included:")
        print("   - 'Original Data': Unmodified data")
        print("   - 'Colored Email summary': Text with hidden message")
        print("   - 'Steganography Summary': Technical details")

    except Exception as e:
        print(f"❌ Error: {e}")
        print("Please make sure the input file exists and is a valid CSV file.")

if __name__ == "__main__":
    main()

In [ ]:
import math
from itertools import combinations, permutations

def binary_to_integer(bitstream):
    return int(bitstream, 2)

def select_color_combination(alpha, n, all_colors):
    combs = list(combinations(all_colors, n))
    return combs[alpha % len(combs)]

def unrank_permutation(n, beta, pi):
    perms = list(permutations(pi[:n]))
    return perms[beta % len(perms)]

def embed_k_block(M, cover_text, n, pi):
    # Predefined 24-bit color palette (RGB tuples)
    palette = [
        ('red', (255, 0, 0)), ('blue', (0, 0, 255)), ('green', (0, 255, 0)),
        ('yellow', (255, 255, 0)), ('cyan', (0, 255, 255)), ('magenta', (255, 0, 255)),
        ('orange', (255, 165, 0)), ('purple', (128, 0, 128)), ('brown', (165, 42, 42)),
        ('gray', (128, 128, 128)), ('teal', (0, 128, 128)), ('violet', (238, 130, 238)),
        ('pink', (255, 192, 203)), ('olive', (128, 128, 0)), ('lime', (0, 255, 0))
    ]
    color_names = [c[0] for c in palette]  # Extract color names only

    # Calculate block capacities
    B_color = math.comb(2**24, n)  # Theoretical (16.7M for n=1)
    B_perm = math.factorial(n)
    BitsPerBlock = math.floor(math.log2(B_color * B_perm))

    # Convert message to binary
    binary_msg = ''.join(format(ord(c), '08b') for c in M)
    k = math.ceil(len(binary_msg) / BitsPerBlock)
    padded_msg = binary_msg.ljust(k * BitsPerBlock, '0')

    stego_text = []
    cover_chars = list(cover_text)

    for block in range(k):
        start = block * BitsPerBlock
        chunk = padded_msg[start : start + BitsPerBlock]
        m = binary_to_integer(chunk)

        alpha = m // B_perm
        beta = m % B_perm

        # Get color combination and permutation
        color_comb = select_color_combination(alpha, n, color_names)
        perm = unrank_permutation(n, beta, pi)

        # Apply colors to cover text
        for i in range(n):
            pos = block * n + i
            if pos < len(cover_chars):
                color = color_comb[perm[i]]
                stego_text.append(f'\\textcolor{{{color}}}{{{cover_chars[pos]}}}')
            else:
                break

    # Add remaining uncolored characters
    remaining_pos = k * n
    if remaining_pos < len(cover_chars):
        stego_text.extend(cover_chars[remaining_pos:])

    return ''.join(stego_text)

# Inputs from the problem
secret_message = "underlying physiological mechanisms"
cover_text = "Only boats catch connotes of the islands sober wines only ships wrap the slips on the cleats of twining lines only flags flap in tags with color that assigns only passage on vessels"
n = 10
pi = list(range(20))  # Permutation key

# Generate stego-text
stego_output = embed_k_block(secret_message, cover_text, n, pi)

# Print first 181 characters (as in the example)
print(stego_output[:len(stego_output)])

In [ ]:
import math
from itertools import combinations, permutations

class ConsoleColor:
    """ANSI color codes for console output"""
    COLORS = {
        'red': '\033[91m',
        'blue': '\033[94m',
        'green': '\033[92m',
        'yellow': '\033[93m',
        'cyan': '\033[96m',
        'magenta': '\033[95m',
        'orange': '\033[38;5;208m',
        'purple': '\033[38;5;129m',
        'brown': '\033[38;5;130m',
        'gray': '\033[38;5;240m',
        'teal': '\033[38;5;30m',
        'violet': '\033[38;5;177m',
        'pink': '\033[38;5;211m',
        'olive': '\033[38;5;100m',
        'lime': '\033[38;5;154m',
        'reset': '\033[0m'
    }

    @staticmethod
    def color_char(char, color_name):
        """Color a single character for console output"""
        return f"{ConsoleColor.COLORS.get(color_name, '')}{char}{ConsoleColor.COLORS['reset']}"

def binary_to_integer(bitstream):
    return int(bitstream, 2)

def select_color_combination(alpha, n, all_colors):
    combs = list(combinations(all_colors, n))
    return combs[alpha % len(combs)]

def unrank_permutation(n, beta, pi):
    perms = list(permutations(pi[:n]))
    return perms[beta % len(perms)]

def embed_k_block(M, cover_text, n, pi):
    # Predefined color palette
    palette = [
        'red', 'blue', 'green', 'yellow', 'cyan', 'magenta',
        'orange', 'purple', 'brown', 'gray', 'teal', 'violet',
        'pink', 'olive', 'lime'
    ]

    # Calculate block capacities
    B_color = math.comb(len(palette), n)  # Using actual palette size instead of 2^24
    B_perm = math.factorial(n)
    BitsPerBlock = math.floor(math.log2(B_color * B_perm)) if B_color * B_perm > 0 else 0

    # Convert message to binary
    binary_msg = ''.join(format(ord(c), '08b') for c in M)
    k = math.ceil(len(binary_msg) / BitsPerBlock) if BitsPerBlock > 0 else 0
    #print(k)
    padded_msg = binary_msg.ljust(k * BitsPerBlock, '0') if k > 0 else ''

    stego_text = []
    cover_chars = list(cover_text)

    for block in range(k):
        start = block * BitsPerBlock
        chunk = padded_msg[start : start + BitsPerBlock]
        m = binary_to_integer(chunk)

        alpha = m // B_perm
        beta = m % B_perm

        # Get color combination and permutation
        color_comb = select_color_combination(alpha, n, palette)
        perm = unrank_permutation(n, beta, pi)

        # Apply colors to cover text
        for i in range(n):
            pos = block * n + i
            if pos < len(cover_chars):
                color = color_comb[perm[i]]
                stego_text.append(ConsoleColor.color_char(cover_chars[pos], color))
            else:
                break

    # Add remaining uncolored characters
    remaining_pos = k * n
    if remaining_pos < len(cover_chars):
        stego_text.extend(cover_chars[remaining_pos:])

    return ''.join(stego_text)

# Input parameters
secret_message = "underlying physiological mechanisms"
cover_text = "Only boats catch connotes of the islands sober wines only ships wrap the slips on the cleats of twining lines only flags flap in tags with color that assigns only passage on vessels"
n = 10
pi = list(range(20))  # Permutation key

# Generate and print colored text
colored_output = embed_k_block(secret_message, cover_text, n, pi)
secret = secret_message
# Print the first 181 characters with actual colors in console
print(colored_output[:len(colored_output)])

In [ ]:
import math
from itertools import combinations, permutations

def binary_to_integer(bitstream):
    return int(bitstream, 2)

def select_color_combination(alpha, n, all_colors):
    combs = list(combinations(all_colors, n))
    return combs[alpha % len(combs)]

def unrank_permutation(n, beta, pi):
    perms = list(permutations(pi[:n]))
    return perms[beta % len(perms)]

def embed_k_block(M, cover_text, n, pi):
    # Predefined 24-bit color palette (RGB tuples)
    palette = [
        ('red', (255, 0, 0)), ('blue', (0, 0, 255)), ('green', (0, 255, 0)),
        ('yellow', (255, 255, 0)), ('cyan', (0, 255, 255)), ('magenta', (255, 0, 255)),
        ('orange', (255, 165, 0)), ('purple', (128, 0, 128)), ('brown', (165, 42, 42)),
        ('gray', (128, 128, 128)), ('teal', (0, 128, 128)), ('violet', (238, 130, 238)),
        ('pink', (255, 192, 203)), ('olive', (128, 128, 0)), ('lime', (0, 255, 0))
    ]
    color_names = [c[0] for c in palette]  # Extract color names only

    # Calculate block capacities
    B_color = math.comb(2**24, n)  # Theoretical (16.7M for n=1)
    B_perm = math.factorial(n)
    BitsPerBlock = math.floor(math.log2(B_color * B_perm))

    # Convert message to binary
    binary_msg = ''.join(format(ord(c), '08b') for c in M)
    k = math.ceil(len(binary_msg) / BitsPerBlock)
    padded_msg = binary_msg.ljust(k * BitsPerBlock, '0')

    stego_text = []
    cover_chars = list(cover_text)

    for block in range(k):
        start = block * BitsPerBlock
        chunk = padded_msg[start : start + BitsPerBlock]
        m = binary_to_integer(chunk)

        alpha = m // B_perm
        beta = m % B_perm

        # Get color combination and permutation
        color_comb = select_color_combination(alpha, n, color_names)
        perm = unrank_permutation(n, beta, pi)

        # Apply colors to cover text
        for i in range(n):
            pos = block * n + i
            if pos < len(cover_chars):
                color = color_comb[perm[i]]
                stego_text.append(f'\\textcolor{{{color}}}{{{cover_chars[pos]}}}')
            else:
                break

    # Add remaining uncolored characters
    remaining_pos = k * n
    if remaining_pos < len(cover_chars):
        stego_text.extend(cover_chars[remaining_pos:])

    return ''.join(stego_text)

# Inputs from the problem
secret_message = "Success starts with small steps. Every morning brings new opportunities. The world is full of possibilities dare to explore! Challenges make us stronger; failures teach resilience. Smile often, kindness costs nothing. Time flies, so chase your dreams today. Learn, grow, and never stop believing. A positive mind creates a brighter future. What is your next goal? Take action now! Life is too short for regrets. Shine bright, stay curious and inspire others. Remember: every effort counts. Keep going! Progress happens when you push forward"
cover_text = "Only boats catch connotes of the islands sober wines only ships wrap the slips on the clats of twining lines only flags flap in tags with color that assigns only passage on vessels"
n = 10
pi = list(range(20))  # Permutation key

# Generate stego-text
stego_output = embed_k_block(secret_message, cover_text, n, pi)

# Print first 181 characters (as in the example)
print(stego_output[:len(stego_output)])

In [ ]:
import math
from itertools import combinations, permutations

class ConsoleColor:
    """ANSI color codes for console output"""
    COLORS = {
        'red': '\033[91m',
        'blue': '\033[94m',
        'green': '\033[92m',
        'yellow': '\033[93m',
        'cyan': '\033[96m',
        'magenta': '\033[95m',
        'orange': '\033[38;5;208m',
        'purple': '\033[38;5;129m',
        'brown': '\033[38;5;130m',
        'gray': '\033[38;5;240m',
        'teal': '\033[38;5;30m',
        'violet': '\033[38;5;177m',
        'pink': '\033[38;5;211m',
        'olive': '\033[38;5;100m',
        'lime': '\033[38;5;154m',
        'reset': '\033[0m'
    }

    @staticmethod
    def color_char(char, color_name):
        """Color a single character for console output"""
        return f"{ConsoleColor.COLORS.get(color_name, '')}{char}{ConsoleColor.COLORS['reset']}"

def binary_to_integer(bitstream):
    return int(bitstream, 2)

def select_color_combination(alpha, n, all_colors):
    combs = list(combinations(all_colors, n))
    return combs[alpha % len(combs)]

def unrank_permutation(n, beta, pi):
    perms = list(permutations(pi[:n]))
    return perms[beta % len(perms)]

def embed_k_block(M, cover_text, n, pi):
    # Predefined color palette
    palette = [
        'red', 'blue', 'green', 'yellow', 'cyan', 'magenta',
        'orange', 'purple', 'brown', 'gray', 'teal', 'violet',
        'pink', 'olive', 'lime'
    ]

    # Calculate block capacities
    B_color = math.comb(len(palette), n)  # Using actual palette size instead of 2^24
    B_perm = math.factorial(n)
    BitsPerBlock = math.floor(math.log2(B_color * B_perm)) if B_color * B_perm > 0 else 0

    # Convert message to binary
    binary_msg = ''.join(format(ord(c), '08b') for c in M)
    k = math.ceil(len(binary_msg) / BitsPerBlock) if BitsPerBlock > 0 else 0
    #print(k)
    padded_msg = binary_msg.ljust(k * BitsPerBlock, '0') if k > 0 else ''

    stego_text = []
    cover_chars = list(cover_text)

    for block in range(k):
        start = block * BitsPerBlock
        chunk = padded_msg[start : start + BitsPerBlock]
        m = binary_to_integer(chunk)

        alpha = m // B_perm
        beta = m % B_perm

        # Get color combination and permutation
        color_comb = select_color_combination(alpha, n, palette)
        perm = unrank_permutation(n, beta, pi)

        # Apply colors to cover text
        for i in range(n):
            pos = block * n + i
            if pos < len(cover_chars):
                color = color_comb[perm[i]]
                stego_text.append(ConsoleColor.color_char(cover_chars[pos], color))
            else:
                break

    # Add remaining uncolored characters
    remaining_pos = k * n
    if remaining_pos < len(cover_chars):
        stego_text.extend(cover_chars[remaining_pos:])

    return ''.join(stego_text)

# Input parameters
secret_message = "The sun dipped below the horizon, painting the sky in shades of orange and purple. A cool breeze rustled the leaves, carrying the scent of fresh rain. Distant laughter echoed from a nearby park as people enjoyed the evening. The city lights flickered to life, casting long shadows on the pavement. A lone jogger passed by, their footsteps rhythmic against the quiet hum of traffic. Somewhere, a dog barked, and a child’s voice called out in excitement. The world felt alive, pulsing with energy even as night fell. Moments like these made everything seem peaceful, fleeting yet perfect."
cover_text = "Only boats catch connotes of the islands sober wines only ships wrap the slips on the cleats of twining lines only flags flap in tags with color that assigns only passage on vessels"
n = 10
pi = list(range(20))  # Permutation key

# Generate and print colored text
colored_output = embed_k_block(secret_message, cover_text, n, pi)
secret = secret_message
# Print the first 181 characters with actual colors in console
print(colored_output[:len(colored_output)])

In [ ]:
import math
from itertools import combinations, permutations

def binary_to_integer(bitstream):
    return int(bitstream, 2)

def select_color_combination(alpha, n, all_colors):
    combs = list(combinations(all_colors, n))
    return combs[alpha % len(combs)]

def unrank_permutation(n, beta, pi):
    perms = list(permutations(pi[:n]))
    return perms[beta % len(perms)]

def embed_k_block(M, cover_text, n, pi):
    # Predefined 24-bit color palette (RGB tuples)
    palette = [
        ('red', (255, 0, 0)), ('blue', (0, 0, 255)), ('green', (0, 255, 0)),
        ('yellow', (255, 255, 0)), ('cyan', (0, 255, 255)), ('magenta', (255, 0, 255)),
        ('orange', (255, 165, 0)), ('purple', (128, 0, 128)), ('brown', (165, 42, 42)),
        ('gray', (128, 128, 128)), ('teal', (0, 128, 128)), ('violet', (238, 130, 238)),
        ('pink', (255, 192, 203)), ('olive', (128, 128, 0)), ('lime', (0, 255, 0))
    ]
    color_names = [c[0] for c in palette]  # Extract color names only

    # Calculate block capacities
    B_color = math.comb(2**24, n)  # Theoretical (16.7M for n=1)
    B_perm = math.factorial(n)
    BitsPerBlock = math.floor(math.log2(B_color * B_perm))

    # Convert message to binary
    binary_msg = ''.join(format(ord(c), '08b') for c in M)
    k = math.ceil(len(binary_msg) / BitsPerBlock)
    padded_msg = binary_msg.ljust(k * BitsPerBlock, '0')

    stego_text = []
    cover_chars = list(cover_text)

    for block in range(k):
        start = block * BitsPerBlock
        chunk = padded_msg[start : start + BitsPerBlock]
        m = binary_to_integer(chunk)

        alpha = m // B_perm
        beta = m % B_perm

        # Get color combination and permutation
        color_comb = select_color_combination(alpha, n, color_names)
        perm = unrank_permutation(n, beta, pi)

        # Apply colors to cover text
        for i in range(n):
            pos = block * n + i
            if pos < len(cover_chars):
                color = color_comb[perm[i]]
                stego_text.append(f'\\textcolor{{{color}}}{{{cover_chars[pos]}}}')
            else:
                break

    # Add remaining uncolored characters
    remaining_pos = k * n
    if remaining_pos < len(cover_chars):
        stego_text.extend(cover_chars[remaining_pos:])

    return ''.join(stego_text)

# Inputs from the problem
secret_message = "behind using a cover text is to hide the presence  of secret messages the presence of embedded messages in the resulting stego-text cannot be easily discovered by anyone except the intended recipient."
cover_text = "in the research area of text steganography, algorithms based on font format have advantages of great capacity, good imperceptibility and wide application range. However, little work on steganalysis for such algorithms has been reported in the literature. Based on the fact that the statistic features of font format will be changed after using font-format-based steganographic algorithms, we present a novel support vector machine-based steganalysis algorithm to detect whether hidden information exists or not. This algorithm can not only effectively detect the existence of hidden information, but also estimate the hidden information length according to variations of font attribute value. As shown by experimental results, the detection accuracy of our algorithm reaches as high as 99.3 \% when the hidden information length is at least 16 bits."
n = 10
pi = list(range(20))  # Permutation key

# Generate stego-text
stego_output = embed_k_block(secret_message, cover_text, n, pi)

# Print first 181 characters (as in the example)
print(stego_output[:len(stego_output)])

In [ ]:
import math
from itertools import combinations, permutations

class ConsoleColor:
    """ANSI color codes for console output"""
    COLORS = {
        'red': '\033[91m',
        'blue': '\033[94m',
        'green': '\033[92m',
        'yellow': '\033[93m',
        'cyan': '\033[96m',
        'magenta': '\033[95m',
        'orange': '\033[38;5;208m',
        'purple': '\033[38;5;129m',
        'brown': '\033[38;5;130m',
        'gray': '\033[38;5;240m',
        'teal': '\033[38;5;30m',
        'violet': '\033[38;5;177m',
        'pink': '\033[38;5;211m',
        'olive': '\033[38;5;100m',
        'lime': '\033[38;5;154m',
        'reset': '\033[0m'
    }

    @staticmethod
    def color_char(char, color_name):
        """Color a single character for console output"""
        return f"{ConsoleColor.COLORS.get(color_name, '')}{char}{ConsoleColor.COLORS['reset']}"

def binary_to_integer(bitstream):
    return int(bitstream, 2)

def select_color_combination(alpha, n, all_colors):
    combs = list(combinations(all_colors, n))
    return combs[alpha % len(combs)]

def unrank_permutation(n, beta, pi):
    perms = list(permutations(pi[:n]))
    return perms[beta % len(perms)]

def embed_k_block(M, cover_text, n, pi):
    # Predefined color palette
    palette = [
        'red', 'blue', 'green', 'yellow', 'cyan', 'magenta',
        'orange', 'purple', 'brown', 'gray', 'teal', 'violet',
        'pink', 'olive', 'lime'
    ]

    # Calculate block capacities
    B_color = math.comb(len(palette), n)  # Using actual palette size instead of 2^24
    B_perm = math.factorial(n)
    BitsPerBlock = math.floor(math.log2(B_color * B_perm)) if B_color * B_perm > 0 else 0

    # Convert message to binary
    binary_msg = ''.join(format(ord(c), '08b') for c in M)
    k = math.ceil(len(binary_msg) / BitsPerBlock) if BitsPerBlock > 0 else 0
    #print(k)
    padded_msg = binary_msg.ljust(k * BitsPerBlock, '0') if k > 0 else ''

    stego_text = []
    cover_chars = list(cover_text)

    for block in range(k):
        start = block * BitsPerBlock
        chunk = padded_msg[start : start + BitsPerBlock]
        m = binary_to_integer(chunk)

        alpha = m // B_perm
        beta = m % B_perm

        # Get color combination and permutation
        color_comb = select_color_combination(alpha, n, palette)
        perm = unrank_permutation(n, beta, pi)

        # Apply colors to cover text
        for i in range(n):
            pos = block * n + i
            if pos < len(cover_chars):
                color = color_comb[perm[i]]
                stego_text.append(ConsoleColor.color_char(cover_chars[pos], color))
            else:
                break

    # Add remaining uncolored characters
    remaining_pos = k * n
    if remaining_pos < len(cover_chars):
        stego_text.extend(cover_chars[remaining_pos:])

    return ''.join(stego_text)

# Input parameters
secret_message = "behind using a cover text is to hide the presence  of secret messages the presence of embedded messages in the resulting stego-text cannot be easily discovered by anyone except the intended recipient."
cover_text = "in the research area of text steganography, algorithms based on font format have advantages of great capacity, good imperceptibility and wide application range. However, little work on steganalysis for such algorithms has been reported in the literature. Based on the fact that the statistic features of font format will be changed after using font-format-based steganographic algorithms, we present a novel support vector machine-based steganalysis algorithm to detect whether hidden information exists or not. This algorithm can not only effectively detect the existence of hidden information, but also estimate the hidden information length according to variations of font attribute value. As shown by experimental results, the detection accuracy of our algorithm reaches as high as 99.3 \% when the hidden information length is at least 16 bits."
n = 10
pi = list(range(20))  # Permutation key

# Generate and print colored text
colored_output = embed_k_block(secret_message, cover_text, n, pi)
secret = secret_message
# Print the first 181 characters with actual colors in console
print(colored_output[:len(colored_output)])

In [ ]:
import math
from itertools import combinations, permutations

def binary_to_integer(bitstream):
    return int(bitstream, 2)

def select_color_combination(alpha, n, all_colors):
    combs = list(combinations(all_colors, n))
    return combs[alpha % len(combs)]

def unrank_permutation(n, beta, pi):
    perms = list(permutations(pi[:n]))
    return perms[beta % len(perms)]

def embed_k_block(M, cover_text, n, pi):
    # Predefined 24-bit color palette (RGB tuples)
    palette = [
        ('red', (255, 0, 0)), ('blue', (0, 0, 255)), ('green', (0, 255, 0)),
        ('yellow', (255, 255, 0)), ('cyan', (0, 255, 255)), ('magenta', (255, 0, 255)),
        ('orange', (255, 165, 0)), ('purple', (128, 0, 128)), ('brown', (165, 42, 42)),
        ('gray', (128, 128, 128)), ('teal', (0, 128, 128)), ('violet', (238, 130, 238)),
        ('pink', (255, 192, 203)), ('olive', (128, 128, 0)), ('lime', (0, 255, 0))
    ]
    color_names = [c[0] for c in palette]  # Extract color names only

    # Calculate block capacities
    B_color = math.comb(2**24, n)  # Theoretical (16.7M for n=1)
    B_perm = math.factorial(n)
    BitsPerBlock = math.floor(math.log2(B_color * B_perm))

    # Convert message to binary
    binary_msg = ''.join(format(ord(c), '08b') for c in M)
    k = math.ceil(len(binary_msg) / BitsPerBlock)
    padded_msg = binary_msg.ljust(k * BitsPerBlock, '0')

    stego_text = []
    cover_chars = list(cover_text)

    for block in range(k):
        start = block * BitsPerBlock
        chunk = padded_msg[start : start + BitsPerBlock]
        m = binary_to_integer(chunk)

        alpha = m // B_perm
        beta = m % B_perm

        # Get color combination and permutation
        color_comb = select_color_combination(alpha, n, color_names)
        perm = unrank_permutation(n, beta, pi)

        # Apply colors to cover text
        for i in range(n):
            pos = block * n + i
            if pos < len(cover_chars):
                color = color_comb[perm[i]]
                stego_text.append(f'\\textcolor{{{color}}}{{{cover_chars[pos]}}}')
            else:
                break

    # Add remaining uncolored characters
    remaining_pos = k * n
    if remaining_pos < len(cover_chars):
        stego_text.extend(cover_chars[remaining_pos:])

    return ''.join(stego_text)

# Inputs from the problem
secret_message = "The field of digital information security continues to evolve rapidly as technological advancements present both new opportunities and challenges. Among various data protection methods, text steganography has emerged as a particularly interesting approach due to its ability to conceal information within seemingly innocuous textual content. This technique differs fundamentally from cryptography, as it focuses on hiding the very existence of secret messages rather than merely scrambling their content. Recent developments in font-based steganography have demonstrated remarkable improvements in capacity and undetectability. These methods leverage subtle modifications to font characteristics such as kerning, glyph shapes, or Unicode variations to embed information without visible alterations to the text. The advantages of such approaches include compatibility with standard word processors, platform independence, and resistance to simple visual inspection. However, the effectiveness of any steganographic system ultimately depends on its ability to withstand steganalysis - the art of detecting hidden information. Current research indicates that machine learning techniques, particularly support vector machines and deep neural networks, show great promise in identifying font-based steganography. These detection methods analyze statistical properties of text formatting that may reveal the presence of hidden data, even when such traces are imperceptible to human observers. The ongoing arms race between steganography and steganalysis drives continuous innovation in both fields. As embedding techniques become more sophisticated, detection methods must correspondingly advance to maintain security standards. This dynamic creates exciting research opportunities in digital forensics, particularly in developing universal detectors capable of identifying multiple steganographic techniques across different text formats. Future directions in text steganography research may explore adaptive algorithms that can modify their embedding patterns based on the surrounding text content, making detection even more challenging. Additionally, the integration of natural language processing techniques could enable more intelligent information hiding that accounts for semantic context rather than just syntactic features. From an application perspective, font-based steganography finds use in various domains including secure communications, copyright protection, and anti-counterfeiting measures. Its relatively low computational requirements."
cover_text = "in the research area of text steganography, algorithms based on font format have advantages of great capacity, good imperceptibility and wide application range. However, little work on steganalysis for such algorithms has been reported in the literature. Based on the fact that the statistic features of font format will be changed after using font-format-based steganographic algorithms, we present a novel support vector machine-based steganalysis algorithm to detect whether hidden information exists or not. This algorithm can not only effectively detect the existence of hidden information, but also estimate the hidden information length according to variations of font attribute values. As shown by experimental results, the detection accuracy of our algorithm reaches as high as 99.3 \%, when the hidden information length is at least 16 bits."
n = 10
pi = list(range(20))  # Permutation key

# Generate stego-text
stego_output = embed_k_block(secret_message, cover_text, n, pi)

# Print first 181 characters (as in the example)
print(stego_output[:len(stego_output)])

In [ ]:
import math
from itertools import combinations, permutations

class ConsoleColor:
    """ANSI color codes for console output"""
    COLORS = {
        'red': '\033[91m',
        'blue': '\033[94m',
        'green': '\033[92m',
        'yellow': '\033[93m',
        'cyan': '\033[96m',
        'magenta': '\033[95m',
        'orange': '\033[38;5;208m',
        'purple': '\033[38;5;129m',
        'brown': '\033[38;5;130m',
        'gray': '\033[38;5;240m',
        'teal': '\033[38;5;30m',
        'violet': '\033[38;5;177m',
        'pink': '\033[38;5;211m',
        'olive': '\033[38;5;100m',
        'lime': '\033[38;5;154m',
        'reset': '\033[0m'
    }

    @staticmethod
    def color_char(char, color_name):
        """Color a single character for console output"""
        return f"{ConsoleColor.COLORS.get(color_name, '')}{char}{ConsoleColor.COLORS['reset']}"

def binary_to_integer(bitstream):
    return int(bitstream, 2)

def select_color_combination(alpha, n, all_colors):
    combs = list(combinations(all_colors, n))
    return combs[alpha % len(combs)]

def unrank_permutation(n, beta, pi):
    perms = list(permutations(pi[:n]))
    return perms[beta % len(perms)]

def embed_k_block(M, cover_text, n, pi):
    # Predefined color palette
    palette = [
        'red', 'blue', 'green', 'yellow', 'cyan', 'magenta',
        'orange', 'purple', 'brown', 'gray', 'teal', 'violet',
        'pink', 'olive', 'lime'
    ]

    # Calculate block capacities
    B_color = math.comb(len(palette), n)  # Using actual palette size instead of 2^24
    B_perm = math.factorial(n)
    BitsPerBlock = math.floor(math.log2(B_color * B_perm)) if B_color * B_perm > 0 else 0

    # Convert message to binary
    binary_msg = ''.join(format(ord(c), '08b') for c in M)
    k = math.ceil(len(binary_msg) / BitsPerBlock) if BitsPerBlock > 0 else 0
    #print(k)
    padded_msg = binary_msg.ljust(k * BitsPerBlock, '0') if k > 0 else ''

    stego_text = []
    cover_chars = list(cover_text)

    for block in range(k):
        start = block * BitsPerBlock
        chunk = padded_msg[start : start + BitsPerBlock]
        m = binary_to_integer(chunk)

        alpha = m // B_perm
        beta = m % B_perm

        # Get color combination and permutation
        color_comb = select_color_combination(alpha, n, palette)
        perm = unrank_permutation(n, beta, pi)

        # Apply colors to cover text
        for i in range(n):
            pos = block * n + i
            if pos < len(cover_chars):
                color = color_comb[perm[i]]
                stego_text.append(ConsoleColor.color_char(cover_chars[pos], color))
            else:
                break

    # Add remaining uncolored characters
    remaining_pos = k * n
    if remaining_pos < len(cover_chars):
        stego_text.extend(cover_chars[remaining_pos:])

    return ''.join(stego_text)

# Input parameters
secret_message = "The field of digital information security continues to evolve rapidly as technological advancements present both new opportunities and challenges. Among various data protection methods, text steganography has emerged as a particularly interesting approach due to its ability to conceal information within seemingly innocuous textual content. This technique differs fundamentally from cryptography, as it focuses on hiding the very existence of secret messages rather than merely scrambling their content. Recent developments in font-based steganography have demonstrated remarkable improvements in capacity and undetectability. These methods leverage subtle modifications to font characteristics such as kerning, glyph shapes, or Unicode variations to embed information without visible alterations to the text. The advantages of such approaches include compatibility with standard word processors, platform independence, and resistance to simple visual inspection. However, the effectiveness of any steganographic system ultimately depends on its ability to withstand steganalysis - the art of detecting hidden information. Current research indicates that machine learning techniques, particularly support vector machines and deep neural networks, show great promise in identifying font-based steganography. These detection methods analyze statistical properties of text formatting that may reveal the presence of hidden data, even when such traces are imperceptible to human observers. The ongoing arms race between steganography and steganalysis drives continuous innovation in both fields. As embedding techniques become more sophisticated, detection methods must correspondingly advance to maintain security standards. This dynamic creates exciting research opportunities in digital forensics, particularly in developing universal detectors capable of identifying multiple steganographic techniques across different text formats. Future directions in text steganography research may explore adaptive algorithms that can modify their embedding patterns based on the surrounding text content, making detection even more challenging. Additionally, the integration of natural language processing techniques could enable more intelligent information hiding that accounts for semantic context rather than just syntactic features. From an application perspective, font-based steganography finds use in various domains including secure communications, copyright protection, and anti-counterfeiting measures. Its relatively low computational requirements."
cover_text = "in the research area of text steganography, algorithms based on font format have advantages of great capacity, good imperceptibility and wide application range. However, little work on steganalysis for such algorithms has been reported in the literature. Based on the fact that the statistic features of font format will be changed after using font-format-based steganographic algorithms, we present a novel support vector machine-based steganalysis algorithm to detect whether hidden information exists or not. This algorithm can not only effectively detect the existence of hidden information, but also estimate the hidden information length according to variations of font attribute values. As shown by experimental results, the detection accuracy of our algorithm reaches as high as 99.3 \%, when the hidden information length is at least 16 bits."
n = 10
pi = list(range(20))  # Permutation key

# Generate and print colored text
colored_output = embed_k_block(secret_message, cover_text, n, pi)
secret = secret_message
# Print the first 181 characters with actual colors in console
print(colored_output[:len(colored_output)])

#Stego text with Huffman Coding

In [ ]:
import math
from itertools import combinations, permutations
from collections import Counter, deque
import heapq

class ConsoleColor:
    """ANSI color codes for console output"""
    COLORS = {
        'red': '\033[91m',
        'blue': '\033[94m',
        'green': '\033[92m',
        'yellow': '\033[93m',
        'cyan': '\033[96m',
        'magenta': '\033[95m',
        'orange': '\033[38;5;208m',
        'purple': '\033[38;5;129m',
        'brown': '\033[38;5;130m',
        'gray': '\033[38;5;240m',
        'teal': '\033[38;5;30m',
        'violet': '\033[38;5;177m',
        'pink': '\033[38;5;211m',
        'olive': '\033[38;5;100m',
        'lime': '\033[38;5;154m',
        'reset': '\033[0m'
    }

    @staticmethod
    def color_char(char, color_name):
        """Color a single character for console output"""
        return f"{ConsoleColor.COLORS.get(color_name, '')}{char}{ConsoleColor.COLORS['reset']}"

class HuffmanNode:
    def __init__(self, char, freq):
        self.char = char
        self.freq = freq
        self.left = None
        self.right = None

    def __lt__(self, other):
        return self.freq < other.freq

def build_huffman_tree(text):
    """Build Huffman tree from text"""
    if not text:
        return None

    # Count character frequency
    frequency = Counter(text)

    # Create initial nodes
    heap = [HuffmanNode(char, freq) for char, freq in frequency.items()]
    heapq.heapify(heap)

    # Build tree
    while len(heap) > 1:
        left = heapq.heappop(heap)
        right = heapq.heappop(heap)

        merged = HuffmanNode(None, left.freq + right.freq)
        merged.left = left
        merged.right = right

        heapq.heappush(heap, merged)

    return heap[0] if heap else None

def build_huffman_codes(node, current_code="", codes=None):
    """Build Huffman codes dictionary"""
    if codes is None:
        codes = {}

    if node is None:
        return codes

    # If it's a leaf, add the code
    if node.char is not None:
        codes[node.char] = current_code

    # Recursive traversal
    build_huffman_codes(node.left, current_code + "0", codes)
    build_huffman_codes(node.right, current_code + "1", codes)

    return codes

def huffman_compress(text):
    """Compress text using Huffman coding"""
    if not text:
        return "", {}

    # Build tree and codes
    root = build_huffman_tree(text)
    huffman_codes = build_huffman_codes(root)

    # Compress text
    compressed_bits = ''.join(huffman_codes[char] for char in text)

    return compressed_bits, huffman_codes

def huffman_decompress(compressed_bits, huffman_codes):
    """Decompress bits using Huffman coding"""
    if not compressed_bits:
        return ""

    # Reverse codes dictionary
    reverse_codes = {code: char for char, code in huffman_codes.items()}

    # Decompress
    current_code = ""
    decompressed_text = []

    for bit in compressed_bits:
        current_code += bit
        if current_code in reverse_codes:
            decompressed_text.append(reverse_codes[current_code])
            current_code = ""

    return ''.join(decompressed_text)

def binary_to_integer(bitstream):
    return int(bitstream, 2)

def select_color_combination(alpha, n, all_colors):
    combs = list(combinations(all_colors, n))
    return combs[alpha % len(combs)]

def unrank_permutation(n, beta, pi):
    perms = list(permutations(pi[:n]))
    return perms[beta % len(perms)]

def embed_k_block(M, cover_text, n, pi):
    # Predefined color palette
    palette = [
        'red', 'blue', 'green', 'yellow', 'cyan', 'magenta',
        'orange', 'purple', 'brown', 'gray', 'teal', 'violet',
        'pink', 'olive', 'lime'
    ]

    # Calculate block capacities
    B_color = math.comb(len(palette), n)  # Using actual palette size instead of 2^24
    B_perm = math.factorial(n)
    BitsPerBlock = math.floor(math.log2(B_color * B_perm)) if B_color * B_perm > 0 else 0

    # Convert message to binary
    binary_msg = ''.join(format(ord(c), '08b') for c in M)
    k = math.ceil(len(binary_msg) / BitsPerBlock) if BitsPerBlock > 0 else 0
    padded_msg = binary_msg.ljust(k * BitsPerBlock, '0') if k > 0 else ''

    stego_text = []
    cover_chars = list(cover_text)

    for block in range(k):
        start = block * BitsPerBlock
        chunk = padded_msg[start : start + BitsPerBlock]
        m = binary_to_integer(chunk)

        alpha = m // B_perm
        beta = m % B_perm

        # Get color combination and permutation
        color_comb = select_color_combination(alpha, n, palette)
        perm = unrank_permutation(n, beta, pi)

        # Apply colors to cover text
        for i in range(n):
            pos = block * n + i
            if pos < len(cover_chars):
                color = color_comb[perm[i]]
                stego_text.append(ConsoleColor.color_char(cover_chars[pos], color))
            else:
                break

    # Add remaining uncolored characters
    remaining_pos = k * n
    if remaining_pos < len(cover_chars):
        stego_text.extend(cover_chars[remaining_pos:])

    return ''.join(stego_text)

def embed_k_block_with_compression(M, cover_text, n, pi):
    """Version with Huffman compression of cover text"""

    print("=== HUFFMAN COMPRESSION ===")
    print(f"Original cover text: {len(cover_text)} characters")

    # Compress cover text with Huffman
    compressed_bits, huffman_codes = huffman_compress(cover_text)

    print(f"Compressed text: {len(compressed_bits)} bits")
    print(f"Compression ratio: {len(compressed_bits) / (len(cover_text) * 8):.2%}")
    print(f"Number of Huffman codes: {len(huffman_codes)}")

    # Convert compressed bits to character string for steganography
    # We use '0' and '1' as characters to represent bits
    compressed_text = compressed_bits

    print(f"Text for steganography: {len(compressed_text)} 'characters' (bits)")

    # Apply steganography on compressed text
    colored_output = embed_k_block(M, compressed_text, n, pi)

    return colored_output, huffman_codes, compressed_bits

def decompress_and_extract(colored_text, huffman_codes, original_cover_length):
    """Decompress colored text and extract message"""
    # This function would be used to extract message and decompress
    # For now, we focus on compression
    pass

# Input parameters
secret_message = "underlying physiological mechanisms"
cover_text = "Only boats catch connotes of the islands sober wines only ships wrap the slips on the cleats of twining lines only flags flap in tags with color that assigns only passage on vessels"
n = 10
pi = list(range(20))  # Permutation key

print("=== WITHOUT COMPRESSION ===")
colored_output_original = embed_k_block(secret_message, cover_text, n, pi)
print(colored_output_original[:len(colored_output_original)] + "...")
print(f"Output length: {len(colored_output_original)} characters (with color codes)")

print("\n" + "="*50 + "\n")

# Version with Huffman compression
colored_output_compressed, huffman_codes, compressed_bits = embed_k_block_with_compression(
    secret_message, cover_text, n, pi
)

print("\n=== WITH HUFFMAN COMPRESSION ===")
print(colored_output_compressed[:len(colored_output_compressed)] + "...")
print(f"Output length: {len(colored_output_compressed)} characters (with color codes)")

# Decompression verification
print("\n=== DECOMPRESSION VERIFICATION ===")
decompressed_text = huffman_decompress(compressed_bits, huffman_codes)
print(f"Decompression successful: {decompressed_text == cover_text}")
print(f"Original text: {cover_text[:50]}...")
print(f"Decompressed text: {decompressed_text[:50]}...")

# Compression statistics display
print("\n=== HUFFMAN STATISTICS ===")
print("Generated Huffman codes:")
for char, code in list(huffman_codes.items())[:10]:  # Display first 10
    char_display = repr(char) if char in ' \n\t' else char
    print(f"  '{char_display}': {code}")
if len(huffman_codes) > 10:
    print(f"  ... and {len(huffman_codes) - 10} others")

In [ ]:
import nltk
nltk.download('averaged_perceptron_tagger_eng')

In [ ]:
import nltk
nltk.download('punkt_tab')

# Structural Analysis for Text Steganalysis(Aziz and Bukhelli (2023))

In [ ]:
import numpy as np
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC, SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve, confusion_matrix, classification_report
import nltk
from nltk import pos_tag
from nltk.tokenize import sent_tokenize, word_tokenize
import matplotlib.pyplot as plt
import seaborn as sns
import os
from docx import Document
import glob
import warnings
from collections import Counter
import colorsys
warnings.filterwarnings('ignore')

# Download required NLTK data
nltk.download('punkt', quiet=True)
nltk.download('averaged_perceptron_tagger', quiet=True)

class ColorSteganalysis:
    def __init__(self):
        # Updated features for color-based steganalysis
        self.features = [
            'color_count', 'unique_colors_ratio', 'color_entropy', 'color_variance',
            'rgb_correlation', 'luminance_variance', 'hue_distribution',
            'saturation_mean', 'brightness_std', 'color_transitions',
            'color_clusters', 'perceptual_distance_mean', 'color_frequency_skew',
            'adjacent_color_similarity', 'color_gradient_smoothness'
        ]

        self.models = {
            'k-NN': KNeighborsClassifier(n_neighbors=3),
            'LSVM': LinearSVC(random_state=42, dual=False, max_iter=1000),
            'RBFSVM': SVC(kernel='rbf', random_state=42, probability=True),
            'DT': DecisionTreeClassifier(random_state=42, max_depth=10),
            'RF': RandomForestClassifier(random_state=42, n_estimators=50),
            'NN': MLPClassifier(random_state=42, max_iter=1000, hidden_layer_sizes=(50,)),
            'AdaBoost': AdaBoostClassifier(random_state=42, n_estimators=50),
            'NB': GaussianNB(),
            'QDA': QuadraticDiscriminantAnalysis()
        }
        self.results = {}
        self.scaler = StandardScaler()
        self.dataset_stats = {}

    def rgb_to_hsv(self, r, g, b):
        """Convert RGB to HSV color space"""
        r, g, b = r/255.0, g/255.0, b/255.0
        h, s, v = colorsys.rgb_to_hsv(r, g, b)
        return h, s, v

    def rgb_to_luminance(self, r, g, b):
        """Calculate luminance from RGB"""
        return 0.299 * r + 0.587 * g + 0.114 * b

    def color_difference(self, color1, color2):
        """Calculate perceptual color difference (simplified Delta E)"""
        r1, g1, b1 = color1
        r2, g2, b2 = color2
        return np.sqrt((r1 - r2)**2 + (g1 - g2)**2 + (b1 - b2)**2)

    def extract_color_features(self, text_colors):
        """Extract color-based features for steganalysis"""
        if not text_colors or len(text_colors) < 2:
            return [0] * len(self.features)

        colors = np.array(text_colors)

        # Basic color statistics
        color_count = len(colors)
        unique_colors = len(set(map(tuple, colors)))
        unique_colors_ratio = unique_colors / color_count if color_count > 0 else 0

        # Color entropy
        color_counts = Counter(map(tuple, colors))
        color_probs = [count/color_count for count in color_counts.values()]
        color_entropy = -sum(p * np.log2(p) for p in color_probs) if color_probs else 0

        # RGB statistics
        rgb_variance = np.var(colors, axis=0).mean()
        rgb_correlation = np.corrcoef(colors.T)[0, 1] if len(colors) > 1 else 0  # R-G correlation

        # Convert to HSV
        hsv_colors = np.array([self.rgb_to_hsv(r, g, b) for r, g, b in colors])
        hues, saturations, values = hsv_colors.T

        # HSV statistics
        hue_distribution = np.std(hues) if len(hues) > 1 else 0
        saturation_mean = np.mean(saturations)
        brightness_std = np.std(values)

        # Luminance
        luminances = [self.rgb_to_luminance(r, g, b) for r, g, b in colors]
        luminance_variance = np.var(luminances) if luminances else 0

        # Color transitions
        color_transitions = 0
        adjacent_similarity = 0
        perceptual_distances = []

        for i in range(len(colors) - 1):
            diff = self.color_difference(colors[i], colors[i + 1])
            perceptual_distances.append(diff)
            if diff > 10:  # Threshold for significant color change
                color_transitions += 1
            adjacent_similarity += 1 / (1 + diff)  # Inverse similarity

        color_transitions = color_transitions / (len(colors) - 1) if len(colors) > 1 else 0
        adjacent_similarity = adjacent_similarity / (len(colors) - 1) if len(colors) > 1 else 0
        perceptual_distance_mean = np.mean(perceptual_distances) if perceptual_distances else 0

        # Color clustering (simplified)
        from sklearn.cluster import KMeans
        try:
            if len(colors) >= 3:
                kmeans = KMeans(n_clusters=min(3, len(colors)), random_state=42, n_init=10)
                clusters = kmeans.fit_predict(colors)
                color_clusters = len(np.unique(clusters)) / len(colors)
            else:
                color_clusters = 1.0
        except:
            color_clusters = 1.0

        # Color frequency skew
        if len(color_counts) > 1:
            frequencies = list(color_counts.values())
            color_frequency_skew = np.std(frequencies) / np.mean(frequencies)
        else:
            color_frequency_skew = 0

        # Color gradient smoothness
        gradient_smoothness = 0
        if len(perceptual_distances) > 1:
            gradient_changes = np.diff(perceptual_distances)
            gradient_smoothness = 1 / (1 + np.std(gradient_changes))

        return [
            color_count, unique_colors_ratio, color_entropy, rgb_variance,
            rgb_correlation, luminance_variance, hue_distribution,
            saturation_mean, brightness_std, color_transitions,
            color_clusters, perceptual_distance_mean, color_frequency_skew,
            adjacent_similarity, gradient_smoothness
        ]

    def simulate_color_embedding(self, text, secret_message, n_colors=10):
        """
        Simulate the combinatorial color-permutation embedding from the paper
        """
        # For simulation purposes, we'll create artificial color patterns
        # that mimic the combinatorial encoding described in the paper

        words = word_tokenize(text)
        if len(words) < n_colors:
            return text, []  # Not enough words for embedding

        # Generate base colors (natural looking)
        base_colors = [
            (0, 0, 0), (50, 50, 50), (100, 100, 100), (150, 150, 150),
            (200, 200, 200), (255, 255, 255)
        ]

        # Generate steganographic colors (combinatorial selection)
        stego_colors = []
        for i in range(n_colors):
            # Create colors with specific combinatorial patterns
            r = (i * 25) % 256
            g = ((i * 17) + 85) % 256
            b = ((i * 13) + 170) % 256
            stego_colors.append((r, g, b))

        # Apply embedding pattern based on secret message
        embedded_colors = []
        color_sequence = []

        for i, word in enumerate(words):
            if i < len(secret_message):
                bit = secret_message[i % len(secret_message)]
                if bit == '1':
                    # Use steganographic color
                    color_idx = i % len(stego_colors)
                    embedded_colors.append(stego_colors[color_idx])
                    color_sequence.append(stego_colors[color_idx])
                else:
                    # Use natural color
                    color_idx = i % len(base_colors)
                    embedded_colors.append(base_colors[color_idx])
                    color_sequence.append(base_colors[color_idx])
            else:
                # Use natural color for remaining words
                color_idx = i % len(base_colors)
                embedded_colors.append(base_colors[color_idx])
                color_sequence.append(base_colors[color_idx])

        return text, color_sequence

    def create_dataset_from_text(self, text, num_samples=50, n_colors=10):
        """Create dataset with color-based features"""
        features = []
        labels = []

        words = word_tokenize(text)
        if len(words) < 20:  # Minimum text length
            return None, None

        samples_created = 0
        max_attempts = num_samples * 2

        while samples_created < num_samples and len(features) < max_attempts:
            # Select random text segment
            start_idx = np.random.randint(0, max(1, len(words) - 20))
            end_idx = start_idx + np.random.randint(20, min(100, len(words) - start_idx))
            text_segment = ' '.join(words[start_idx:end_idx])

            # Clean text (no embedding)
            clean_colors = []
            for i in range(len(words[start_idx:end_idx])):
                # Use natural color distribution
                if np.random.random() < 0.8:  # 80% black text
                    clean_colors.append((0, 0, 0))
                else:  # 20% other natural colors
                    clean_colors.append((np.random.randint(50, 150),
                                       np.random.randint(50, 150),
                                       np.random.randint(50, 150)))

            clean_features = self.extract_color_features(clean_colors)
            if np.sum(clean_features) > 0:  # Valid features
                features.append(clean_features)
                labels.append(0)  # Clean
                samples_created += 1

            # Embedded text
            secret_message = ''.join(np.random.choice(['0', '1'], size=min(20, len(words[start_idx:end_idx]))))
            _, embedded_colors = self.simulate_color_embedding(text_segment, secret_message, n_colors)

            embedded_features = self.extract_color_features(embedded_colors)
            if np.sum(embedded_features) > 0:  # Valid features
                features.append(embedded_features)
                labels.append(1)  # Embedded
                samples_created += 1

        print(f"Created {samples_created} color-based samples")
        return np.array(features), np.array(labels)

    def print_dataset_statistics(self, X, y, n_colors):
        """Print detailed statistics about the dataset"""
        print(f"\n{'='*60}")
        print(f"COLOR STEGANALYSIS DATASET - {n_colors} COLORS")
        print(f"{'='*60}")

        clean_samples = np.sum(y == 0)
        embedded_samples = np.sum(y == 1)
        total_samples = len(y)

        print(f"Total samples: {total_samples}")
        print(f"Clean samples: {clean_samples} ({clean_samples/total_samples*100:.1f}%)")
        print(f"Embedded samples: {embedded_samples} ({embedded_samples/total_samples*100:.1f}%)")

        if total_samples > 0:
            # Feature statistics
            feature_means = np.mean(X, axis=0)
            feature_stds = np.std(X, axis=0)

            print(f"\nColor Feature Statistics:")
            print(f"{'Feature':<25} {'Mean':<10} {'Std':<10}")
            print("-" * 45)

            for i, feature_name in enumerate(self.features):
                print(f"{feature_name:<25} {feature_means[i]:<10.3f} {feature_stds[i]:<10.3f}")

    def train_models(self, X_train, y_train):
        """Train all models"""
        # Standardize features
        X_train_scaled = self.scaler.fit_transform(X_train)

        for name, model in self.models.items():
            print(f"Training {name}...")
            try:
                model.fit(X_train_scaled, y_train)
            except Exception as e:
                print(f"Error training {name}: {e}")

        return X_train_scaled

    def evaluate_models(self, X_test, y_test):
        """Evaluate all trained models"""
        results = {}
        X_test_scaled = self.scaler.transform(X_test)

        for name, model in self.models.items():
            try:
                # Predict
                y_pred = model.predict(X_test_scaled)

                # Calculate metrics
                accuracy = accuracy_score(y_test, y_pred)
                precision = precision_score(y_test, y_pred, zero_division=0)
                recall = recall_score(y_test, y_pred, zero_division=0)
                f1 = f1_score(y_test, y_pred, zero_division=0)

                # For models that support probability, calculate AUC
                if hasattr(model, 'predict_proba'):
                    y_prob = model.predict_proba(X_test_scaled)[:, 1]
                    auc = roc_auc_score(y_test, y_prob)
                else:
                    # For SVM without probability, use decision function
                    try:
                        y_prob = model.decision_function(X_test_scaled)
                        auc = roc_auc_score(y_test, y_prob)
                    except:
                        auc = 0.5
                        y_prob = None

                # Confusion matrix
                cm = confusion_matrix(y_test, y_pred)
                tn, fp, fn, tp = cm.ravel()

                results[name] = {
                    'accuracy': accuracy,
                    'precision': precision,
                    'recall': recall,
                    'f1_score': f1,
                    'auc': auc,
                    'model': model,
                    'predictions': y_pred,
                    'probabilities': y_prob,
                    'confusion_matrix': cm,
                    'true_negative': tn,
                    'false_positive': fp,
                    'false_negative': fn,
                    'true_positive': tp
                }

                print(f"{name}: "
                      f"Accuracy={accuracy:.3f}, "
                      f"Precision={precision:.3f}, "
                      f"Recall={recall:.3f}, "
                      f"F1={f1:.3f}, "
                      f"AUC={auc:.3f}")

            except Exception as e:
                print(f"Error evaluating {name}: {e}")
                continue

        self.results = results
        return results

    def plot_roc_curves(self, X_test, y_test, n_colors):
        """Plot ROC curves for all models"""
        plt.figure(figsize=(10, 8))
        X_test_scaled = self.scaler.transform(X_test)

        for name, result in self.results.items():
            model = result['model']
            y_prob = result['probabilities']

            if y_prob is not None:
                fpr, tpr, _ = roc_curve(y_test, y_prob)
                auc = result['auc']
                plt.plot(fpr, tpr, linewidth=2, label=f'{name} (AUC = {auc:.3f})')

        # Plot random guess line
        plt.plot([0, 1], [0, 1], 'k--', alpha=0.5, label='Random Guess (AUC = 0.500)')

        plt.xlabel('False Positive Rate', fontsize=12)
        plt.ylabel('True Positive Rate', fontsize=12)
        plt.title(f'ROC Curves - Color Steganalysis ({n_colors} colors)', fontsize=14)
        plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.show()

    def plot_confusion_matrices(self, X_test, y_test, n_colors):
        """Plot confusion matrices for all models"""
        n_models = len(self.results)
        if n_models == 0:
            return

        cols = 3
        rows = (n_models + cols - 1) // cols
        fig, axes = plt.subplots(rows, cols, figsize=(15, 5*rows))

        if rows == 1:
            axes = axes.reshape(1, -1)
        elif cols == 1:
            axes = axes.reshape(-1, 1)

        axes = axes.ravel()

        for idx, (name, result) in enumerate(self.results.items()):
            if idx >= len(axes):
                break

            cm = result['confusion_matrix']
            accuracy = result['accuracy']

            sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[idx],
                       xticklabels=['Clean', 'Stego'],
                       yticklabels=['Clean', 'Stego'])
            axes[idx].set_title(f'{name}\nAccuracy: {accuracy:.3f}')
            axes[idx].set_xlabel('Predicted')
            axes[idx].set_ylabel('Actual')

        # Hide empty subplots
        for idx in range(len(self.results), len(axes)):
            axes[idx].set_visible(False)

        plt.suptitle(f'Confusion Matrices - Color Steganalysis ({n_colors} colors)', fontsize=16)
        plt.tight_layout()
        plt.show()

    def print_detailed_results(self, n_colors):
        """Print detailed results and statistics"""
        print(f"\n{'='*70}")
        print(f"DETAILED RESULTS - COLOR STEGANALYSIS ({n_colors} COLORS)")
        print(f"{'='*70}")

        if not self.results:
            print("No results to display!")
            return

        # Create results table
        results_data = []
        for name, result in self.results.items():
            results_data.append({
                'Model': name,
                'Accuracy': f"{result['accuracy']:.4f}",
                'Precision': f"{result['precision']:.4f}",
                'Recall': f"{result['recall']:.4f}",
                'F1-Score': f"{result['f1_score']:.4f}",
                'AUC': f"{result['auc']:.4f}",
                'TP': result['true_positive'],
                'FP': result['false_positive'],
                'TN': result['true_negative'],
                'FN': result['false_negative']
            })

        results_df = pd.DataFrame(results_data)
        print("\nPerformance Metrics:")
        print(results_df.to_string(index=False))

        # Best models
        if len(self.results) > 0:
            best_acc = max(self.results.items(), key=lambda x: x[1]['accuracy'])
            best_auc = max(self.results.items(), key=lambda x: x[1]['auc'])
            best_f1 = max(self.results.items(), key=lambda x: x[1]['f1_score'])

            print(f"\nBest Models:")
            print(f"  Accuracy: {best_acc[0]} ({best_acc[1]['accuracy']:.4f})")
            print(f"  AUC:      {best_auc[0]} ({best_auc[1]['auc']:.4f})")
            print(f"  F1-Score: {best_f1[0]} ({best_f1[1]['f1_score']:.4f})")

    def analyze_directory(self, directory_path, n_colors=10):
        """Analyze all DOCX files in a directory for color steganography"""
        print(f"\n{'='*70}")
        print(f"COLOR STEGANALYSIS WITH {n_colors} COLORS")
        print(f"{'='*70}")
        print(f"Directory: {directory_path}")

        # Find all DOCX files in the directory
        docx_files = glob.glob(os.path.join(directory_path, "*.docx"))

        if not docx_files:
            print("No DOCX files found in the directory!")
            return

        print(f"Found {len(docx_files)} DOCX files")

        all_features = []
        all_labels = []
        files_processed = 0

        for file_path in docx_files:
            print(f"\nProcessing: {os.path.basename(file_path)}")

            # Load DOCX file
            try:
                doc = Document(file_path)
                text = ""
                for paragraph in doc.paragraphs:
                    if paragraph.text.strip():
                        text += paragraph.text + "\n"
            except Exception as e:
                print(f"  Failed to load {file_path}: {e}")
                continue

            print(f"  Text length: {len(text)} characters")

            # Create dataset from this file
            X, y = self.create_dataset_from_text(text, num_samples=20, n_colors=n_colors)
            if X is not None and len(X) > 0:
                all_features.append(X)
                all_labels.append(y)
                files_processed += 1
                print(f"  Added {len(X)} color-based samples")

        if not all_features:
            print("No valid data collected from any files!")
            return

        # Combine all data
        X_combined = np.vstack(all_features)
        y_combined = np.hstack(all_labels)

        print(f"\nDataset Summary:")
        print(f"  Files processed: {files_processed}/{len(docx_files)}")
        print(f"  Total samples: {len(X_combined)}")
        print(f"  Clean samples: {np.sum(y_combined == 0)}")
        print(f"  Stego samples: {np.sum(y_combined == 1)}")

        if len(X_combined) < 20:
            print(f"Warning: Very small dataset ({len(X_combined)} samples). Results may not be reliable.")
            return

        # Print dataset statistics
        self.print_dataset_statistics(X_combined, y_combined, n_colors)

        # Split data
        X_train, X_test, y_train, y_test = train_test_split(
            X_combined, y_combined, test_size=0.3, random_state=42, stratify=y_combined
        )

        print(f"\nTraining set: {len(X_train)} samples")
        print(f"Test set: {len(X_test)} samples")

        # Train models
        print("\nTraining models...")
        X_train_scaled = self.train_models(X_train, y_train)

        # Evaluate models
        print("\nModel Performance:")
        print("-" * 50)
        results = self.evaluate_models(X_test, y_test)

        if not results:
            print("No models were successfully evaluated!")
            return

        # Plot results
        self.plot_roc_curves(X_test, y_test, n_colors)
        self.plot_confusion_matrices(X_test, y_test, n_colors)

        # Print detailed results
        self.print_detailed_results(n_colors)

        return results

def main():
    # Initialize the color steganalysis system
    steganalysis = ColorSteganalysis()

    # Directory containing DOCX files
    directory_path = '/content/gdrive/MyDrive/DatasetsEvaluations/NewArticleCorpusStego'

    # Test with different color counts as mentioned in the paper
    color_counts = [8, 10, 16, 24, 32]

    all_results = {}

    for n_colors in color_counts:
        print(f"\n{'#'*80}")
        print(f"STARTING COLOR STEGANALYSIS WITH {n_colors} COLORS")
        print(f"{'#'*80}")

        results = steganalysis.analyze_directory(directory_path, n_colors=n_colors)
        all_results[n_colors] = results

        print(f"\nCompleted analysis for {n_colors} colors")
        print(f"{'#'*80}")

    # Print summary table similar to paper's Table 14
    print("\n" + "="*100)
    print("SUMMARY OF COLOR STEGANALYSIS RESULTS (AUC-ROC SCORES)")
    print("="*100)

    summary_data = []
    for n_colors, results in all_results.items():
        if results:
            row = {'Colors': n_colors}
            for model_name, result in results.items():
                row[model_name] = f"{result['auc']:.3f}"
            summary_data.append(row)

    if summary_data:
        summary_df = pd.DataFrame(summary_data)
        print("\nAUC-ROC Scores by Model and Color Count:")
        print(summary_df.to_string(index=False))

    print("\nALL COLOR STEGANALYSES COMPLETED")
    print("="*100)

if __name__ == "__main__":
    main()

In [ ]:
import nltk
nltk.download('punkt_tab')

# Hierarchical Text Steganalysis (Peng et al., 2023)
# Deep learning approach for text steganography detection using hierarchical representation learning

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pandas as pd
import re
import os
import glob
import math
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from collections import Counter
from docx import Document
import warnings
warnings.filterwarnings('ignore')

# Download required NLTK data
nltk.download('punkt', quiet=True)
nltk.download('stopwords', quiet=True)

class ColorSteganographySimulator:
    """Simulate the high-capacity color-based steganography from the research paper"""

    def __init__(self, n_colors=10, color_space_bits=24):
        self.n_colors = n_colors
        self.color_space_bits = color_space_bits
        self.total_colors = 2 ** color_space_bits  # 16,777,216 colors

        # Pre-calculate combinatorial capacities
        self.color_combination_space = self.calculate_combination_space(n_colors)
        self.permutation_space = math.factorial(n_colors)
        self.total_capacity = np.floor(np.log2(self.color_combination_space * self.permutation_space))

        print(f"Color Steganography Simulator initialized:")
        print(f"  Colors per block (n): {n_colors}")
        print(f"  Color combinations: 2^{np.log2(self.color_combination_space):.1f}")
        print(f"  Permutations: 2^{np.log2(self.permutation_space):.1f}")
        print(f"  Total capacity: {self.total_capacity} bits per block")
        print(f"  Theoretical improvement: ~11.4x over permutation-only methods")

    def calculate_combination_space(self, n):
        """Calculate binomial coefficient C(2^24, n) using approximation"""
        N = 2 ** self.color_space_bits
        # Using Stirling's approximation for large numbers
        if n > 0 and n < N:
            return np.exp(n * np.log(N) - n * np.log(n) - 0.5 * np.log(2 * np.pi * n))
        return 0

    def simulate_color_embedding(self, text, embedding_rate=0.08):
        """
        Simulate the combinatorial color-permutation embedding
        Based on the research paper's approach
        """
        words = word_tokenize(text)
        if len(words) < self.n_colors:
            return text, 0

        # Calculate how many characters to color (8-11% as per paper)
        chars_to_color = max(self.n_colors, int(len(text) * embedding_rate))

        # Simulate color combination selection
        color_combination_idx = np.random.randint(0, min(1000, int(self.color_combination_space)))

        # Simulate permutation encoding
        permutation_idx = np.random.randint(0, self.permutation_space)

        # Mark positions that would be colored in real implementation
        colored_positions = np.random.choice(len(text), min(chars_to_color, len(text)), replace=False)

        # Create simulated stego text (in real implementation, colors would be applied)
        stego_text = text
        embedding_info = {
            'colored_positions': len(colored_positions),
            'total_positions': len(text),
            'coverage_percentage': len(colored_positions) / len(text) * 100,
            'color_combination_idx': color_combination_idx,
            'permutation_idx': permutation_idx,
            'embedding_capacity': self.total_capacity
        }

        return stego_text, embedding_info

    def generate_adaptive_embedding(self, text, cover_aware=True, lambda_param=0.1):
        """
        Simulate adaptive steganography that considers cover text statistics
        Based on the adaptive framework in the research paper
        """
        if cover_aware:
            # Analyze cover text statistics
            char_freq = Counter(text.lower())
            total_chars = len(text)
            entropy = self.calculate_shannon_entropy(char_freq, total_chars)

            # Adaptive color selection based on cover statistics
            # In real implementation, this would bias color selection toward natural-looking palettes
            adaptation_factor = 1.0 - min(1.0, entropy / 8.0)  # Normalize entropy
            adaptive_embedding_rate = 0.08 * (1.0 + lambda_param * adaptation_factor)
        else:
            adaptive_embedding_rate = 0.08

        return self.simulate_color_embedding(text, adaptive_embedding_rate)

    def calculate_shannon_entropy(self, freq_dict, total):
        """Calculate Shannon entropy"""
        if total == 0:
            return 0.0
        entropy = 0.0
        for count in freq_dict.values():
            probability = count / total
            if probability > 0:
                entropy -= probability * np.log2(probability)
        return entropy

class DOCXDatasetProcessor:
    """Process DOCX files and generate clean/stego samples - Enhanced with color steganography simulation"""

    def __init__(self):
        self.stop_words = set(stopwords.words('english'))
        self.color_steg_simulator = ColorSteganographySimulator(n_colors=10)

    def load_docx_file(self, file_path):
        """Load and extract text from .docx file"""
        try:
            doc = Document(file_path)
            text = ""
            paragraph_count = 0
            for paragraph in doc.paragraphs:
                if paragraph.text.strip():  # Ignore empty paragraphs
                    text += paragraph.text + "\n\n"
                    paragraph_count += 1
            print(f"  Loaded {paragraph_count} paragraphs from {os.path.basename(file_path)}")
            return self.clean_text(text)
        except Exception as e:
            print(f"Error loading DOCX file {file_path}: {e}")
            return None

    def clean_text(self, text):
        """Clean text by removing headers, footers, and normalizing"""
        if not text:
            return ""

        # Remove common metadata patterns
        lines = text.split('\n')
        cleaned_lines = []

        for line in lines:
            # Skip common metadata and page numbers
            line_lower = line.lower().strip()
            if any(keyword in line_lower for keyword in ['page', 'chapter', 'section', '©', 'all rights reserved']):
                continue
            if len(line.strip()) > 0:  # Only add non-empty lines
                cleaned_lines.append(line.strip())

        cleaned_text = '\n\n'.join(cleaned_lines)

        # Normalize whitespace
        cleaned_text = re.sub(r'\n\s*\n', '\n\n', cleaned_text)
        cleaned_text = re.sub(r' +', ' ', cleaned_text)

        return cleaned_text.strip()

    def enhance_small_paragraphs(self, paragraphs, min_length=50):
        """Enhance small paragraphs by merging or expanding them"""
        enhanced_paragraphs = []
        i = 0

        while i < len(paragraphs):
            current_para = paragraphs[i]

            # If paragraph is too short, try to merge with next one
            if len(current_para) < min_length and i < len(paragraphs) - 1:
                next_para = paragraphs[i + 1]
                merged_para = current_para + " " + next_para

                # If merged paragraph is still short but reasonable, keep it
                if len(merged_para) >= min_length or len(merged_para) > 30:
                    enhanced_paragraphs.append(merged_para)
                    i += 2  # Skip next paragraph since we merged it
                else:
                    enhanced_paragraphs.append(current_para)
                    i += 1
            else:
                enhanced_paragraphs.append(current_para)
                i += 1

        return enhanced_paragraphs

    def simulate_steganography_embedding(self, text, method='color_combinatorial'):
        """
        Simulate different steganography embedding methods - Enhanced with color-based approaches
        """
        if method == 'color_combinatorial':
            return self._color_combinatorial_embedding(text)
        elif method == 'color_adaptive':
            return self._color_adaptive_embedding(text)
        elif method == 'paragraph_manipulation':
            return self._paragraph_manipulation_embedding(text)
        elif method == 'sentence_reordering':
            return self._sentence_reordering_embedding(text)
        elif method == 'k_block_extension':
            return self._k_block_extension_embedding(text)
        else:
            return self._color_combinatorial_embedding(text)

    def _color_combinatorial_embedding(self, text):
        """Simulate high-capacity color combinatorial steganography from the research paper"""
        stego_text, embedding_info = self.color_steg_simulator.simulate_color_embedding(text)

        # Add subtle statistical patterns that might be introduced by color encoding
        words = word_tokenize(text)
        if len(words) > 10:
            # Simulate the statistical effects of color encoding
            # High-capacity methods might introduce subtle distribution changes
            modified_words = words.copy()

            # Simulate permutation effects (very subtle)
            if len(modified_words) >= 5 and np.random.random() < 0.1:
                idx1, idx2 = np.random.choice(len(modified_words), 2, replace=False)
                if idx1 > 0 and idx2 < len(modified_words) - 1:
                    modified_words[idx1], modified_words[idx2] = modified_words[idx2], modified_words[idx1]

            return ' '.join(modified_words)

        return text

    def _color_adaptive_embedding(self, text):
        """Simulate adaptive color steganography that considers cover statistics"""
        stego_text, embedding_info = self.color_steg_simulator.generate_adaptive_embedding(
            text, cover_aware=True, lambda_param=0.1
        )

        # Adaptive methods aim to minimize statistical detectability
        # They preserve natural text statistics better
        return stego_text

    def _k_block_extension_embedding(self, text):
        """Simulate k-block extension for enhanced capacity"""
        # Split text into blocks and apply combinatorial encoding to each
        sentences = sent_tokenize(text)
        if len(sentences) < 2:
            return text

        # Simulate block-based processing (k-block extension)
        k = min(3, len(sentences) // 2)  # Number of blocks
        block_size = max(1, len(sentences) // k)

        processed_sentences = []
        for i in range(0, len(sentences), block_size):
            block = sentences[i:i + block_size]
            # Apply combinatorial encoding to each block
            block_text = ' '.join(block)
            processed_block, _ = self.color_steg_simulator.simulate_color_embedding(block_text)
            processed_sentences.append(processed_block)

        return ' '.join(processed_sentences)

    def _paragraph_manipulation_embedding(self, text):
        """Simulate paragraph-level steganography for small texts"""
        paragraphs = [p.strip() for p in text.split('\n\n') if p.strip()]

        if len(paragraphs) < 2:
            # If only one paragraph, split it artificially
            if paragraphs and len(paragraphs[0]) > 80:
                sentences = sent_tokenize(paragraphs[0])
                if len(sentences) >= 2:
                    # Split into two paragraphs
                    split_point = max(1, len(sentences) // 2)
                    return '\n\n'.join([' '.join(sentences[:split_point]),
                                      ' '.join(sentences[split_point:])])
            return text

        # For small number of paragraphs, use subtle manipulations
        embedded_paragraphs = paragraphs.copy()

        # Randomly swap adjacent paragraphs
        if len(embedded_paragraphs) >= 2 and np.random.random() < 0.3:
            idx = np.random.randint(0, len(embedded_paragraphs) - 1)
            embedded_paragraphs[idx], embedded_paragraphs[idx + 1] = embedded_paragraphs[idx + 1], embedded_paragraphs[idx]

        # Randomly split a long paragraph
        for i in range(len(embedded_paragraphs)):
            if len(embedded_paragraphs[i]) > 100 and np.random.random() < 0.4:
                sentences = sent_tokenize(embedded_paragraphs[i])
                if len(sentences) >= 2:
                    split_point = max(1, len(sentences) // 2)
                    embedded_paragraphs[i] = ' '.join(sentences[:split_point])
                    embedded_paragraphs.insert(i + 1, ' '.join(sentences[split_point:]))
                    break

        return '\n\n'.join(embedded_paragraphs)

    def _sentence_reordering_embedding(self, text):
        """Simulate sentence-level steganography for small texts"""
        sentences = sent_tokenize(text)

        if len(sentences) < 3:
            return text

        # For small texts, use more subtle reordering
        if len(sentences) >= 3:
            # Swap only two sentences to keep coherence
            idx1, idx2 = np.random.choice(len(sentences), 2, replace=False)
            # Ensure we don't swap first and last sentences (too obvious)
            if idx1 != 0 and idx2 != len(sentences) - 1:
                sentences[idx1], sentences[idx2] = sentences[idx2], sentences[idx1]

        return ' '.join(sentences)

    def create_samples_from_docx(self, file_path, num_clean_samples=8, num_stego_samples=8,
                               min_paragraphs=2, max_paragraphs=6, min_text_length=30):
        """Create clean and stego samples from a DOCX file - Enhanced with color steganography"""
        text = self.load_docx_file(file_path)
        if text is None or len(text) < 50:
            print(f"  Text too short or failed to load: {len(text) if text else 0} chars")
            return [], []

        paragraphs = [p.strip() for p in text.split('\n\n') if p.strip()]

        # Enhance small paragraphs by merging
        enhanced_paragraphs = self.enhance_small_paragraphs(paragraphs)

        if len(enhanced_paragraphs) < min_paragraphs:
            print(f"  Not enough paragraphs in {os.path.basename(file_path)}: {len(enhanced_paragraphs)}")
            return [], []

        clean_samples = []
        stego_samples = []

        # Create clean samples with adaptive paragraph count
        samples_created = 0
        max_attempts = num_clean_samples * 3

        while samples_created < num_clean_samples and len(clean_samples) < max_attempts:
            # Adaptive paragraph count based on available paragraphs
            available_paras = min(max_paragraphs, len(enhanced_paragraphs))
            num_paras = np.random.randint(min_paragraphs, available_paras + 1)

            if len(enhanced_paragraphs) - num_paras < 0:
                num_paras = len(enhanced_paragraphs)

            start_idx = np.random.randint(0, len(enhanced_paragraphs) - num_paras + 1)
            sample_paragraphs = enhanced_paragraphs[start_idx:start_idx + num_paras]
            clean_text = '\n\n'.join(sample_paragraphs)

            if len(clean_text) >= min_text_length:
                clean_samples.append(clean_text)
                samples_created += 1

        # Create stego samples from clean samples - Enhanced with color steganography methods
        for clean_text in clean_samples[:num_stego_samples]:
            # Use color-based embedding methods with higher probability
            methods = ['color_combinatorial', 'color_adaptive', 'k_block_extension', 'paragraph_manipulation']
            method_weights = [0.5, 0.3, 0.1, 0.1]  # Emphasize color-based methods

            method = np.random.choice(methods, p=method_weights)
            stego_text = self.simulate_steganography_embedding(clean_text, method)
            stego_samples.append(stego_text)

        print(f"  Created {len(clean_samples)} clean and {len(stego_samples)} stego samples")
        print(f"  Color steganography capacity: {self.color_steg_simulator.total_capacity} bits/block")

        return clean_samples, stego_samples

class HierarchicalSteganalysisDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_sentences=6, max_words=20, max_chars=80):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_sentences = max_sentences
        self.max_words = max_words
        self.max_chars = max_chars
        self.stop_words = set(stopwords.words('english'))
        self.processor = DOCXDatasetProcessor()

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]

        # Hierarchical processing with enhanced features for color steganography detection
        processed_text = self.hierarchical_processing(text)

        return processed_text, torch.tensor(label, dtype=torch.float32)

    def hierarchical_processing(self, text):
        """Convert text to hierarchical representation - Enhanced for color steganalysis"""
        # Enhanced character-level features for color encoding detection
        char_features = self.get_enhanced_char_level_features(text)

        # Sentence-level segmentation and features
        sentences = sent_tokenize(text)
        sentences = sentences[:self.max_sentences]

        # Enhanced word-level features for combinatorial pattern detection
        word_level_features = []
        sentence_level_features = []

        for sentence in sentences:
            words = word_tokenize(sentence.lower())
            words = [word for word in words if word.isalnum() and word not in self.stop_words]
            words = words[:self.max_words]

            word_indices = [self.tokenizer.get(word, 0) for word in words]

            if len(word_indices) < self.max_words:
                word_indices += [0] * (self.max_words - len(word_indices))

            word_level_features.append(word_indices)

            # Enhanced sentence-level features for steganalysis
            sent_features = self.get_enhanced_sentence_features(sentence)
            sentence_level_features.append(sent_features)

        # Pad sentence sequences
        while len(word_level_features) < self.max_sentences:
            word_level_features.append([0] * self.max_words)
            sentence_level_features.append([0.0] * 10)  # 10 sentence features

        # Enhanced document-level features for color steganography detection
        doc_features = self.get_enhanced_document_features(text)

        hierarchical_representation = {
            'char_level': torch.tensor(char_features, dtype=torch.float32),
            'word_level': torch.tensor(word_level_features, dtype=torch.long),
            'sentence_level': torch.tensor(sentence_level_features, dtype=torch.float32),
            'document_level': torch.tensor(doc_features, dtype=torch.float32)
        }

        return hierarchical_representation

    def get_enhanced_char_level_features(self, text):
        """Enhanced character-level features for color steganography detection"""
        if len(text) == 0:
            return [0.0] * 12

        char_count = len(text)
        digit_count = sum(c.isdigit() for c in text)
        letter_count = sum(c.isalpha() for c in text)
        space_count = sum(c.isspace() for c in text)
        special_count = char_count - digit_count - letter_count - space_count

        char_freq = Counter(text.lower())
        entropy = self.calculate_entropy(char_freq, char_count)

        # Additional features for color steganography detection
        upper_ratio = sum(1 for c in text if c.isupper()) / char_count if char_count > 0 else 0
        punctuation_ratio = sum(1 for c in text if c in '.,;!?') / char_count if char_count > 0 else 0
        unique_char_ratio = len(set(text.lower())) / char_count if char_count > 0 else 0

        # Features specific to combinatorial patterns
        consecutive_identical = self.count_consecutive_identical_chars(text)
        pattern_regularity = self.measure_pattern_regularity(text)

        features = [
            char_count / 500.0,
            digit_count / char_count if char_count > 0 else 0,
            letter_count / char_count if char_count > 0 else 0,
            space_count / char_count if char_count > 0 else 0,
            special_count / char_count if char_count > 0 else 0,
            entropy,
            unique_char_ratio,
            punctuation_ratio,
            upper_ratio,
            consecutive_identical / char_count if char_count > 0 else 0,
            pattern_regularity,
            len(text) / self.max_chars
        ]

        return features

    def get_enhanced_sentence_features(self, sentence):
        """Enhanced sentence-level features for combinatorial pattern detection"""
        words = word_tokenize(sentence)
        if len(words) == 0:
            return [0.0] * 10

        char_count = len(sentence)
        word_count = len(words)
        avg_word_len = char_count / word_count if word_count > 0 else 0
        punctuation_count = sum(1 for c in sentence if c in '.,;!?')
        capital_ratio = sum(1 for c in sentence if c.isupper()) / char_count if char_count > 0 else 0

        content_words = [word for word in words if word.lower() not in self.stop_words and word.isalpha()]
        content_ratio = len(content_words) / word_count if word_count > 0 else 0

        # Additional features for combinatorial pattern detection
        word_length_variance = np.var([len(word) for word in words]) if len(words) > 1 else 0
        positional_entropy = self.calculate_positional_entropy(words)

        features = [
            word_count / 30.0,
            char_count / 150.0,
            avg_word_len / 10.0,
            punctuation_count / 5.0,
            capital_ratio,
            content_ratio,
            len([w for w in words if len(w) > 5]) / word_count if word_count > 0 else 0,
            len(set(words)) / word_count if word_count > 0 else 0,
            word_length_variance / 10.0,
            positional_entropy
        ]

        return features

    def get_enhanced_document_features(self, text):
        """Enhanced document-level features for high-capacity steganography detection"""
        sentences = sent_tokenize(text)
        words = word_tokenize(text.lower())
        words = [word for word in words if word.isalnum()]

        if len(words) == 0 or len(sentences) == 0:
            return [0.0] * 15

        total_chars = len(text)
        total_words = len(words)
        total_sentences = len(sentences)

        avg_sentence_length = total_words / total_sentences
        avg_word_length = total_chars / total_words if total_words > 0 else 0
        word_diversity = len(set(words)) / total_words if total_words > 0 else 0

        long_words = len([w for w in words if len(w) > 5])
        complex_word_ratio = long_words / total_words if total_words > 0 else 0

        paragraph_count = text.count('\n\n') + 1
        special_char_ratio = sum(1 for c in text if not c.isalnum() and not c.isspace()) / total_chars if total_chars > 0 else 0

        word_freq = Counter(words)
        word_entropy = self.calculate_entropy(word_freq, total_words)

        char_freq = Counter(text.lower())
        char_entropy = self.calculate_entropy(char_freq, total_chars)

        # Additional features for high-capacity steganography detection
        sentence_length_variance = np.var([len(word_tokenize(s)) for s in sentences]) if len(sentences) > 1 else 0
        combinatorial_pattern_score = self.detect_combinatorial_patterns(text)
        block_structure_metric = self.analyze_block_structure(text)

        features = [
            total_words / 500.0,
            total_sentences / 50.0,
            avg_sentence_length / 25.0,
            avg_word_length / 10.0,
            word_diversity,
            complex_word_ratio,
            special_char_ratio,
            paragraph_count / 5.0,
            word_entropy,
            char_entropy,
            len([s for s in sentences if len(word_tokenize(s)) > 10]) / total_sentences if total_sentences > 0 else 0,
            len([s for s in sentences if len(word_tokenize(s)) < 3]) / total_sentences if total_sentences > 0 else 0,
            sentence_length_variance / 100.0,
            combinatorial_pattern_score,
            block_structure_metric
        ]

        return features[:15]

    def count_consecutive_identical_chars(self, text, max_lookback=3):
        """Count consecutive identical character patterns"""
        if len(text) < 2:
            return 0
        count = 0
        for i in range(1, min(len(text), max_lookback + 1)):
            if text[i] == text[i-1]:
                count += 1
        return count

    def measure_pattern_regularity(self, text, window_size=5):
        """Measure regularity in character patterns"""
        if len(text) < window_size:
            return 0
        variations = 0
        for i in range(window_size, len(text)):
            if text[i] != text[i-window_size]:
                variations += 1
        return variations / (len(text) - window_size) if len(text) > window_size else 0

    def calculate_positional_entropy(self, words):
        """Calculate positional entropy of words"""
        if len(words) < 2:
            return 0
        position_freq = {}
        for i, word in enumerate(words):
            if word not in position_freq:
                position_freq[word] = []
            position_freq[word].append(i)

        entropy = 0
        for word, positions in position_freq.items():
            if len(positions) > 1:
                mean_pos = np.mean(positions)
                variance = np.var(positions)
                if variance > 0:
                    prob = len(positions) / len(words)
                    entropy -= prob * np.log2(prob)
        return entropy

    def detect_combinatorial_patterns(self, text):
        """Detect patterns that might indicate combinatorial encoding"""
        words = word_tokenize(text.lower())
        if len(words) < 5:
            return 0

        # Look for unusual word repetition patterns
        word_freq = Counter(words)
        repeated_words = sum(1 for count in word_freq.values() if count > 1)
        repetition_ratio = repeated_words / len(word_freq) if len(word_freq) > 0 else 0

        return repetition_ratio

    def analyze_block_structure(self, text):
        """Analyze block-like structure that might indicate k-block encoding"""
        sentences = sent_tokenize(text)
        if len(sentences) < 3:
            return 0

        sentence_lengths = [len(word_tokenize(s)) for s in sentences]
        length_variance = np.var(sentence_lengths) if len(sentence_lengths) > 1 else 0

        # Normalize variance
        return min(1.0, length_variance / 100.0)

    def calculate_entropy(self, freq_dict, total):
        """Calculate Shannon entropy"""
        if total == 0:
            return 0.0
        entropy = 0.0
        for count in freq_dict.values():
            probability = count / total
            if probability > 0:
                entropy -= probability * np.log2(probability)
        return entropy

class HierarchicalAttention(nn.Module):
    """Hierarchical Attention Mechanism"""
    def __init__(self, hidden_size):
        super(HierarchicalAttention, self).__init__()
        self.hidden_size = hidden_size
        self.attention = nn.Linear(hidden_size, 1)

    def forward(self, inputs):
        # inputs shape: (batch_size, sequence_length, hidden_size)
        attention_weights = torch.softmax(self.attention(inputs).squeeze(-1), dim=-1)
        weighted_output = torch.sum(inputs * attention_weights.unsqueeze(-1), dim=1)
        return weighted_output, attention_weights

class HierarchicalSteganalysisModel(nn.Module):
    def __init__(self, vocab_size=10000, char_feat_size=12, sent_feat_size=10, doc_feat_size=15,
                 word_embed_dim=50, char_hidden_size=32, sent_hidden_size=32, doc_hidden_size=32,
                 lstm_hidden_size=64, num_layers=1, dropout=0.3):
        super(HierarchicalSteganalysisModel, self).__init__()

        self.vocab_size = vocab_size
        self.lstm_hidden_size = lstm_hidden_size

        # Word-level embedding and encoding
        self.word_embedding = nn.Embedding(vocab_size, word_embed_dim, padding_idx=0)
        self.word_lstm = nn.LSTM(word_embed_dim, lstm_hidden_size, num_layers,
                                batch_first=True, bidirectional=True, dropout=dropout)
        self.word_attention = HierarchicalAttention(lstm_hidden_size * 2)

        # Character-level processing
        self.char_processor = nn.Sequential(
            nn.Linear(char_feat_size, char_hidden_size),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(char_hidden_size, lstm_hidden_size),
            nn.ReLU()
        )

        # Sentence-level processing
        self.sent_feat_processor = nn.Sequential(
            nn.Linear(sent_feat_size, sent_hidden_size),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(sent_hidden_size, lstm_hidden_size),
            nn.ReLU()
        )

        # Document-level processing
        self.doc_feat_processor = nn.Sequential(
            nn.Linear(doc_feat_size, doc_hidden_size),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(doc_hidden_size, lstm_hidden_size),
            nn.ReLU()
        )

        # Sentence-level LSTM
        self.sentence_lstm = nn.LSTM(lstm_hidden_size * 2, lstm_hidden_size, num_layers,
                                   batch_first=True, bidirectional=True, dropout=dropout)
        self.sentence_attention = HierarchicalAttention(lstm_hidden_size * 2)

        # Calculate the total size for the final classifier
        # document_representation: lstm_hidden_size * 2 (bidirectional)
        # char_features: lstm_hidden_size
        # doc_features: lstm_hidden_size
        total_final_size = (lstm_hidden_size * 2) + lstm_hidden_size + lstm_hidden_size

        # Final classification layers
        self.final_classifier = nn.Sequential(
            nn.Linear(total_final_size, 128),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1),
            nn.Sigmoid()
        )

        self.dropout = nn.Dropout(dropout)

    def forward(self, inputs):
        char_level = inputs['char_level']
        word_level = inputs['word_level']
        sentence_level = inputs['sentence_level']
        document_level = inputs['document_level']

        batch_size, max_sentences, max_words = word_level.shape

        # Process character-level features
        char_features = self.char_processor(char_level)  # (batch_size, lstm_hidden_size)

        # Process document-level features
        doc_features = self.doc_feat_processor(document_level)  # (batch_size, lstm_hidden_size)

        # Process word-level features
        word_level_flat = word_level.view(-1, max_words)  # (batch_size * max_sentences, max_words)
        word_embedded = self.word_embedding(word_level_flat)  # (batch_size * max_sentences, max_words, word_embed_dim)

        word_lstm_out, _ = self.word_lstm(word_embedded)  # (batch_size * max_sentences, max_words, lstm_hidden_size * 2)
        sentence_representations, word_attention_weights = self.word_attention(word_lstm_out)  # (batch_size * max_sentences, lstm_hidden_size * 2)

        sentence_representations = sentence_representations.view(batch_size, max_sentences, -1)  # (batch_size, max_sentences, lstm_hidden_size * 2)
        word_attention_weights = word_attention_weights.view(batch_size, max_sentences, max_words)  # (batch_size, max_sentences, max_words)

        # Process sentence-level features
        sentence_feat_processed = self.sent_feat_processor(
            sentence_level.view(-1, sentence_level.size(-1))
        ).view(batch_size, max_sentences, -1)  # (batch_size, max_sentences, lstm_hidden_size)

        # Combine sentence representations with processed features
        combined_sent_repr = sentence_representations + sentence_feat_processed  # (batch_size, max_sentences, lstm_hidden_size * 2)

        # Sentence-level LSTM
        sent_lstm_out, _ = self.sentence_lstm(combined_sent_repr)  # (batch_size, max_sentences, lstm_hidden_size * 2)
        document_representation, sent_attention_weights = self.sentence_attention(sent_lstm_out)  # (batch_size, lstm_hidden_size * 2)

        # Combine all hierarchical representations
        final_representation = torch.cat([
            document_representation,  # (batch_size, lstm_hidden_size * 2)
            char_features,            # (batch_size, lstm_hidden_size)
            doc_features              # (batch_size, lstm_hidden_size)
        ], dim=1)  # (batch_size, (lstm_hidden_size * 2) + lstm_hidden_size + lstm_hidden_size)

        # Final classification
        output = self.final_classifier(final_representation)  # (batch_size, 1)

        return {
            'prediction': output.squeeze(-1),  # (batch_size,)
            'word_attention': word_attention_weights,
            'sentence_attention': sent_attention_weights
        }

class HierarchicalTextSteganalysis:
    def __init__(self, max_sentences=6, max_words=20, vocab_size=3000):
        self.max_sentences = max_sentences
        self.max_words = max_words
        self.vocab_size = vocab_size
        self.tokenizer = {}
        self.model = None
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.history = {'train_loss': [], 'val_loss': [], 'train_acc': [], 'val_acc': []}
        self.processor = DOCXDatasetProcessor()

    def build_vocabulary(self, texts):
        """Build vocabulary from training texts"""
        print("Building vocabulary...")
        all_words = []
        for text in texts:
            words = word_tokenize(text.lower())
            words = [word for word in words if word.isalnum()]
            all_words.extend(words)

        word_freq = Counter(all_words)
        common_words = word_freq.most_common(self.vocab_size - 1)

        self.tokenizer = {'<PAD>': 0}
        for idx, (word, freq) in enumerate(common_words, 1):
            self.tokenizer[word] = idx

        print(f"Vocabulary built with {len(self.tokenizer)} tokens")
        return self.tokenizer

    def create_data_loader(self, texts, labels, batch_size=16, shuffle=True):
        """Create DataLoader for training"""
        dataset = HierarchicalSteganalysisDataset(
            texts, labels, self.tokenizer, self.max_sentences, self.max_words
        )
        return DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, num_workers=0)

    def initialize_model(self, hidden_size=64, dropout=0.3):
        """Initialize the hierarchical model"""
        self.model = HierarchicalSteganalysisModel(
            vocab_size=len(self.tokenizer),
            lstm_hidden_size=hidden_size,
            dropout=dropout
        ).to(self.device)

        print(f"Model initialized on {self.device}")
        print(f"Total parameters: {sum(p.numel() for p in self.model.parameters()):,}")
        return self.model

    def train(self, train_loader, val_loader, epochs=30, learning_rate=0.001, patience=7):
        """Train the hierarchical model"""
        if self.model is None:
            raise ValueError("Model not initialized. Call initialize_model() first.")

        criterion = nn.BCELoss()
        optimizer = optim.Adam(self.model.parameters(), lr=learning_rate, weight_decay=1e-5)
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=3, factor=0.5)

        best_val_loss = float('inf')
        patience_counter = 0

        print("Starting training...")
        for epoch in range(epochs):
            # Training phase
            self.model.train()
            train_loss = 0.0
            train_correct = 0
            train_total = 0

            train_bar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{epochs} [Train]')
            for batch_idx, (inputs, labels) in enumerate(train_bar):
                # Move all tensors to device
                inputs_device = {}
                for key, value in inputs.items():
                    if isinstance(value, torch.Tensor):
                        inputs_device[key] = value.to(self.device)
                labels = labels.to(self.device)

                optimizer.zero_grad()
                outputs = self.model(inputs_device)
                loss = criterion(outputs['prediction'], labels)
                loss.backward()
                torch.nn.utils.clip_grad_norm_(self.model.parameters(), max_norm=1.0)
                optimizer.step()

                train_loss += loss.item()
                predictions = (outputs['prediction'] > 0.5).float()
                train_correct += (predictions == labels).sum().item()
                train_total += labels.size(0)

                train_bar.set_postfix({
                    'Loss': f'{loss.item():.4f}',
                    'Acc': f'{train_correct/train_total:.4f}'
                })

            # Validation phase
            self.model.eval()
            val_loss = 0.0
            val_correct = 0
            val_total = 0

            with torch.no_grad():
                val_bar = tqdm(val_loader, desc=f'Epoch {epoch+1}/{epochs} [Val]')
                for inputs, labels in val_bar:
                    inputs_device = {}
                    for key, value in inputs.items():
                        if isinstance(value, torch.Tensor):
                            inputs_device[key] = value.to(self.device)
                    labels = labels.to(self.device)

                    outputs = self.model(inputs_device)
                    loss = criterion(outputs['prediction'], labels)

                    val_loss += loss.item()
                    predictions = (outputs['prediction'] > 0.5).float()
                    val_correct += (predictions == labels).sum().item()
                    val_total += labels.size(0)

                    val_bar.set_postfix({
                        'Loss': f'{loss.item():.4f}',
                        'Acc': f'{val_correct/val_total:.4f}'
                    })

            # Calculate metrics
            avg_train_loss = train_loss / len(train_loader)
            avg_val_loss = val_loss / len(val_loader)
            train_acc = train_correct / train_total
            val_acc = val_correct / val_total

            self.history['train_loss'].append(avg_train_loss)
            self.history['val_loss'].append(avg_val_loss)
            self.history['train_acc'].append(train_acc)
            self.history['val_acc'].append(val_acc)

            scheduler.step(avg_val_loss)

            print(f'Epoch {epoch+1}/{epochs}:')
            print(f'  Train Loss: {avg_train_loss:.4f}, Train Acc: {train_acc:.4f}')
            print(f'  Val Loss: {avg_val_loss:.4f}, Val Acc: {val_acc:.4f}')

            # Early stopping
            if avg_val_loss < best_val_loss:
                best_val_loss = avg_val_loss
                patience_counter = 0
                torch.save({
                    'model_state_dict': self.model.state_dict(),
                    'tokenizer': self.tokenizer,
                    'config': {
                        'max_sentences': self.max_sentences,
                        'max_words': self.max_words,
                        'vocab_size': self.vocab_size
                    }
                }, 'best_hierarchical_model.pth')
                print(f"  Saved best model with val_loss: {avg_val_loss:.4f}")
            else:
                patience_counter += 1

            if patience_counter >= patience:
                print(f"Early stopping at epoch {epoch+1}")
                break

        # Load best model
        checkpoint = torch.load('best_hierarchical_model.pth')
        self.model.load_state_dict(checkpoint['model_state_dict'])
        print("Training completed!")

    def evaluate(self, test_loader):
        """Evaluate the model on test data"""
        if self.model is None:
            raise ValueError("Model not trained. Call train() first.")

        self.model.eval()
        all_predictions = []
        all_probabilities = []
        all_labels = []

        with torch.no_grad():
            for inputs, labels in tqdm(test_loader, desc='Evaluating'):
                inputs_device = {}
                for key, value in inputs.items():
                    if isinstance(value, torch.Tensor):
                        inputs_device[key] = value.to(self.device)
                labels = labels.to(self.device)

                outputs = self.model(inputs_device)
                probabilities = outputs['prediction'].cpu().numpy()
                predictions = (probabilities > 0.5).astype(int)

                all_probabilities.extend(probabilities)
                all_predictions.extend(predictions)
                all_labels.extend(labels.cpu().numpy())

        accuracy = accuracy_score(all_labels, all_predictions)
        precision = precision_score(all_labels, all_predictions, zero_division=0)
        recall = recall_score(all_labels, all_predictions, zero_division=0)
        f1 = f1_score(all_labels, all_predictions, zero_division=0)
        auc = roc_auc_score(all_labels, all_probabilities)

        print(f"\nEvaluation Results:")
        print(f"Accuracy:  {accuracy:.4f}")
        print(f"Precision: {precision:.4f}")
        print(f"Recall:    {recall:.4f}")
        print(f"F1-Score:  {f1:.4f}")
        print(f"AUC:       {auc:.4f}")

        cm = confusion_matrix(all_labels, all_predictions)
        self.plot_confusion_matrix(cm)

        print("\nClassification Report:")
        print(classification_report(all_labels, all_predictions, target_names=['Clean', 'Stego']))

        return {
            'accuracy': accuracy,
            'precision': precision,
            'recall': recall,
            'f1_score': f1,
            'auc': auc,
            'predictions': all_predictions,
            'probabilities': all_probabilities,
            'labels': all_labels
        }

    def plot_confusion_matrix(self, cm):
        """Plot confusion matrix"""
        plt.figure(figsize=(8, 6))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                   xticklabels=['Clean', 'Stego'],
                   yticklabels=['Clean', 'Stego'])
        plt.title('Confusion Matrix - Hierarchical Text Steganalysis')
        plt.xlabel('Predicted')
        plt.ylabel('Actual')
        plt.show()

    def plot_training_history(self):
        """Plot training history"""
        if not self.history['train_loss']:
            print("No training history available.")
            return

        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

        ax1.plot(self.history['train_loss'], label='Train Loss')
        ax1.plot(self.history['val_loss'], label='Val Loss')
        ax1.set_title('Training and Validation Loss')
        ax1.set_xlabel('Epoch')
        ax1.set_ylabel('Loss')
        ax1.legend()
        ax1.grid(True, alpha=0.3)

        ax2.plot(self.history['train_acc'], label='Train Accuracy')
        ax2.plot(self.history['val_acc'], label='Val Accuracy')
        ax2.set_title('Training and Validation Accuracy')
        ax2.set_xlabel('Epoch')
        ax2.set_ylabel('Accuracy')
        ax2.legend()
        ax2.grid(True, alpha=0.3)

        plt.tight_layout()
        plt.show()

    def analyze_docx_directory(self, directory_path, num_samples_per_file=12, test_size=0.2,
                             epochs=25, batch_size=16):
        """Complete analysis of DOCX files in directory - Optimized for color steganography"""
        print("=" * 80)
        print("ENHANCED HIERARCHICAL TEXT STEGANALYSIS")
        print("OPTIMIZED FOR HIGH-CAPACITY COLOR STEGANOGRAPHY DETECTION")
        print("=" * 80)

        # Find all DOCX files
        docx_files = glob.glob(os.path.join(directory_path, "*.docx"))
        if not docx_files:
            print(f"No DOCX files found in directory: {directory_path}")
            return

        print(f"Found {len(docx_files)} DOCX files")

        # Process files and create dataset
        all_texts = []
        all_labels = []
        files_processed = 0

        for file_path in docx_files:
            print(f"\nProcessing: {os.path.basename(file_path)}")
            clean_samples, stego_samples = self.processor.create_samples_from_docx(
                file_path,
                num_clean_samples=num_samples_per_file,
                num_stego_samples=num_samples_per_file
            )

            if clean_samples and stego_samples:
                all_texts.extend(clean_samples)
                all_labels.extend([0] * len(clean_samples))

                all_texts.extend(stego_samples)
                all_labels.extend([1] * len(stego_samples))
                files_processed += 1

        if not all_texts:
            print("No valid samples created from any files!")
            return

        print(f"\nDataset created successfully from {files_processed} files!")
        print(f"Total samples: {len(all_texts)}")
        print(f"Clean samples: {sum(1 for label in all_labels if label == 0)}")
        print(f"Stego samples: {sum(1 for label in all_labels if label == 1)}")

        # Analyze text lengths for debugging
        text_lengths = [len(text) for text in all_texts]
        print(f"Text length statistics:")
        print(f"  Min: {min(text_lengths)} chars")
        print(f"  Max: {max(text_lengths)} chars")
        print(f"  Avg: {np.mean(text_lengths):.1f} chars")
        print(f"  Std: {np.std(text_lengths):.1f} chars")

        # Build vocabulary
        self.build_vocabulary(all_texts)

        # Split data
        X_train, X_test, y_train, y_test = train_test_split(
            all_texts, all_labels, test_size=test_size, random_state=42, stratify=all_labels
        )

        X_train, X_val, y_train, y_val = train_test_split(
            X_train, y_train, test_size=0.2, random_state=42, stratify=y_train
        )

        print(f"\nData split:")
        print(f"Training samples: {len(X_train)}")
        print(f"Validation samples: {len(X_val)}")
        print(f"Test samples: {len(X_test)}")

        # Create data loaders
        train_loader = self.create_data_loader(X_train, y_train, batch_size=batch_size)
        val_loader = self.create_data_loader(X_val, y_val, batch_size=batch_size)
        test_loader = self.create_data_loader(X_test, y_test, batch_size=batch_size, shuffle=False)

        # Initialize and train model
        self.initialize_model(hidden_size=64, dropout=0.3)

        # Train model
        self.train(train_loader, val_loader, epochs=epochs, learning_rate=0.001, patience=7)

        # Evaluate model
        results = self.evaluate(test_loader)

        # Plot training history
        self.plot_training_history()

        return results

    def predict_single_text(self, text):
        """Predict if a single text contains steganography"""
        if self.model is None:
            raise ValueError("Model not trained. Call train() first.")

        self.model.eval()
        dataset = HierarchicalSteganalysisDataset(
            [text], [0], self.tokenizer, self.max_sentences, self.max_words
        )

        with torch.no_grad():
            inputs, _ = dataset[0]
            inputs_device = {}
            for key, value in inputs.items():
                inputs_device[key] = value.unsqueeze(0).to(self.device)

            outputs = self.model(inputs_device)
            probability = outputs['prediction'].item()
            prediction = 1 if probability > 0.5 else 0

            return {
                'prediction': prediction,
                'probability': probability,
                'word_attention': outputs['word_attention'],
                'sentence_attention': outputs['sentence_attention']
            }

def main():
    """Main function to analyze DOCX files with enhanced color steganography detection"""

    directory_path = '/content/gdrive/MyDrive/DatasetsEvaluations/NewArticleCorpusStego'

    if not os.path.exists(directory_path):
        print(f"Directory not found: {directory_path}")
        print("Please check the path and try again.")
        return

    print("=" * 80)
    print("ENHANCED HIERARCHICAL TEXT STEGANALYSIS")
    print("OPTIMIZED FOR HIGH-CAPACITY COLOR STEGANOGRAPHY DETECTION")
    print("Based on: High Embedding Capacity Text Steganography Using Optimal")
    print("Color Combinations from 24-bit Space")
    print("=" * 80)
    print(f"Analyzing DOCX files in: {directory_path}")

    # Initialize enhanced steganalysis system
    steganalyzer = HierarchicalTextSteganalysis(
        max_sentences=6,
        max_words=20,
        vocab_size=3000
    )

    try:
        # Analyze directory with enhanced detection capabilities
        results = steganalyzer.analyze_docx_directory(
            directory_path=directory_path,
            num_samples_per_file=12,
            test_size=0.2,
            epochs=25,
            batch_size=16
        )

        if results:
            print("\n" + "=" * 80)
            print("ENHANCED DETECTION RESULTS SUMMARY")
            print("=" * 80)
            print(f"Detection Performance against High-Capacity Color Steganography:")
            print(f"  Accuracy:  {results['accuracy']:.4f}")
            print(f"  Precision: {results['precision']:.4f}")
            print(f"  Recall:    {results['recall']:.4f}")
            print(f"  F1-Score:  {results['f1_score']:.4f}")
            print(f"  AUC:       {results['auc']:.4f}")

            # Research paper context interpretation
            print("\nResearch Context Interpretation:")
            if results['accuracy'] > 0.85:
                print("  ✓ Excellent detection of high-capacity color steganography")
                print("  ✓ Comparable to state-of-the-art steganalysis frameworks")
            elif results['accuracy'] > 0.75:
                print("  ✓ Good detection capability against combinatorial methods")
                print("  ○ May struggle with adaptive color steganography (λ=0.1)")
            elif results['accuracy'] > 0.65:
                print("  ○ Moderate detection - combinatorial patterns partially detected")
                print("  ⚠ Adaptive methods with λ>0.2 may evade detection")
            else:
                print("  ⚠ Limited detection capability against high-capacity methods")
                print("  🔧 Consider feature engineering for combinatorial patterns")

    except Exception as e:
        print(f"Error during enhanced analysis: {e}")
        import traceback
        traceback.print_exc()

if __name__ == "__main__":
    main()

#TS-RNN

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from docx import Document
from docx.shared import RGBColor
import pickle
import re
from collections import Counter
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectKBest, f_classif
import xml.etree.ElementTree as ET
from math import comb, factorial
import colorsys

class ColorSteganalysis:
    def __init__(self, color_features=18, lstm_units=128, num_layers=2):
        self.color_features = color_features
        self.lstm_units = lstm_units
        self.num_layers = num_layers
        self.model = None
        self.feature_selector = None
        self.scaler = None

        # Updated based on paper findings: typical n values used
        self.typical_color_counts = [8, 10, 16, 24, 32]
        self.max_sequence_length = 500

        # Paper-specific parameters
        self.typical_coverage_range = (0.08, 0.11)
        self.high_capacity_threshold = 0.15
        self.blocks_per_document = {}

    def extract_rgb_from_color(self, color_obj):
        """Extract RGB values from RGBColor object safely"""
        try:
            if hasattr(color_obj, 'rgb'):
                rgb_int = color_obj.rgb

                if rgb_int is not None:
                    if isinstance(rgb_int, int):
                        r = (rgb_int >> 16) & 0xFF
                        g = (rgb_int >> 8) & 0xFF
                        b = rgb_int & 0xFF
                        return (r, g, b)
                    else:
                        rgb_str = str(rgb_int)
                        if rgb_str.startswith('RGBColor'):
                            numbers = re.findall(r'\d+', rgb_str)
                            if len(numbers) >= 3:
                                return (int(numbers[0]), int(numbers[1]), int(numbers[2]))
                        elif ',' in rgb_str:
                            numbers = re.findall(r'\d+', rgb_str)
                            if len(numbers) >= 3:
                                return (int(numbers[0]), int(numbers[1]), int(numbers[2]))
            return None
        except Exception as e:
            print(f"Error extracting RGB from color object: {e}")
            return None

    def build_enhanced_model(self):
        """Build enhanced model based on paper's combinatorial characteristics"""
        # Feature-based branch
        feature_input = tf.keras.Input(shape=(self.color_features,), name='feature_input')

        # Sequence-based branch for color permutation patterns
        sequence_input = tf.keras.Input(shape=(self.max_sequence_length, 3), name='sequence_input')

        # Enhanced CNN for combinatorial pattern detection
        x_seq = layers.Conv1D(32, 5, activation='relu')(sequence_input)
        x_seq = layers.MaxPooling1D(2)(x_seq)
        x_seq = layers.Conv1D(64, 5, activation='relu')(x_seq)
        x_seq = layers.MaxPooling1D(2)(x_seq)
        x_seq = layers.Conv1D(128, 5, activation='relu')(x_seq)
        x_seq = layers.GlobalAveragePooling1D()(x_seq)

        # Additional LSTM for sequential pattern analysis
        lstm_seq = layers.LSTM(64, return_sequences=True)(sequence_input)
        lstm_seq = layers.LSTM(32)(lstm_seq)

        # Combine all branches
        combined = layers.concatenate([feature_input, x_seq, lstm_seq])

        # Enhanced dense layers
        x = layers.Dense(128, activation='relu')(combined)
        x = layers.Dropout(0.5)(x)
        x = layers.Dense(64, activation='relu')(x)
        x = layers.Dropout(0.3)(x)
        x = layers.Dense(32, activation='relu')(x)

        outputs = layers.Dense(1, activation='sigmoid')(x)

        self.model = models.Model(
            inputs=[feature_input, sequence_input],
            outputs=outputs
        )

        self.model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
            loss='binary_crossentropy',
            metrics=['accuracy', 'precision', 'recall', 'auc']
        )

        return self.model

    def extract_combinatorial_features(self, file_path):
        """Extract combinatorial color-permutation features based on paper methodology"""
        try:
            doc = Document(file_path)
            combinatorial_features = {}

            # Extract all colored text runs
            colored_runs = []
            color_sequence = []
            positions = []

            char_count = 0
            for paragraph in doc.paragraphs:
                para_text = paragraph.text
                for run in paragraph.runs:
                    if run.font.color and run.font.color.rgb is not None:
                        rgb_tuple = self.extract_rgb_from_color(run.font.color)
                        if rgb_tuple:
                            colored_runs.append(run)
                            color_sequence.append(rgb_tuple)
                            positions.append(char_count)
                    char_count += len(run.text)
                char_count += 1

            # Feature 1: Color coverage ratio
            total_chars = char_count
            color_coverage = len(colored_runs) / total_chars if total_chars > 0 else 0
            combinatorial_features['color_coverage'] = color_coverage

            # Feature 2: Color combination diversity
            unique_colors = len(set(color_sequence))
            combinatorial_features['unique_colors'] = unique_colors

            # Feature 3: Block pattern detection
            if len(colored_runs) > 0:
                block_sizes = []
                for n in self.typical_color_counts:
                    if len(colored_runs) % n == 0:
                        block_sizes.append(n)

                combinatorial_features['potential_blocks'] = len(block_sizes)
                combinatorial_features['likely_block_size'] = max(block_sizes) if block_sizes else 0

                if combinatorial_features['likely_block_size'] > 0:
                    k_blocks = len(colored_runs) // combinatorial_features['likely_block_size']
                    combinatorial_features['k_blocks'] = k_blocks
                else:
                    combinatorial_features['k_blocks'] = 0
            else:
                combinatorial_features.update({
                    'potential_blocks': 0,
                    'likely_block_size': 0,
                    'k_blocks': 0
                })

            # Feature 4: Combinatorial space analysis
            if combinatorial_features['likely_block_size'] > 0:
                n = combinatorial_features['likely_block_size']
                try:
                    color_combinations = comb(16777216, n)
                    permutations = factorial(n)
                    theoretical_capacity = np.log2(color_combinations * permutations)
                    combinatorial_features['theoretical_capacity_bits'] = theoretical_capacity
                except (ValueError, OverflowError):
                    combinatorial_features['theoretical_capacity_bits'] = 0
            else:
                combinatorial_features['theoretical_capacity_bits'] = 0

            # Feature 5: RGB value patterns
            if len(color_sequence) >= 2:
                color_changes = []
                rgb_patterns = []

                for i in range(1, len(color_sequence)):
                    change = sum(abs(a - b) for a, b in zip(color_sequence[i], color_sequence[i-1]))
                    color_changes.append(change)
                    pattern = tuple(sorted([color_sequence[i-1], color_sequence[i]]))
                    rgb_patterns.append(pattern)

                combinatorial_features['avg_color_change'] = np.mean(color_changes) if color_changes else 0
                combinatorial_features['color_change_std'] = np.std(color_changes) if color_changes else 0
                combinatorial_features['unique_patterns'] = len(set(rgb_patterns)) if rgb_patterns else 0
            else:
                combinatorial_features.update({
                    'avg_color_change': 0,
                    'color_change_std': 0,
                    'unique_patterns': 0
                })

            # Feature 6: Statistical distribution
            if color_sequence:
                reds, greens, blues = zip(*color_sequence)

                combinatorial_features['color_std_red'] = np.std(reds)
                combinatorial_features['color_std_green'] = np.std(greens)
                combinatorial_features['color_std_blue'] = np.std(blues)
                combinatorial_features['color_mean_red'] = np.mean(reds)
                combinatorial_features['color_mean_green'] = np.mean(greens)
                combinatorial_features['color_mean_blue'] = np.mean(blues)

                color_entropy = self.calculate_color_entropy(color_sequence)
                combinatorial_features['color_entropy'] = color_entropy
            else:
                combinatorial_features.update({
                    'color_std_red': 0, 'color_std_green': 0, 'color_std_blue': 0,
                    'color_mean_red': 0, 'color_mean_green': 0, 'color_mean_blue': 0,
                    'color_entropy': 0
                })

            # Feature 7: Spatial distribution analysis
            combinatorial_features['spatial_regularity'] = self.calculate_enhanced_spatial_regularity(positions)

            # Feature 8: Adaptive steganography detection
            combinatorial_features['adaptive_likelihood'] = self.detect_adaptive_patterns(color_sequence, combinatorial_features)

            return combinatorial_features, color_sequence

        except Exception as e:
            print(f"Error extracting combinatorial features from {file_path}: {e}")
            return {}, []

    def calculate_color_entropy(self, color_sequence):
        """Calculate entropy of color distribution"""
        if len(color_sequence) < 2:
            return 0

        hsv_values = []
        for r, g, b in color_sequence:
            try:
                h, s, v = colorsys.rgb_to_hsv(r/255.0, g/255.0, b/255.0)
                hsv_values.append((h, s, v))
            except:
                continue

        if not hsv_values:
            return 0

        hues = [h for h, s, v in hsv_values]
        hue_histogram, _ = np.histogram(hues, bins=16, range=(0, 1))
        hue_probs = hue_histogram / len(hues)
        hue_probs = hue_probs[hue_probs > 0]

        if len(hue_probs) == 0:
            return 0

        entropy = -np.sum(hue_probs * np.log2(hue_probs))
        return entropy

    def calculate_enhanced_spatial_regularity(self, positions):
        """Enhanced spatial analysis for k-block patterns"""
        if len(positions) < 2:
            return 0

        spacings = [positions[i+1] - positions[i] for i in range(len(positions)-1)]

        if len(spacings) >= 3:
            spacing_std = np.std(spacings)
            spacing_mean = np.mean(spacings)
            spacing_cv = spacing_std / spacing_mean if spacing_mean > 0 else 0

            regularity_score = 1 - min(spacing_cv, 1.0)
            return regularity_score
        else:
            return 0

    def detect_adaptive_patterns(self, color_sequence, features):
        """Detect adaptive steganography patterns from paper"""
        if len(color_sequence) < 10:
            return 0

        coverage = features.get('color_coverage', 0)
        unique_colors = features.get('unique_colors', 0)
        entropy = features.get('color_entropy', 0)

        adaptive_score = 0

        if 0.07 <= coverage <= 0.12:
            adaptive_score += 0.3

        if unique_colors in self.typical_color_counts:
            adaptive_score += 0.3

        if entropy > 2.0:
            adaptive_score += 0.4

        return min(adaptive_score, 1.0)

    def create_color_sequence_matrix(self, color_sequence, max_length=500):
        """Convert color sequence to normalized matrix"""
        if not color_sequence:
            return np.zeros((max_length, 3))

        normalized_sequence = np.array(color_sequence) / 255.0

        if len(normalized_sequence) > max_length:
            normalized_sequence = normalized_sequence[:max_length]
        else:
            padding = max_length - len(normalized_sequence)
            normalized_sequence = np.pad(normalized_sequence,
                                       ((0, padding), (0, 0)),
                                       mode='constant')

        return normalized_sequence

    def load_dataset_from_directory(self, directory_path):
        """Load and process Word files with combinatorial analysis"""
        features_list = []
        sequences_list = []
        file_paths = []
        labels = []

        if not os.path.exists(directory_path):
            print(f"Directory not found: {directory_path}")
            return features_list, sequences_list, file_paths, labels

        successful_files = 0
        docx_files = [f for f in os.listdir(directory_path) if f.endswith('.docx')]

        print(f"Found {len(docx_files)} .docx files in directory")

        for filename in docx_files:
            file_path = os.path.join(directory_path, filename)
            combinatorial_features, color_sequence = self.extract_combinatorial_features(file_path)

            if combinatorial_features and color_sequence and len(color_sequence) > 0:
                feature_values = list(combinatorial_features.values())

                if len(feature_values) < self.color_features:
                    feature_values.extend([0] * (self.color_features - len(feature_values)))
                elif len(feature_values) > self.color_features:
                    feature_values = feature_values[:self.color_features]

                features_list.append(feature_values)
                sequences_list.append(self.create_color_sequence_matrix(color_sequence))
                file_paths.append(file_path)

                coverage = combinatorial_features.get('color_coverage', 0)
                unique_colors = combinatorial_features.get('unique_colors', 0)
                theoretical_capacity = combinatorial_features.get('theoretical_capacity_bits', 0)

                stego_indicators = 0
                if coverage >= 0.05:
                    stego_indicators += 1
                if unique_colors in self.typical_color_counts:
                    stego_indicators += 1
                if theoretical_capacity > 100:
                    stego_indicators += 1

                labels.append(1 if stego_indicators >= 2 else 0)
                successful_files += 1

        print(f"Successfully processed {successful_files} documents from {directory_path}")
        return features_list, sequences_list, file_paths, labels

    def create_synthetic_dataset(self, num_samples=100):
        """Create a synthetic dataset for testing"""
        print(f"Creating synthetic dataset with {num_samples} samples...")

        features_list = []
        sequences_list = []
        labels = []

        feature_names = [
            'color_coverage', 'unique_colors', 'theoretical_capacity_bits', 'k_blocks',
            'color_std_red', 'color_std_green', 'color_std_blue',
            'color_mean_red', 'color_mean_green', 'color_mean_blue',
            'color_entropy', 'avg_color_change', 'color_change_std', 'unique_patterns',
            'spatial_regularity', 'adaptive_likelihood', 'potential_blocks', 'likely_block_size'
        ]

        # Create synthetic stego documents
        for i in range(num_samples // 2):
            features = {
                'color_coverage': np.random.uniform(0.08, 0.12),
                'unique_colors': np.random.choice([8, 10, 16, 24, 32]),
                'theoretical_capacity_bits': np.random.uniform(200, 300),
                'k_blocks': np.random.randint(2, 10),
                'color_std_red': np.random.uniform(30, 80),
                'color_std_green': np.random.uniform(30, 80),
                'color_std_blue': np.random.uniform(30, 80),
                'color_mean_red': np.random.uniform(100, 150),
                'color_mean_green': np.random.uniform(100, 150),
                'color_mean_blue': np.random.uniform(100, 150),
                'color_entropy': np.random.uniform(2.5, 3.5),
                'avg_color_change': np.random.uniform(50, 150),
                'color_change_std': np.random.uniform(20, 60),
                'unique_patterns': np.random.randint(5, 20),
                'spatial_regularity': np.random.uniform(0.7, 0.9),
                'adaptive_likelihood': np.random.uniform(0.6, 0.9),
                'potential_blocks': np.random.randint(1, 4),
                'likely_block_size': np.random.choice([8, 10, 16, 24, 32])
            }

            feature_values = [features[name] for name in feature_names]
            features_list.append(feature_values)

            color_sequence = []
            num_colors = np.random.randint(50, 200)
            for _ in range(num_colors):
                r = np.random.randint(0, 255)
                g = np.random.randint(0, 255)
                b = np.random.randint(0, 255)
                color_sequence.append((r, g, b))

            sequences_list.append(self.create_color_sequence_matrix(color_sequence))
            labels.append(1)

        # Create synthetic clean documents
        for i in range(num_samples // 2):
            features = {
                'color_coverage': np.random.uniform(0.001, 0.02),
                'unique_colors': np.random.randint(1, 5),
                'theoretical_capacity_bits': np.random.uniform(0, 50),
                'k_blocks': 0,
                'color_std_red': np.random.uniform(5, 20),
                'color_std_green': np.random.uniform(5, 20),
                'color_std_blue': np.random.uniform(5, 20),
                'color_mean_red': np.random.uniform(100, 150),
                'color_mean_green': np.random.uniform(100, 150),
                'color_mean_blue': np.random.uniform(100, 150),
                'color_entropy': np.random.uniform(0.5, 1.5),
                'avg_color_change': np.random.uniform(10, 40),
                'color_change_std': np.random.uniform(5, 20),
                'unique_patterns': np.random.randint(1, 5),
                'spatial_regularity': np.random.uniform(0.1, 0.3),
                'adaptive_likelihood': np.random.uniform(0.1, 0.3),
                'potential_blocks': 0,
                'likely_block_size': 0
            }

            feature_values = [features[name] for name in feature_names]
            features_list.append(feature_values)

            color_sequence = []
            if np.random.random() < 0.3:
                for _ in range(np.random.randint(1, 10)):
                    r = np.random.randint(0, 255)
                    g = np.random.randint(0, 255)
                    b = np.random.randint(0, 255)
                    color_sequence.append((r, g, b))

            sequences_list.append(self.create_color_sequence_matrix(color_sequence))
            labels.append(0)

        print(f"Created synthetic dataset with {len(features_list)} samples")
        print(f"Feature shape: {len(features_list[0])} features per sample")
        return features_list, sequences_list, [], labels

    def train_model(self, stego_directory, epochs=30, batch_size=16, use_synthetic=True):
        """Train the enhanced combinatorial steganalysis model"""
        print("Loading documents for combinatorial analysis...")

        if use_synthetic:
            features, sequences, paths, labels = self.create_synthetic_dataset(num_samples=100)
        else:
            features, sequences, paths, labels = self.load_dataset_from_directory(stego_directory)

        if not features:
            print("No valid documents found for training! Using synthetic dataset.")
            features, sequences, paths, labels = self.create_synthetic_dataset(num_samples=100)

        X_features = np.array(features)
        X_sequences = np.array(sequences)
        y = np.array(labels)

        print(f"Training samples: {len(X_features)}")
        print(f"Feature shape: {X_features.shape}")
        print(f"Sequence shape: {X_sequences.shape}")
        print(f"Class distribution: {np.sum(y)} stego, {len(y)-np.sum(y)} clean")

        # Update color_features to match actual data
        self.color_features = X_features.shape[1]
        print(f"Updated model to expect {self.color_features} features")

        # Build model with correct input shape
        self.build_enhanced_model()

        print("Enhanced model architecture:")
        self.model.summary()

        # Enhanced training with callbacks
        early_stopping = tf.keras.callbacks.EarlyStopping(
            monitor='val_auc', patience=10, restore_best_weights=True, mode='max'
        )

        reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss', factor=0.5, patience=5, min_lr=1e-7
        )

        # Train model
        print("Training enhanced model...")
        history = self.model.fit(
            [X_features, X_sequences], y,
            batch_size=batch_size,
            epochs=epochs,
            validation_split=0.2,
            verbose=1,
            class_weight={0: 1, 1: 2},
            callbacks=[early_stopping, reduce_lr]
        )

        # Plot training history
        self.plot_enhanced_training_history(history)

        return history

    def plot_enhanced_training_history(self, history):
        """Enhanced training history visualization"""
        fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 10))

        # Plot accuracy
        ax1.plot(history.history['accuracy'], label='Training Accuracy', linewidth=2)
        ax1.plot(history.history['val_accuracy'], label='Validation Accuracy', linewidth=2)
        ax1.set_title('Model Accuracy')
        ax1.set_xlabel('Epoch')
        ax1.set_ylabel('Accuracy')
        ax1.legend()
        ax1.grid(True, alpha=0.3)

        # Plot AUC
        if 'auc' in history.history:
            ax2.plot(history.history['auc'], label='Training AUC', linewidth=2)
            ax2.plot(history.history['val_auc'], label='Validation AUC', linewidth=2)
            ax2.set_title('Model AUC')
            ax2.set_xlabel('Epoch')
            ax2.set_ylabel('AUC')
            ax2.legend()
            ax2.grid(True, alpha=0.3)

        # Plot loss
        ax3.plot(history.history['loss'], label='Training Loss', linewidth=2)
        ax3.plot(history.history['val_loss'], label='Validation Loss', linewidth=2)
        ax3.set_title('Model Loss')
        ax3.set_xlabel('Epoch')
        ax3.set_ylabel('Loss')
        ax3.legend()
        ax3.grid(True, alpha=0.3)

        plt.tight_layout()
        plt.show()

    def save_model(self, model_path):
        """Save the trained model and metadata"""
        if self.model is not None:
            # Create directory if it doesn't exist
            os.makedirs(os.path.dirname(model_path), exist_ok=True)

            # Save the model
            self.model.save(model_path)

            # Save metadata
            metadata_path = model_path.replace('.h5', '_metadata.pkl')
            metadata = {
                'color_features': self.color_features,
                'typical_color_counts': self.typical_color_counts,
                'typical_coverage_range': self.typical_coverage_range,
                'max_sequence_length': self.max_sequence_length
            }

            with open(metadata_path, 'wb') as f:
                pickle.dump(metadata, f)

            print(f"Model saved to {model_path}")
            print(f"Metadata saved to {metadata_path}")
        else:
            print("No model to save. Please train the model first.")

    def load_model(self, model_path):
        """Load a trained model and metadata"""
        try:
            if not os.path.exists(model_path):
                print(f"Model file not found: {model_path}")
                return False

            # Load the model
            self.model = tf.keras.models.load_model(model_path)

            # Load metadata
            metadata_path = model_path.replace('.h5', '_metadata.pkl')
            if os.path.exists(metadata_path):
                with open(metadata_path, 'rb') as f:
                    metadata = pickle.load(f)
                    self.color_features = metadata.get('color_features', 18)
                    self.typical_color_counts = metadata.get('typical_color_counts', [8, 10, 16, 24, 32])
                    self.typical_coverage_range = metadata.get('typical_coverage_range', (0.08, 0.11))
                    self.max_sequence_length = metadata.get('max_sequence_length', 500)

            print("Model loaded successfully!")
            return True

        except Exception as e:
            print(f"Error loading model: {e}")
            return False

    def predict_document(self, file_path, threshold=0.65):
        """Predict if a document contains steganographic content"""
        if self.model is None:
            print("Model not trained. Please train the model first.")
            return 0, 0.0, "Model not trained"

        combinatorial_features, color_sequence = self.extract_combinatorial_features(file_path)
        if not combinatorial_features:
            return 0, 0.0, "No features extracted"

        # Prepare features
        feature_values = list(combinatorial_features.values())
        if len(feature_values) < self.color_features:
            feature_values.extend([0] * (self.color_features - len(feature_values)))
        elif len(feature_values) > self.color_features:
            feature_values = feature_values[:self.color_features]

        features_array = np.array([feature_values])
        sequence_matrix = np.array([self.create_color_sequence_matrix(color_sequence)])

        # Predict
        prediction = self.model.predict([features_array, sequence_matrix], verbose=0)[0][0]

        # Apply paper-based heuristics
        coverage = combinatorial_features.get('color_coverage', 0)
        unique_colors = combinatorial_features.get('unique_colors', 0)
        theoretical_capacity = combinatorial_features.get('theoretical_capacity_bits', 0)

        adjusted_prediction = prediction

        if 0.07 <= coverage <= 0.12:
            adjusted_prediction = min(1.0, adjusted_prediction + 0.15)
        elif coverage > 0.15:
            adjusted_prediction = min(1.0, adjusted_prediction + 0.25)

        if unique_colors in self.typical_color_counts:
            adjusted_prediction = min(1.0, adjusted_prediction + 0.10)

        if theoretical_capacity > 200:
            adjusted_prediction = min(1.0, adjusted_prediction + 0.15)

        result = "Stego" if adjusted_prediction >= threshold else "Clean"
        confidence_level = "HIGH" if adjusted_prediction >= 0.8 else "MEDIUM" if adjusted_prediction >= 0.6 else "LOW"

        return int(adjusted_prediction >= threshold), adjusted_prediction, f"{result} ({confidence_level})"

    def analyze_directory(self, directory_path, threshold=0.65):
        """Analyze all documents in a directory"""
        print(f"Analyzing directory: {directory_path}")

        if not os.path.exists(directory_path):
            print(f"Directory not found: {directory_path}")
            return []

        results = []
        docx_files = [f for f in os.listdir(directory_path) if f.endswith('.docx')]

        if not docx_files:
            print("No .docx files found in the directory")
            return results

        for filename in docx_files:
            file_path = os.path.join(directory_path, filename)
            label, confidence, result = self.predict_document(file_path, threshold)

            combinatorial_features, _ = self.extract_combinatorial_features(file_path)
            coverage = combinatorial_features.get('color_coverage', 0) * 100
            unique_colors = combinatorial_features.get('unique_colors', 0)
            theoretical_capacity = combinatorial_features.get('theoretical_capacity_bits', 0)

            results.append({
                'filename': filename,
                'prediction': label,
                'confidence': confidence,
                'result': result,
                'color_coverage': coverage,
                'unique_colors': unique_colors,
                'theoretical_capacity': theoretical_capacity
            })

            print(f"{filename}: {result} (conf: {confidence:.4f}, coverage: {coverage:.1f}%, colors: {unique_colors})")

        return results

    def generate_comprehensive_report(self, results):
        """Generate comprehensive analysis report"""
        if not results:
            return "No results to report"

        stego_docs = [r for r in results if r['prediction'] == 1]
        clean_docs = [r for r in results if r['prediction'] == 0]

        report = []
        report.append("=" * 60)
        report.append("COMBINATORIAL STEGANALYSIS REPORT")
        report.append("=" * 60)
        report.append(f"Total documents analyzed: {len(results)}")
        report.append(f"Potential steganography documents: {len(stego_docs)}")
        report.append(f"Clean documents: {len(clean_docs)}")
        report.append(f"Detection rate: {len(stego_docs)/len(results)*100:.1f}%")

        if results:
            avg_confidence = np.mean([r['confidence'] for r in results])
            avg_coverage = np.mean([r['color_coverage'] for r in results])

            report.append(f"\nStatistical Summary:")
            report.append(f"  Average confidence: {avg_confidence:.4f}")
            report.append(f"  Average color coverage: {avg_coverage:.2f}%")

            if stego_docs:
                report.append(f"\nTop steganography suspects:")
                sorted_stego = sorted(stego_docs, key=lambda x: x['confidence'], reverse=True)
                for i, doc in enumerate(sorted_stego[:5]):
                    report.append(f"  {i+1}. {doc['filename']}")
                    report.append(f"     Confidence: {doc['confidence']:.4f}")
                    report.append(f"     Coverage: {doc['color_coverage']:.1f}%")
                    report.append(f"     Colors: {doc['unique_colors']}")

        return "\n".join(report)

def main():
    """Main function to run the steganalysis system"""
    steganalyzer = ColorSteganalysis()

    stego_directory = '/content/gdrive/MyDrive/DatasetsEvaluations/NewArticleCorpusStego'
    model_path = '/content/gdrive/MyDrive/Models/combinatorial_color_steganalysis.h5'

    if os.path.exists(model_path) and steganalyzer.load_model(model_path):
        print("Model loaded successfully!")
    else:
        print("Training new model...")
        history = steganalyzer.train_model(stego_directory, epochs=30, use_synthetic=True)

        if history is not None:
            print("Model training completed!")
            steganalyzer.save_model(model_path)

            if os.path.exists(stego_directory):
                print("\nAnalyzing documents...")
                results = steganalyzer.analyze_directory(stego_directory, threshold=0.65)
                report = steganalyzer.generate_comprehensive_report(results)
                print("\n" + report)

if __name__ == "__main__":
    main()

In [ ]:
pip install tensorflow scikit-learn matplotlib python-docx

In [ ]:
pip install tensorflow docx2txt nltk matplotlib scikit-learn

In [ ]:
import nltk
nltk.download('punkt_tab')

#CNN-based Text Steganalysis

In [ ]:
import os
import numpy as np
import pandas as pd
from collections import Counter
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Reshape, LSTM, Bidirectional, Concatenate, Conv1D, MaxPooling1D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
import nltk
from nltk.tokenize import word_tokenize
import matplotlib.pyplot as plt
import re
from sklearn.metrics import roc_auc_score, precision_recall_fscore_support

# Download required NLTK data
nltk.download('punkt', quiet=True)

class AdvancedColorSteganalysis:
    def __init__(self, max_vocab_size=10000, max_sequence_length=500, embedding_dim=100):
        self.max_vocab_size = max_vocab_size
        self.max_sequence_length = max_sequence_length
        self.embedding_dim = embedding_dim
        self.vocab = None
        self.model = None

        # Paper-specific parameters
        self.paper_capacity_gain = 11.4  # 11.4x improvement over baseline
        self.expected_coverage_range = (0.08, 0.11)  # 8-11% coverage from paper
        self.expected_capacity_range = (1.75, 3.00)  # 175-300% capacity from paper

    def extract_text_from_file(self, file_path):
        """Extract text from various file types with fallback methods"""
        try:
            # Method 1: Try reading as text file
            if file_path.endswith('.txt'):
                with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
                    return f.read()

            # Method 2: For other files, try basic text extraction
            else:
                with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
                    return f.read()

        except Exception as e:
            print(f"Error extracting from {file_path}: {e}")
            return ""

    def detect_combinatorial_patterns(self, text):
        """Detect combinatorial color encoding patterns based on paper's methodology"""
        patterns = {
            'color_sequences': 0,
            'hex_patterns': 0,
            'rgb_patterns': 0,
            'color_keywords': 0,
            'combinatorial_indicators': 0,
            'high_capacity_indicators': 0,
            'adaptive_patterns': 0,
            'k_block_indicators': 0
        }

        # Look for hex color patterns (24-bit RGB) - paper uses full 24-bit space
        hex_colors = re.findall(r'#[0-9A-Fa-f]{6}', text)
        patterns['hex_patterns'] = len(hex_colors)

        # Look for RGB patterns
        rgb_patterns = re.findall(r'rgb\(\s*\d+\s*,\s*\d+\s*,\s*\d+\s*\)', text, re.IGNORECASE)
        patterns['rgb_patterns'] = len(rgb_patterns)

        # Look for paper-specific keywords
        paper_keywords = [
            'rgb', 'color', 'palette', 'combinatorial', 'permutation',
            'embedding', 'steganography', 'capacity', '24-bit', 'bitspace',
            'dynamic', 'adaptive', 'k-block', 'lexicographic', 'unranking'
        ]
        patterns['color_keywords'] = sum(1 for keyword in paper_keywords if keyword in text.lower())

        # High capacity indicators based on paper's methodology
        text_length = len(text)
        if text_length > 0:
            color_density = patterns['hex_patterns'] / text_length

            # Paper mentions 8-11% coverage for high capacity
            if 0.05 <= color_density <= 0.15:  # Slightly wider range for detection
                patterns['high_capacity_indicators'] += 2
            elif color_density < 0.05 and patterns['hex_patterns'] >= 3:
                patterns['high_capacity_indicators'] += 1  # Low coverage, high info density

        # Combinatorial indicators based on paper's theoretical framework
        if patterns['hex_patterns'] >= 10:  # Paper uses n=10 colors typically
            patterns['combinatorial_indicators'] += 2
        elif patterns['hex_patterns'] >= 5:
            patterns['combinatorial_indicators'] += 1

        # K-block extension indicators
        if patterns['hex_patterns'] >= 20 and patterns['hex_patterns'] % 10 == 0:
            patterns['k_block_indicators'] += 1  # Multiple of typical block size

        # Adaptive steganography indicators
        if patterns['color_keywords'] >= 3 and patterns['hex_patterns'] > 0:
            patterns['adaptive_patterns'] += 1

        return patterns

    def calculate_paper_based_scores(self, text, patterns):
        """Calculate detection scores based on paper's specific methodology"""
        scores = {
            'combinatorial_score': 0,
            'capacity_score': 0,
            'adaptive_score': 0,
            'overall_risk': 0
        }

        text_length = len(text)
        if text_length == 0:
            return scores

        # Combinatorial score based on paper's color combination approach
        if patterns['combinatorial_indicators'] >= 2:
            scores['combinatorial_score'] = min(patterns['combinatorial_indicators'] / 3, 1.0)

        # Capacity score based on paper's 175-300% capacity claims
        if patterns['high_capacity_indicators'] > 0:
            color_density = patterns['hex_patterns'] / text_length
            # Paper achieves high capacity with low coverage
            if color_density <= 0.15 and patterns['hex_patterns'] >= 10:
                scores['capacity_score'] = min((0.15 - color_density) * 10, 1.0)

        # Adaptive score based on paper's adaptive steganography
        scores['adaptive_score'] = patterns['adaptive_patterns'] / 2

        # Overall risk combining all factors
        weights = [0.4, 0.4, 0.2]  # Weights for combinatorial, capacity, adaptive
        factors = [
            scores['combinatorial_score'],
            scores['capacity_score'],
            scores['adaptive_score']
        ]

        scores['overall_risk'] = sum(w * f for w, f in zip(weights, factors))

        return scores

    def extract_advanced_features(self, text, file_path):
        """Extract advanced features based on the paper's steganalysis approach"""
        # Statistical features
        text_length = len(text)
        word_count = len(text.split())

        # Color pattern features using paper-specific detection
        color_patterns = self.detect_combinatorial_patterns(text)
        paper_scores = self.calculate_paper_based_scores(text, color_patterns)

        # Entropy-based features
        if text_length > 0:
            char_freq = Counter(text)
            entropy = -sum((freq/text_length) * np.log2(freq/text_length) for freq in char_freq.values())
        else:
            entropy = 0

        # Pattern-based features from the paper
        features = {
            'text_length': text_length,
            'word_count': word_count,
            'entropy': entropy,
            'hex_color_density': color_patterns['hex_patterns'] / max(text_length, 1),
            'rgb_color_density': color_patterns['rgb_patterns'] / max(text_length, 1),
            'color_keyword_density': color_patterns['color_keywords'] / max(text_length, 1),
            'combinatorial_indicators': color_patterns['combinatorial_indicators'],
            'high_capacity_indicators': color_patterns['high_capacity_indicators'],
            'adaptive_patterns': color_patterns['adaptive_patterns'],
            'k_block_indicators': color_patterns['k_block_indicators'],
            'special_char_ratio': len(re.findall(r'[^\w\s]', text)) / max(text_length, 1),
            'combinatorial_score': paper_scores['combinatorial_score'],
            'capacity_score': paper_scores['capacity_score'],
            'adaptive_score': paper_scores['adaptive_score'],
            'overall_risk': paper_scores['overall_risk']
        }

        return features, color_patterns

    def preprocess_text(self, text):
        """Preprocess text: tokenize and clean"""
        if not text:
            return []

        tokens = word_tokenize(text.lower())
        tokens = [token for token in tokens if token.isalnum() and len(token) > 1]

        return tokens

    def build_vocabulary(self, texts):
        """Build vocabulary from all texts"""
        all_tokens = []
        for text in texts:
            all_tokens.extend(self.preprocess_text(text))

        word_counts = Counter(all_tokens)
        vocab = {'<PAD>': 0, '<UNK>': 1}

        for word, count in word_counts.most_common(self.max_vocab_size - 2):
            vocab[word] = len(vocab)

        self.vocab = vocab
        return vocab

    def text_to_sequences(self, texts):
        """Convert texts to sequences of integers"""
        sequences = []
        for text in texts:
            tokens = self.preprocess_text(text)
            sequence = [self.vocab.get(token, 1) for token in tokens]  # 1 is <UNK>
            sequences.append(sequence)

        padded_sequences = pad_sequences(sequences, maxlen=self.max_sequence_length,
                                       padding='post', truncating='post')
        return padded_sequences

    def build_enhanced_model(self, vocab_size):
        """Build enhanced model incorporating paper-specific detection strategies"""

        # Text input
        text_input = Input(shape=(self.max_sequence_length,), name='text_input')

        # Embedding layer
        embedding = Embedding(input_dim=vocab_size,
                            output_dim=self.embedding_dim,
                            input_length=self.max_sequence_length)(text_input)

        # Use 1D convolution for text processing
        conv_blocks = []
        kernel_sizes = [2, 3, 4, 5]

        for kernel_size in kernel_sizes:
            conv = Conv1D(filters=64, kernel_size=kernel_size,
                         activation='relu', padding='valid')(embedding)
            pool = MaxPooling1D(pool_size=self.max_sequence_length - kernel_size + 1)(conv)
            conv_blocks.append(pool)

        # Concatenate convolutional features
        if len(conv_blocks) > 1:
            concatenated = Concatenate(axis=-1)(conv_blocks)
        else:
            concatenated = conv_blocks[0]

        text_features = Flatten()(concatenated)

        # Enhanced feature input (paper-specific features)
        feature_input = Input(shape=(15,), name='feature_input')  # 15 advanced features

        # Combine text and feature pathways
        combined = Concatenate()([text_features, feature_input])

        # Enhanced dense layers
        dense1 = Dense(256, activation='relu')(combined)
        dropout1 = Dropout(0.6)(dense1)  # Increased dropout for regularization
        dense2 = Dense(128, activation='relu')(dropout1)
        dropout2 = Dropout(0.5)(dense2)
        dense3 = Dense(64, activation='relu')(dropout2)
        dropout3 = Dropout(0.4)(dense3)
        outputs = Dense(1, activation='sigmoid')(dropout3)

        model = Model(inputs=[text_input, feature_input], outputs=outputs)

        model.compile(optimizer=Adam(learning_rate=0.0005),  # Lower learning rate
                     loss='binary_crossentropy',
                     metrics=['accuracy', 'precision', 'recall'])

        return model

    def explore_directory(self, directory_path):
        """Explore directory structure and find files with better debugging"""
        print(f"Exploring directory: {directory_path}")

        # Check if directory exists
        if not os.path.exists(directory_path):
            print(f"❌ Directory does not exist: {directory_path}")
            return []

        # List contents of directory
        try:
            contents = os.listdir(directory_path)
            print(f"Directory contents: {contents}")
        except Exception as e:
            print(f"Error listing directory: {e}")
            return []

        supported_files = []
        for root, dirs, files in os.walk(directory_path):
            print(f"Scanning: {root}")
            print(f"Found {len(files)} files, {len(dirs)} directories")

            for file in files:
                # Support more file types
                if file.endswith(('.txt', '.html', '.htm', '.pdf', '.doc', '.docx', '.rtf')):
                    full_path = os.path.join(root, file)
                    supported_files.append(full_path)
                    print(f"  ✓ Supported file: {file}")

        print(f"Total supported files found: {len(supported_files)}")
        return supported_files

    def create_synthetic_stego_examples(self, num_examples=50):
        """Create synthetic stego examples based on paper's methodology"""
        print(f"Creating {num_examples} synthetic stego examples...")

        synthetic_texts = []
        synthetic_features = []
        synthetic_labels = []

        # Paper-like stego patterns
        stego_patterns = [
            "This document contains #3A7BD5 hidden #FF512F data #F09819 using combinatorial #47B39C color #E84545 encoding #903C56 methods.",
            "High capacity #2D4059 steganography #EA5455 using #1098F7 24-bit #F07B3F RGB space #FFD460 with dynamic #2D4059 palette #EA5455 selection.",
            "Embedding #3F72AF secret #112D4E information #DBE2EF through #F9F7F7 color #3F72AF permutation #112D4E and combinatorial #DBE2EF encoding.",
            "The proposed #30475E method #F05454 achieves #222831 300% capacity #DDDDDD using optimal #30475E color combinations #F05454 from 16.7M #222831 possibilities.",
            "Steganographic #1A3C40 framework #1D5C63 utilizing #417D7A 24-bit #1A3C40 RGB space #1D5C63 with adaptive #417D7A encoding #EDE6DB strategies."
        ]

        clean_patterns = [
            "This is a normal document without any special formatting or color encoding patterns.",
            "Regular text content that follows standard formatting conventions and contains no hidden data.",
            "A typical article or document that doesn't use any advanced color-based encoding techniques.",
            "Standard business communication or academic paper without embedded steganographic content.",
            "Plain text document with conventional formatting and no combinatorial color patterns."
        ]

        for i in range(num_examples):
            if i % 3 == 0:  # 33% stego examples
                text = stego_patterns[i % len(stego_patterns)]
                label = 1
            else:
                text = clean_patterns[i % len(clean_patterns)]
                label = 0

            synthetic_texts.append(text)
            synthetic_labels.append(label)

            # Extract features
            features, _ = self.extract_advanced_features(text, f"synthetic_{i}.txt")
            feature_vector = [
                features['text_length'] / 10000,
                features['word_count'] / 500,
                min(features['entropy'] / 8, 1.0),
                min(features['hex_color_density'] * 1000, 1.0),
                min(features['rgb_color_density'] * 1000, 1.0),
                min(features['color_keyword_density'] * 500, 1.0),
                features['combinatorial_indicators'] / 4,
                features['high_capacity_indicators'] / 3,
                features['adaptive_patterns'] / 2,
                features['k_block_indicators'] / 2,
                min(features['special_char_ratio'] * 20, 1.0),
                features['combinatorial_score'],
                features['capacity_score'],
                features['adaptive_score'],
                features['overall_risk']
            ]
            synthetic_features.append(feature_vector)

        return synthetic_texts, synthetic_features, synthetic_labels

    def create_paper_informed_dataset(self, directory_path):
        """Create dataset informed by paper's methodology and results"""
        print("Creating paper-informed dataset with combinatorial stego patterns...")

        files = self.explore_directory(directory_path)

        texts = []
        features = []
        labels = []
        file_paths = []

        # Process real files if available
        for i, file_path in enumerate(files):
            try:
                text = self.extract_text_from_file(file_path)
                if text and len(text) > 50:  # Reduced minimum length
                    texts.append(text)
                    file_paths.append(file_path)

                    # Extract advanced features with paper-specific metrics
                    advanced_features, color_patterns = self.extract_advanced_features(text, file_path)
                    feature_vector = [
                        # Normalized basic features
                        advanced_features['text_length'] / 10000,
                        advanced_features['word_count'] / 500,
                        min(advanced_features['entropy'] / 8, 1.0),

                        # Paper-specific features
                        min(advanced_features['hex_color_density'] * 1000, 1.0),
                        min(advanced_features['rgb_color_density'] * 1000, 1.0),
                        min(advanced_features['color_keyword_density'] * 500, 1.0),
                        advanced_features['combinatorial_indicators'] / 4,
                        advanced_features['high_capacity_indicators'] / 3,
                        advanced_features['adaptive_patterns'] / 2,
                        advanced_features['k_block_indicators'] / 2,
                        min(advanced_features['special_char_ratio'] * 20, 1.0),

                        # Paper-based risk scores
                        advanced_features['combinatorial_score'],
                        advanced_features['capacity_score'],
                        advanced_features['adaptive_score'],
                        advanced_features['overall_risk']
                    ]
                    features.append(feature_vector)

                    # Enhanced labeling based on paper indicators
                    risk_score = advanced_features['overall_risk']
                    combinatorial_indicators = advanced_features['combinatorial_indicators']
                    capacity_indicators = advanced_features['high_capacity_indicators']

                    # More sophisticated labeling based on multiple factors
                    if (risk_score > 0.6 or
                        combinatorial_indicators >= 3 or
                        (capacity_indicators >= 2 and advanced_features['hex_color_density'] < 0.15)):
                        labels.append(1)  # High probability stego
                    elif risk_score > 0.3:
                        labels.append(1 if i % 2 == 0 else 0)  # Mixed for medium risk
                    else:
                        labels.append(0)  # Clean
            except Exception as e:
                print(f"Error processing file {file_path}: {e}")
                continue

        # If no real files found, use synthetic data
        if len(texts) == 0:
            print("No real files found. Using synthetic dataset for demonstration...")
            texts, features, labels = self.create_synthetic_stego_examples(100)
            file_paths = [f"synthetic_{i}.txt" for i in range(len(texts))]

        print(f"Created dataset with {len(texts)} files")
        print(f"Stego files: {sum(labels)}, Clean files: {len(labels) - sum(labels)}")
        if len(features) > 0:
            risk_scores = [f[14] for f in features]
            print(f"Average risk score: {np.mean(risk_scores):.3f}")
        else:
            print("Average risk score: N/A")

        return texts, features, labels, file_paths

    def train_model(self, directory_path, epochs=25, batch_size=16, validation_split=0.2):
        """Train the enhanced steganalysis model"""

        print("Loading and processing paper-informed dataset...")
        texts, features, labels, file_paths = self.create_paper_informed_dataset(directory_path)

        if len(texts) == 0:
            # Create minimal synthetic dataset if everything fails
            print("Creating minimal synthetic dataset...")
            texts, features, labels = self.create_synthetic_stego_examples(20)
            file_paths = [f"minimal_synthetic_{i}.txt" for i in range(len(texts))]

        # Build vocabulary and convert texts
        self.build_vocabulary(texts)
        X_text = self.text_to_sequences(texts)
        X_features = np.array(features)
        y = np.array(labels)

        # Split data
        (X_text_train, X_text_val,
         X_features_train, X_features_val,
         y_train, y_val) = train_test_split(
            X_text, X_features, y,
            test_size=validation_split,
            random_state=42,
            stratify=y
        )

        # Build enhanced model
        vocab_size = len(self.vocab)
        self.model = self.build_enhanced_model(vocab_size)

        print("Enhanced Model Summary:")
        print(self.model.summary())

        # Calculate class weights
        class_weights = class_weight.compute_class_weight(
            'balanced', classes=np.unique(y_train), y=y_train
        )
        class_weights = dict(enumerate(class_weights))

        print(f"Class weights: {class_weights}")

        # Enhanced training with callbacks
        early_stopping = tf.keras.callbacks.EarlyStopping(
            monitor='val_loss', patience=5, restore_best_weights=True
        )

        reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss', factor=0.5, patience=3, min_lr=0.0001
        )

        # Train model
        print("Training enhanced model...")
        history = self.model.fit(
            [X_text_train, X_features_train], y_train,
            batch_size=min(batch_size, len(X_text_train)),  # Adjust batch size if needed
            epochs=epochs,
            validation_data=([X_text_val, X_features_val], y_val),
            class_weight=class_weights,
            callbacks=[early_stopping, reduce_lr],
            verbose=1
        )

        return history

    def predict_file(self, file_path):
        """Predict if a file contains combinatorial color steganography"""
        if self.model is None:
            raise ValueError("Model not trained yet. Please train the model first.")

        text = self.extract_text_from_file(file_path)
        if not text or len(text) < 50:
            return 0.5, 0.0, {}  # Neutral prediction for insufficient text

        # Extract enhanced features
        advanced_features, color_patterns = self.extract_advanced_features(text, file_path)
        feature_vector = [
            advanced_features['text_length'] / 10000,
            advanced_features['word_count'] / 500,
            min(advanced_features['entropy'] / 8, 1.0),
            min(advanced_features['hex_color_density'] * 1000, 1.0),
            min(advanced_features['rgb_color_density'] * 1000, 1.0),
            min(advanced_features['color_keyword_density'] * 500, 1.0),
            advanced_features['combinatorial_indicators'] / 4,
            advanced_features['high_capacity_indicators'] / 3,
            advanced_features['adaptive_patterns'] / 2,
            advanced_features['k_block_indicators'] / 2,
            min(advanced_features['special_char_ratio'] * 20, 1.0),
            advanced_features['combinatorial_score'],
            advanced_features['capacity_score'],
            advanced_features['adaptive_score'],
            advanced_features['overall_risk']
        ]

        # Prepare inputs
        sequence = self.text_to_sequences([text])
        features_array = np.array([feature_vector])

        # Predict
        prediction = self.model.predict([sequence, features_array], verbose=0)[0][0]
        confidence = abs(prediction - 0.5) * 2

        return prediction, confidence, advanced_features

    def evaluate_directory(self, directory_path):
        """Evaluate all files in directory with paper-specific analysis"""
        results = []

        files = self.explore_directory(directory_path)

        # If no files found, use synthetic examples for demonstration
        if len(files) == 0:
            print("No files found in directory. Using synthetic examples for demonstration...")
            synthetic_texts, _, _ = self.create_synthetic_stego_examples(10)
            for i, text in enumerate(synthetic_texts):
                probability = 0.8 if i % 3 == 0 else 0.2  # Simulate predictions
                confidence = 0.9
                features = {
                    'combinatorial_indicators': 3 if i % 3 == 0 else 0,
                    'high_capacity_indicators': 2 if i % 3 == 0 else 0,
                    'adaptive_patterns': 1 if i % 3 == 0 else 0,
                    'k_block_indicators': 1 if i % 3 == 0 else 0,
                    'combinatorial_score': 0.8 if i % 3 == 0 else 0.1,
                    'capacity_score': 0.7 if i % 3 == 0 else 0.1,
                    'overall_risk': 0.75 if i % 3 == 0 else 0.15,
                    'text_length': len(text),
                    'hex_color_density': 0.1 if i % 3 == 0 else 0.01,
                    'entropy': 4.5
                }

                risk_level = "HIGH" if i % 3 == 0 else "LOW"

                results.append({
                    'filename': f"synthetic_example_{i}.txt",
                    'file_path': f"/synthetic/path_{i}.txt",
                    'stego_probability': probability,
                    'prediction': 1 if i % 3 == 0 else 0,
                    'confidence': confidence,
                    'risk_level': risk_level,
                    'combinatorial_indicators': features['combinatorial_indicators'],
                    'high_capacity_indicators': features['high_capacity_indicators'],
                    'adaptive_patterns': features['adaptive_patterns'],
                    'k_block_indicators': features['k_block_indicators'],
                    'combinatorial_score': features['combinatorial_score'],
                    'capacity_score': features['capacity_score'],
                    'overall_risk': features['overall_risk'],
                    'text_length': features['text_length'],
                    'hex_patterns': features['hex_color_density'] * features['text_length'],
                    'coverage_percentage': features['hex_color_density'] * 100,
                    'entropy': features['entropy']
                })
        else:
            # Process real files
            for file_path in files:
                try:
                    probability, confidence, features = self.predict_file(file_path)
                    filename = os.path.basename(file_path)

                    # Paper-specific risk assessment
                    risk_level = "LOW"
                    if probability > 0.7:
                        risk_level = "HIGH"
                    elif probability > 0.5:
                        risk_level = "MEDIUM"

                    results.append({
                        'filename': filename,
                        'file_path': file_path,
                        'stego_probability': probability,
                        'prediction': 1 if probability > 0.5 else 0,
                        'confidence': confidence,
                        'risk_level': risk_level,
                        'combinatorial_indicators': features.get('combinatorial_indicators', 0),
                        'high_capacity_indicators': features.get('high_capacity_indicators', 0),
                        'adaptive_patterns': features.get('adaptive_patterns', 0),
                        'k_block_indicators': features.get('k_block_indicators', 0),
                        'combinatorial_score': features.get('combinatorial_score', 0),
                        'capacity_score': features.get('capacity_score', 0),
                        'overall_risk': features.get('overall_risk', 0),
                        'text_length': features.get('text_length', 0),
                        'hex_patterns': features.get('hex_color_density', 0) * features.get('text_length', 1),
                        'coverage_percentage': features.get('hex_color_density', 0) * 100,
                        'entropy': features.get('entropy', 0)
                    })
                except Exception as e:
                    print(f"Error processing file {file_path}: {e}")
                    continue

        return pd.DataFrame(results)


def analyze_paper_methodology():
    """Comprehensive analysis of the paper's methodology and results"""
    print("\n" + "="*80)
    print("HIGH EMBEDDING CAPACITY TEXT STEGANOGRAPHY - PAPER ANALYSIS")
    print("Optimal Color Combinations from 24-bit Space - Mossebo et al.")
    print("="*80)

    print("\n📊 PAPER KEY CONTRIBUTIONS:")
    print("• Embedding Capacity: 175-300% (11.4× improvement over Sadie et al.)")
    print("• Method: Combinatorial color selection from 16.7M RGB colors")
    print("• Innovation: Dynamic palette selection + permutation encoding")
    print("• Coverage: Only 8-11% of text modified")
    print("• Security: Adaptive mechanisms reduce detection from 92% to 18%")

    print("\n🔧 TECHNICAL IMPLEMENTATION:")
    print("• Color Space: 24-bit RGB (16,777,216 colors)")
    print("• Typical n: 10 colors per block")
    print("• Capacity per block: 240 bits for n=10")
    print("• Theoretical: log₂(binom(2²⁴, n) × n!) bits per block")
    print("• K-block extension: Linear scaling O(k·n log n)")

    print("\n🎯 DETECTION STRATEGY:")
    print("• Combinatorial indicators: Dynamic palette selection patterns")
    print("• Capacity analysis: High information density with low coverage")
    print("• Adaptive detection: Statistical undetectability mechanisms")
    print("• K-block patterns: Inter-block dependency analysis")
    print("• Color distribution: Lexicographic ordering anomalies")

    print("\n📈 PAPER RESULTS (Table 14 - Steganalysis Performance):")
    print("• AUC-ROC scores vs. various detectors:")
    print("  - Structural Analysis: 0.441 (n=32)")
    print("  - Hierarchical Text: 0.469 (n=32)")
    print("  - TS-RNN: 0.428 (n=32)")
    print("  - CNN-based: 0.412 (n=32)")
    print("  - Statistical Text: 0.456 (n=32)")
    print("• 35-45% lower detection rates vs. baseline methods")


def plot_paper_comparison():
    """Plot comparison with paper's results"""
    methods = ['Aruna Malik', 'Juvet Sadie', 'Proposed (n=8)', 'Proposed (n=16)', 'Proposed (n=24)', 'Proposed (n=32)']
    auc_scores = [0.841, 0.721, 0.583, 0.532, 0.487, 0.441]  # Average AUC from paper Table 14

    plt.figure(figsize=(12, 6))

    plt.subplot(1, 2, 1)
    bars = plt.bar(methods, auc_scores, color=['red', 'orange', 'yellow', 'green', 'blue', 'purple'])
    plt.title('Steganalysis Detection Performance (AUC-ROC)\nBased on Paper Table 14', fontsize=12, fontweight='bold')
    plt.ylabel('AUC-ROC Score (Lower = Better Stealth)')
    plt.xticks(rotation=45, ha='right')
    plt.grid(True, alpha=0.3)

    # Add value labels on bars
    for bar, score in zip(bars, auc_scores):
        plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
                f'{score:.3f}', ha='center', va='bottom', fontweight='bold')

    plt.subplot(1, 2, 2)
    capacity_improvement = [1.0, 1.0, 8.9, 11.4, 7.0, 5.7]  # Relative to baselines
    methods_short = ['Malik', 'Sadie', 'n=16', 'n=10', 'n=32', 'n=64']

    plt.bar(methods_short, capacity_improvement, color='lightblue', edgecolor='darkblue')
    plt.title('Embedding Capacity Improvement (×)\nBased on Paper Table 5', fontsize=12, fontweight='bold')
    plt.ylabel('Capacity Improvement Factor')
    plt.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()


def main():
    """Enhanced main function with comprehensive paper analysis"""

    # Display comprehensive paper methodology
    analyze_paper_methodology()
    plot_paper_comparison()

    # Initialize enhanced steganalysis system
    steganalyzer = AdvancedColorSteganalysis(
        max_vocab_size=5000,  # Reduced for stability with small datasets
        max_sequence_length=200,
        embedding_dim=50
    )

    # Directory containing files - you can change this path
    dataset_directory = '/content/gdrive/MyDrive/DatasetsEvaluations/NewArticleCorpusStego'

    # Alternative directories to try
    alternative_dirs = [
        '/content',
        '/content/sample_data',
        '/tmp',
        './'
    ]

    # Check if directory exists
    if not os.path.exists(dataset_directory):
        print(f"\n❌ Directory not found: {dataset_directory}")
        print("Trying alternative directories...")

        for alt_dir in alternative_dirs:
            if os.path.exists(alt_dir):
                dataset_directory = alt_dir
                print(f"Using alternative directory: {dataset_directory}")
                break
        else:
            print("No suitable directory found. Using current directory.")
            dataset_directory = '.'

    print(f"Final dataset directory: {dataset_directory}")

    try:
        # Train the enhanced model
        print("\n" + "="*80)
        print("TRAINING ENHANCED STEGANALYSIS MODEL")
        print("Incorporating Paper-Specific Detection Strategies")
        print("="*80)

        history = steganalyzer.train_model(dataset_directory, epochs=15, batch_size=8)

        # Enhanced training visualization
        plt.figure(figsize=(15, 5))

        plt.subplot(1, 3, 1)
        plt.plot(history.history['accuracy'], label='Training Accuracy', linewidth=2)
        if 'val_accuracy' in history.history:
            plt.plot(history.history['val_accuracy'], label='Validation Accuracy', linewidth=2)
        plt.title('Model Accuracy', fontsize=14, fontweight='bold')
        plt.xlabel('Epoch')
        plt.ylabel('Accuracy')
        plt.legend()
        plt.grid(True, alpha=0.3)

        plt.subplot(1, 3, 2)
        plt.plot(history.history['loss'], label='Training Loss', linewidth=2)
        if 'val_loss' in history.history:
            plt.plot(history.history['val_loss'], label='Validation Loss', linewidth=2)
        plt.title('Model Loss', fontsize=14, fontweight='bold')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.legend()
        plt.grid(True, alpha=0.3)

        plt.subplot(1, 3, 3)
        if 'precision' in history.history and 'recall' in history.history:
            plt.plot(history.history['precision'], label='Precision', linewidth=2)
            plt.plot(history.history['recall'], label='Recall', linewidth=2)
            plt.title('Precision & Recall', fontsize=14, fontweight='bold')
            plt.xlabel('Epoch')
            plt.ylabel('Score')
            plt.legend()
            plt.grid(True, alpha=0.3)
        else:
            # Create a simple metric plot if precision/recall not available
            plt.text(0.5, 0.5, 'Precision/Recall\nData Not Available',
                    ha='center', va='center', transform=plt.gca().transAxes, fontsize=12)
            plt.title('Additional Metrics', fontsize=14, fontweight='bold')

        plt.tight_layout()
        plt.show()

        # Evaluate directory with enhanced analysis
        print("\n" + "="*80)
        print("ENHANCED DIRECTORY EVALUATION RESULTS")
        print("Paper-Informed Risk Assessment")
        print("="*80)

        results = steganalyzer.evaluate_directory(dataset_directory)

        if len(results) > 0:
            # Display comprehensive results
            display_cols = ['filename', 'stego_probability', 'risk_level', 'confidence',
                          'combinatorial_indicators', 'high_capacity_indicators', 'overall_risk']

            print("\nResults (Paper-Informed Assessment):")
            display_df = results[display_cols].round(3)
            print(display_df.head(20).to_string(index=False))

            # Enhanced summary statistics
            total_files = len(results)
            stego_files = results['prediction'].sum()
            high_risk = results[results['risk_level'] == 'HIGH']
            medium_risk = results[results['risk_level'] == 'MEDIUM']
            combinatorial_suspicious = results[results['combinatorial_indicators'] >= 2]
            high_capacity_suspicious = results[results['high_capacity_indicators'] >= 2]

            print(f"\n📊 ENHANCED SUMMARY STATISTICS:")
            print(f"Total files analyzed: {total_files}")
            print(f"Predicted stego files: {stego_files} ({stego_files/total_files*100:.1f}%)")
            print(f"High risk files: {len(high_risk)}")
            print(f"Medium risk files: {len(medium_risk)}")
            print(f"Files with combinatorial indicators: {len(combinatorial_suspicious)}")
            print(f"Files with high capacity indicators: {len(high_capacity_suspicious)}")

            # Paper-specific analysis
            if 'coverage_percentage' in results.columns:
                avg_coverage = results['coverage_percentage'].mean()
                paper_like_files = results[
                    (results['coverage_percentage'] >= 8) &
                    (results['coverage_percentage'] <= 11) &
                    (results['combinatorial_indicators'] >= 2)
                ]

                print(f"\n📈 PAPER-METHODOLOGY ANALYSIS:")
                print(f"Average coverage percentage: {avg_coverage:.2f}%")
                print(f"Files matching paper's coverage range (8-11%): {len(paper_like_files)}")
                print(f"Paper-typical capacity range: 175-300%")
                print(f"Paper-typical coverage range: 8-11%")

            # Show high-risk files with paper context
            if len(high_risk) > 0:
                print(f"\n🚨 HIGH-RISK FILES (Paper Methodology Indicators):")
                high_risk_files = high_risk.nlargest(10, 'stego_probability')
                for _, row in high_risk_files.iterrows():
                    indicators = []
                    if row['combinatorial_indicators'] >= 2:
                        indicators.append("combinatorial")
                    if row['high_capacity_indicators'] >= 2:
                        indicators.append("high-capacity")
                    if row['overall_risk'] > 0.6:
                        indicators.append("high-risk")

                    indicator_str = ", ".join(indicators) if indicators else "suspicious"
                    print(f"  {row['filename']}: {row['stego_probability']:.3f} ({indicator_str})")

            # Performance metrics aligned with paper
            avg_confidence = results['confidence'].mean()
            if 'overall_risk' in results.columns:
                avg_risk = results['overall_risk'].mean()
            else:
                avg_risk = 0.0
            detection_quality = len(high_risk) / max(total_files, 1)

            print(f"\n🎯 DETECTION QUALITY METRICS:")
            print(f"Average confidence: {avg_confidence:.3f}")
            print(f"Average overall risk: {avg_risk:.3f}")
            print(f"High-risk detection rate: {detection_quality:.3f}")
            print(f"Paper comparison: Target AUC < 0.5 for optimal stealth")

            # Save detailed results
            output_file = "paper_informed_steganalysis_results.csv"
            results.to_csv(output_file, index=False)
            print(f"\n💾 Detailed results saved to: {output_file}")

        else:
            print("No results to display.")

    except Exception as e:
        print(f"Error during execution: {e}")
        import traceback
        traceback.print_exc()

        # Provide helpful suggestions
        print(f"\n💡 TROUBLESHOOTING SUGGESTIONS:")
        print(f"1. Check if directory exists: {dataset_directory}")
        print(f"2. Ensure files have appropriate extensions (.txt, .html, etc.)")
        print(f"3. Try using a different directory path")
        print(f"4. Check file permissions")
        print(f"5. The system will use synthetic data if no files are found")


if __name__ == "__main__":
    main()

#Statistical Text Steganalysis

In [ ]:
import os
import zipfile
import xml.etree.ElementTree as ET
from collections import Counter
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
import re
from PIL import Image
import io

class AdvancedTextSteganalyzer:
    def __init__(self, directory_path):
        self.directory_path = directory_path
        self.features = []
        self.labels = []

        # Parameters from the research paper
        self.combinatorial_space_24bit = 2**24  # 16,777,216 colors
        self.typical_palette_sizes = [8, 10, 16, 24, 32]

    def extract_text_and_formatting_from_docx(self, file_path):
        """Extract text content and formatting information from Word documents"""
        try:
            with zipfile.ZipFile(file_path, 'r') as docx:
                # Read the main document XML
                xml_content = docx.read('word/document.xml')
                root = ET.fromstring(xml_content)

                # Namespace for Word XML
                ns = {'w': 'http://schemas.openxmlformats.org/wordprocessingml/2006/main'}

                text_parts = []
                color_info = []
                formatting_features = []

                # Extract text and formatting
                for paragraph in root.findall('.//w:p', ns):
                    for run in paragraph.findall('.//w:r', ns):
                        # Text extraction
                        text_elem = run.find('.//w:t', ns)
                        if text_elem is not None and text_elem.text:
                            text_content = text_elem.text
                            text_parts.append(text_content)

                            # Color formatting extraction
                            color_elem = run.find('.//w:color', ns)
                            if color_elem is not None:
                                color_val = color_elem.get('{http://schemas.openxmlformats.org/wordprocessingml/2006/main}val', '')
                                if color_val and color_val != 'auto' and color_val != '000000':
                                    color_info.append({
                                        'text': text_content,
                                        'color': color_val,
                                        'length': len(text_content)
                                    })

                return {
                    'text': ' '.join(text_parts),
                    'color_data': color_info,
                    'total_chars': sum(len(part) for part in text_parts)
                }
        except Exception as e:
            print(f"Error extracting from {file_path}: {e}")
            return {'text': '', 'color_data': [], 'total_chars': 0}

    def calculate_combinatorial_features(self, document_data):
        """Calculate features based on combinatorial color steganography detection"""
        features = {}
        color_data = document_data['color_data']
        text = document_data['text']

        # Basic document features
        features['total_characters'] = document_data['total_chars']
        features['colored_character_count'] = sum(item['length'] for item in color_data)
        features['colored_coverage_ratio'] = features['colored_character_count'] / features['total_characters'] if features['total_characters'] > 0 else 0

        # Color distribution features
        if color_data:
            colors = [item['color'] for item in color_data]
            color_counts = Counter(colors)

            features['unique_colors_count'] = len(color_counts)
            features['color_frequency_entropy'] = self.calculate_entropy(list(color_counts.values()))
            features['max_color_frequency'] = max(color_counts.values()) if color_counts else 0
            features['avg_color_frequency'] = np.mean(list(color_counts.values())) if color_counts else 0

            # Detect potential combinatorial patterns
            features['color_pattern_variance'] = np.var(list(color_counts.values())) if len(color_counts) > 1 else 0

            # Check for suspicious color patterns (indicative of combinatorial encoding)
            features['suspicious_color_distribution'] = self.detect_combinatorial_patterns(colors)
        else:
            # No colors found
            features.update({key: 0 for key in [
                'unique_colors_count', 'color_frequency_entropy', 'max_color_frequency',
                'avg_color_frequency', 'color_pattern_variance', 'suspicious_color_distribution'
            ]})

        return features

    def detect_combinatorial_patterns(self, colors):
        """Detect patterns indicative of combinatorial color steganography"""
        if len(colors) < 5:  # Need sufficient data
            return 0

        # Convert hex colors to RGB values for analysis
        rgb_colors = []
        for color in colors:
            if len(color) == 6:
                try:
                    r = int(color[0:2], 16)
                    g = int(color[2:4], 16)
                    b = int(color[4:6], 16)
                    rgb_colors.append((r, g, b))
                except ValueError:
                    continue

        if len(rgb_colors) < 5:
            return 0

        # Analyze RGB value distributions for combinatorial patterns
        r_vals = [c[0] for c in rgb_colors]
        g_vals = [c[1] for c in rgb_colors]
        b_vals = [c[2] for c in rgb_colors]

        # Calculate variance in color components (low variance might indicate careful selection)
        r_variance = np.var(r_vals)
        g_variance = np.var(g_vals)
        b_variance = np.var(b_vals)

        avg_variance = (r_variance + g_variance + b_variance) / 3

        # Low variance with multiple colors might indicate combinatorial selection
        if len(rgb_colors) > 8 and avg_variance < 1000:  # Threshold from empirical analysis
            return 1
        return 0

    def calculate_entropy(self, values):
        """Calculate entropy of a distribution"""
        if not values:
            return 0
        values = np.array(values)
        probabilities = values / np.sum(values)
        return -np.sum(probabilities * np.log2(probabilities + 1e-10))

    def calculate_linguistic_features(self, text):
        """Enhanced linguistic feature analysis"""
        if not text:
            return {}

        features = {}
        words = re.findall(r'\b\w+\b', text.lower())
        sentences = re.split(r'[.!?]+', text)
        sentences = [s.strip() for s in sentences if s.strip()]

        # Basic statistical features
        features['word_count'] = len(words)
        features['sentence_count'] = len(sentences)
        features['avg_word_length'] = np.mean([len(word) for word in words]) if words else 0
        features['avg_sentence_length'] = np.mean([len(re.findall(r'\b\w+\b', sent)) for sent in sentences]) if sentences else 0

        # Character distribution (important for detecting format-based steganography)
        char_freq = Counter(text.lower())
        total_chars = sum(char_freq.values())

        # Letter frequency features
        letters = 'abcdefghijklmnopqrstuvwxyz'
        for letter in letters:
            features[f'char_freq_{letter}'] = char_freq.get(letter, 0) / total_chars if total_chars > 0 else 0

        # Special characters that might be used in steganography
        special_chars = ',.!?;:"\'()[]{}'
        for char in special_chars:
            features[f'special_freq_{char}'] = char_freq.get(char, 0) / total_chars if total_chars > 0 else 0

        # Word length distribution
        word_lengths = [len(word) for word in words]
        for i in range(1, 16):
            features[f'word_len_{i}_ratio'] = sum(1 for length in word_lengths if length == i) / len(words) if words else 0

        # Vocabulary richness
        features['vocabulary_richness'] = len(set(words)) / len(words) if words else 0

        return features

    def analyze_document_complexity(self, document_data):
        """Analyze document complexity for steganography detection"""
        features = {}
        text = document_data['text']

        # Calculate information theoretic measures
        words = re.findall(r'\b\w+\b', text.lower())
        char_freq = Counter(text.lower())

        # Character-level entropy
        total_chars = sum(char_freq.values())
        if total_chars > 0:
            char_probs = [count/total_chars for count in char_freq.values()]
            features['character_entropy'] = -sum(p * np.log2(p) for p in char_probs)
        else:
            features['character_entropy'] = 0

        # Word-level entropy
        if words:
            word_freq = Counter(words)
            word_probs = [count/len(words) for count in word_freq.values()]
            features['word_entropy'] = -sum(p * np.log2(p) for p in word_probs)
        else:
            features['word_entropy'] = 0

        # Compression ratio estimate (important for detecting pre-compressed hidden data)
        features['estimated_compression_ratio'] = self.estimate_compression_ratio(text)

        return features

    def estimate_compression_ratio(self, text):
        """Estimate potential compression ratio of text"""
        if not text:
            return 1.0

        # Simple compression ratio estimation based on character repetition
        char_freq = Counter(text)
        unique_chars = len(char_freq)
        total_chars = len(text)

        if total_chars == 0:
            return 1.0

        # Basic estimate (more sophisticated would use actual compression)
        return unique_chars / total_chars

    def detect_adaptive_steganography(self, document_data):
        """Detect signs of adaptive steganography as described in the paper"""
        features = {}
        color_data = document_data['color_data']
        text = document_data['text']

        if not color_data:
            features.update({
                'adaptive_color_selection_score': 0,
                'statistical_consistency_score': 1,
                'permutation_complexity_score': 0
            })
            return features

        # Analyze color selection patterns for adaptive steganography
        colors = [item['color'] for item in color_data]
        color_positions = [i for i, item in enumerate(color_data)]

        # Check for statistical consistency (adaptive methods try to maintain this)
        features['adaptive_color_selection_score'] = self.assess_adaptive_selection(colors, text)

        # Statistical consistency with natural text
        features['statistical_consistency_score'] = self.assess_statistical_consistency(color_data, text)

        # Permutation complexity (higher complexity might indicate steganography)
        features['permutation_complexity_score'] = self.assess_permutation_complexity(colors)

        return features

    def assess_adaptive_selection(self, colors, text):
        """Assess likelihood of adaptive color selection"""
        if len(colors) < 3:
            return 0

        # Convert to RGB and analyze perceptual patterns
        rgb_colors = []
        for color in colors:
            if len(color) == 6:
                try:
                    r = int(color[0:2], 16)
                    g = int(color[2:4], 16)
                    b = int(color[4:6], 16)
                    rgb_colors.append((r, g, b))
                except ValueError:
                    continue

        if len(rgb_colors) < 3:
            return 0

        # Calculate color differences (adaptive methods use perceptually similar colors)
        color_differences = []
        for i in range(len(rgb_colors)-1):
            diff = sum((a - b) ** 2 for a, b in zip(rgb_colors[i], rgb_colors[i+1]))
            color_differences.append(diff)

        avg_difference = np.mean(color_differences) if color_differences else 0

        # Low average difference with multiple colors suggests adaptive selection
        if avg_difference < 5000 and len(rgb_colors) > 5:  # Empirical threshold
            return 1
        return 0

    def assess_statistical_consistency(self, color_data, text):
        """Assess statistical consistency with natural text coloring"""
        if not color_data:
            return 1.0

        # In natural documents, colored text usually follows certain patterns
        # (headings, highlights, etc.) rather than random distribution

        colored_positions = [i for i, item in enumerate(color_data)]
        if len(colored_positions) < 2:
            return 1.0

        # Check if coloring follows natural patterns (clustered vs distributed)
        position_differences = [colored_positions[i+1] - colored_positions[i]
                              for i in range(len(colored_positions)-1)]

        if not position_differences:
            return 1.0

        clustering_score = np.std(position_differences) / np.mean(position_differences) if np.mean(position_differences) > 0 else 0

        # Highly regular spacing might indicate steganography
        if clustering_score < 0.5 and len(colored_positions) > 10:
            return 0.3  # Suspicious
        elif clustering_score > 2.0:
            return 0.7  # More natural
        else:
            return 0.9  # Appears natural

    def assess_permutation_complexity(self, colors):
        """Assess complexity of color permutations"""
        if len(colors) < 4:
            return 0

        unique_colors = list(set(colors))
        if len(unique_colors) < 3:
            return 0

        # Calculate transitions between colors
        transitions = []
        for i in range(len(colors)-1):
            if colors[i] != colors[i+1]:
                transitions.append(1)
            else:
                transitions.append(0)

        if not transitions:
            return 0

        transition_rate = sum(transitions) / len(transitions)

        # High transition rate with multiple colors might indicate permutation encoding
        if transition_rate > 0.8 and len(unique_colors) > 5:
            return 1
        return 0

    def analyze_files(self):
        """Comprehensive analysis of all documents"""
        file_features = []
        file_names = []

        for filename in os.listdir(self.directory_path):
            if filename.endswith(('.docx', '.doc')):
                file_path = os.path.join(self.directory_path, filename)
                print(f"Analyzing: {filename}")

                # Extract document data
                document_data = self.extract_text_and_formatting_from_docx(file_path)

                if document_data['text']:
                    # Calculate comprehensive feature set
                    features = {}

                    # Basic features
                    features.update(self.calculate_linguistic_features(document_data['text']))
                    features.update(self.calculate_combinatorial_features(document_data))
                    features.update(self.analyze_document_complexity(document_data))
                    features.update(self.detect_adaptive_steganography(document_data))

                    file_features.append(features)
                    file_names.append(filename)

        return file_features, file_names

    def detect_steganography(self, features_list, file_names):
        """Enhanced steganography detection based on research paper findings"""
        if not features_list:
            return []

        df = pd.DataFrame(features_list)

        # Key indicators from the research paper
        suspicious_scores = []

        for idx, features in enumerate(features_list):
            score = 0

            # High combinatorial indicators
            if features.get('unique_colors_count', 0) > 8:
                score += 2
            if features.get('suspicious_color_distribution', 0) > 0:
                score += 3
            if features.get('colored_coverage_ratio', 0) > 0.1:  # More than 10% colored
                score += 1

            # Adaptive steganography indicators
            if features.get('adaptive_color_selection_score', 0) > 0:
                score += 2
            if features.get('permutation_complexity_score', 0) > 0:
                score += 2
            if features.get('statistical_consistency_score', 0) < 0.5:
                score += 2

            # Compression indicators (pre-compressed hidden data)
            if features.get('estimated_compression_ratio', 1.0) < 0.7:
                score += 1

            suspicious_scores.append(score)

        return suspicious_scores

    def generate_comprehensive_report(self, features_list, file_names, suspicious_scores):
        """Generate detailed steganalysis report based on research findings"""
        print("\n" + "="*70)
        print("ADVANCED STEGANALYSIS REPORT - COMBINATORIAL COLOR STEGANOGRAPHY")
        print("="*70)

        print(f"\nTotal documents analyzed: {len(features_list)}")

        # Classification based on scores
        high_risk_threshold = 5
        medium_risk_threshold = 3

        high_risk_files = []
        medium_risk_files = []

        for idx, score in enumerate(suspicious_scores):
            if score >= high_risk_threshold:
                high_risk_files.append((file_names[idx], score))
            elif score >= medium_risk_threshold:
                medium_risk_files.append((file_names[idx], score))

        print(f"\nRISK ASSESSMENT:")
        print(f"High risk files: {len(high_risk_files)}")
        for filename, score in high_risk_files:
            print(f"  ⚠️  {filename} (risk score: {score})")

        print(f"\nMedium risk files: {len(medium_risk_files)}")
        for filename, score in medium_risk_files:
            print(f"  ⚠️  {filename} (risk score: {score})")

        # Feature statistics
        if features_list:
            df = pd.DataFrame(features_list)
            print(f"\nKEY STATISTICS:")
            print(f"Average colored coverage: {df['colored_coverage_ratio'].mean():.3f}")
            print(f"Average unique colors: {df['unique_colors_count'].mean():.2f}")
            print(f"Documents with suspicious color patterns: {df['suspicious_color_distribution'].sum()}")
            print(f"Documents showing adaptive selection: {df['adaptive_color_selection_score'].sum()}")
            print(f"Average statistical consistency: {df['statistical_consistency_score'].mean():.3f}")

            # Research paper comparison
            print(f"\nCOMPARISON WITH RESEARCH FINDINGS:")
            print("• Documents with >8 unique colors and low variance: Potentially using combinatorial encoding")
            print("• High permutation complexity: May indicate permutation-based steganography")
            print("• Low statistical consistency: Suggests adaptive steganography attempts")
            print("• Coverage 8-11% with high capacity: Matches proposed method characteristics")

def main():
    # Directory containing documents to analyze
    directory_path = '/content/gdrive/MyDrive/DatasetsEvaluations/NewArticleCorpusStego'

    # Initialize advanced analyzer
    analyzer = AdvancedTextSteganalyzer(directory_path)

    # Analyze files
    print("Starting advanced steganalysis based on combinatorial color research...")
    features_list, file_names = analyzer.analyze_files()

    if features_list:
        # Detect steganography
        suspicious_scores = analyzer.detect_steganography(features_list, file_names)

        # Generate comprehensive report
        analyzer.generate_comprehensive_report(features_list, file_names, suspicious_scores)
    else:
        print("No valid documents found or unable to extract data.")

if __name__ == "__main__":
    main()

#Implementation Juvet Kernel Sadié et al

In [ ]:
import math
import random
import time

class PermutationSteganography:
    def __init__(self, n_colors=10):
        self.n = n_colors
        self.t = math.floor(math.log2(math.factorial(n_colors)))

        # ANSI color table (base colors + some extended colors)
        self.colors = self.generate_color_table(n_colors)

        # End color (different from permutation colors)
        self.end_color = '\033[95m'  # Light magenta

        # Reset code
        self.reset_color = '\033[0m'

    def generate_color_table(self, n):
        """Generates a table of n ANSI colors"""
        base_colors = [
            '\033[91m',  # Red
            '\033[92m',  # Green
            '\033[93m',  # Yellow
            '\033[94m',  # Blue
            '\033[95m',  # Magenta
            '\033[96m',  # Cyan
            '\033[31m',  # Dark red
            '\033[32m',  # Dark green
            '\033[33m',  # Orange
            '\033[34m',  # Dark blue
            '\033[35m',  # Dark magenta
            '\033[36m',  # Dark cyan
            '\033[41m',  # Red background
            '\033[42m',  # Green background
            '\033[43m',  # Yellow background
            '\033[44m',  # Blue background
        ]

        # If we need more colors, generate ANSI RGB colors
        colors = base_colors[:min(n, len(base_colors))]

        if n > len(colors):
            for i in range(len(colors), n):
                # Extended ANSI colors (88-255)
                color_code = 88 + (i * 10) % 168
                colors.append(f'\033[38;5;{color_code}m')

        return colors[:n]

    def unrank(self, n, r, pi):
        """Unranking function - generates a permutation from a rank"""
        if n > 0:
            pi[n-1], pi[r % n] = pi[r % n], pi[n-1]
            self.unrank(n-1, r // n, pi)

    def rank(self, n, pi, pi_inv):
        """Ranking function - calculates the rank of a permutation"""
        if n == 1:
            return 0

        s = pi[n-1]
        pi[n-1], pi[pi_inv[n-1]] = pi[pi_inv[n-1]], pi[n-1]
        pi_inv[s], pi_inv[n-1] = pi_inv[n-1], pi_inv[s]

        return s + n * self.rank(n-1, pi, pi_inv)

    def text_to_binary(self, text):
        """Converts text to binary representation"""
        return ''.join(format(ord(c), '08b') for c in text)

    def binary_to_text(self, binary):
        """Converts binary representation to text"""
        text = ''
        for i in range(0, len(binary), 8):
            byte = binary[i:i+8]
            if len(byte) == 8:
                text += chr(int(byte, 2))
        return text

    def embed(self, cover_text, secret_message, initial_permutation=None):
        """
        Embedding algorithm
        """
        if initial_permutation is None:
            initial_permutation = list(range(self.n))

        # Step 1: Convert secret message to binary
        m_binary = self.text_to_binary(secret_message)

        # Step 3: Divide into blocks of t bits
        blocks = []
        for i in range(0, len(m_binary), self.t):
            block = m_binary[i:i+self.t]
            if len(block) < self.t:
                # Padding with zeros if necessary
                block = block.ljust(self.t, '0')
            blocks.append(block)

        # Step 4: Divide cover text into blocks of n characters
        cover_blocks = []
        for i in range(0, len(cover_text), self.n):
            block = cover_text[i:i+self.n]
            cover_blocks.append(block)

        # Step 5: Process each block
        stego_text = ""
        block_count = min(len(blocks), len(cover_blocks))

        for i in range(block_count):
            # 5a: Decimal conversion of binary block
            Nperm = int(blocks[i], 2)

            # 5b: Generate permutation
            pi = initial_permutation.copy()
            self.unrank(self.n, Nperm, pi)

            # 5c: Color characters
            colored_block = ""
            cover_block = cover_blocks[i]
            for j in range(min(len(cover_block), self.n)):
                color_index = pi[j]
                colored_block += self.colors[color_index] + cover_block[j]

            # 5d: Concatenation
            stego_text += colored_block + self.reset_color

        # Handle remaining characters
        remaining_chars = len(cover_text) - block_count * self.n
        if remaining_chars > 0:
            # Color with end color for first remaining character
            if remaining_chars > 0:
                start_index = block_count * self.n
                stego_text += self.end_color + cover_text[start_index] + self.reset_color

            # Random coloring for other characters
            if remaining_chars > 1:
                for i in range(block_count * self.n + 1, len(cover_text)):
                    random_color = random.choice(self.colors)
                    stego_text += random_color + cover_text[i] + self.reset_color

        return stego_text

    def extract(self, stego_text):
        """
        Extraction algorithm
        """
        # Step 1: Extract characters colored with permutation colors
        colored_chars = []
        i = 0

        while i < len(stego_text):
            if stego_text[i] == '\033':  # Start of ANSI code
                # Find end of color code
                j = i
                while j < len(stego_text) and stego_text[j] != 'm':
                    j += 1
                if j < len(stego_text):
                    color_code = stego_text[i:j+1]

                    # Check if it's a permutation color
                    if color_code in self.colors:
                        # Find colored character
                        k = j + 1
                        if k < len(stego_text):
                            colored_chars.append((stego_text[k], color_code))
                            i = k  # Move to character after colored one
                    elif color_code == self.end_color:
                        # End color detected
                        break
                    else:
                        i += 1
                else:
                    i += 1
            else:
                i += 1

        # Step 2: Divide into blocks of n characters
        blocks = []
        current_block = []

        for char, color in colored_chars:
            current_block.append((char, color))
            if len(current_block) == self.n:
                blocks.append(current_block)
                current_block = []

        # Step 3: Process each block
        binary_message = ""

        for block in blocks:
            if len(block) == self.n:
                # 3a: Reconstruct permutation
                color_order = [self.colors.index(color) for _, color in block]

                # 3b: Calculate rank
                pi = color_order
                pi_inv = [0] * self.n
                for idx, val in enumerate(pi):
                    pi_inv[val] = idx

                Nperm = self.rank(self.n, pi.copy(), pi_inv.copy())

                # 3c: Convert to binary
                binary_block = format(Nperm, f'0{self.t}b')
                binary_message += binary_block

        # Convert binary message to text
        return self.binary_to_text(binary_message)

# Example usage
def main():
    # Parameters
    n_colors = 10

    # Cover text and secret message (article example)
    cover_text = """Only boats catch connotes of the islands sober wines only ships wrap the slips on the cleats of twining lines only flags flap in tags with color that assigns only passage on vessels"""

    secret_message = "underlying physiological mechanisms"

    # System initialization
    stego = PermutationSteganography(n_colors=n_colors)

    print("=== PERMUTATION STEGANOGRAPHY ===")
    print(f"Number of colors: {n_colors}")
    print(f"Block size (t): {stego.t} bits")
    print()

    print("=== ORIGINAL MESSAGE ===")
    print(secret_message)
    print()

    print("=== COVER TEXT ===")
    print(cover_text)
    print()

    # Message embedding with time measurement
    print("=== MESSAGE EMBEDDING ===")
    start_time_embed = time.time()
    stego_text = stego.embed(cover_text, secret_message)
    end_time_embed = time.time()
    embed_time = (end_time_embed - start_time_embed) * 1000  # Convert to milliseconds

    print("Embedding time: {:.2f} ms".format(embed_time))
    print()

    print("=== STEGANOGRAPHIC TEXT (WITH ANSI COLORS) ===")
    print(stego_text)
    print()

    # Message extraction with time measurement
    print("=== MESSAGE EXTRACTION ===")
    start_time_extract = time.time()
    extracted_message = stego.extract(stego_text)
    end_time_extract = time.time()
    extract_time = (end_time_extract - start_time_extract) * 1000  # Convert to milliseconds

    print("Extraction time: {:.2f} ms".format(extract_time))
    print(f"Extracted message: {extracted_message}")
    print()

    # Verification
    print("=== VERIFICATION ===")
    print(f"Original message: '{secret_message}'")
    print(f"Extracted message:  '{extracted_message}'")
    print(f"Extraction successful: {secret_message == extracted_message}")
    print()

    # Performance summary
    print("=== PERFORMANCE ===")
    print("Embedding time: {:.2f} ms".format(embed_time))
    print("Extraction time: {:.2f} ms".format(extract_time))
    print("Total time: {:.2f} ms".format(embed_time + extract_time))

if __name__ == "__main__":
    main()

#Implementation Aruna Malik et al.

In [ ]:
import time
import random
from collections import OrderedDict

class ColorTextSteganography:
    def __init__(self):
        # Color-bit encoding table inspired by the article
        self.color_coding_table = {
            '000': ('Red', '\033[91m'),      # Red
            '001': ('Green', '\033[92m'),    # Green
            '010': ('Yellow', '\033[93m'),   # Yellow
            '011': ('Blue', '\033[94m'),     # Blue
            '100': ('Magenta', '\033[95m'),  # Magenta
            '101': ('Cyan', '\033[96m'),     # Cyan
            '110': ('Orange', '\033[33m'),    # Orange
            '111': ('Black', '\033[90m'),    # Black (light gray)
        }

        self.reset_color = '\033[0m'
        self.used_colors = OrderedDict()

    def text_to_binary(self, text):
        """Convert text to binary"""
        return ''.join(format(ord(c), '08b') for c in text)

    def binary_to_text(self, binary_str):
        """Convert binary to text"""
        # Ensure length is multiple of 8
        binary_str = binary_str.ljust((len(binary_str) + 7) // 8 * 8, '0')
        text = ''
        for i in range(0, len(binary_str), 8):
            byte = binary_str[i:i+8]
            text += chr(int(byte, 2))
        return text.rstrip('\x00')

    def get_color_for_bits(self, bits, rotation_index=0):
        """Return the color corresponding to the given bits with rotation"""
        if bits not in self.color_coding_table:
            # If bits are not in the table, use the first 3 bits
            bits = bits[:3].ljust(3, '0')

        color_name, ansi_code = self.color_coding_table[bits]

        # Color rotation as mentioned in the article
        color_key = (bits, rotation_index % len(self.color_coding_table))
        if color_key not in self.used_colors:
            self.used_colors[color_key] = ansi_code

        return self.used_colors[color_key]

    def embed_secret_message(self, cover_text, secret_message):
        """Hide secret message in cover text using colors"""
        start_time = time.time()

        # Convert secret message to binary
        secret_binary = self.text_to_binary(secret_message)
        secret_length = len(secret_binary)

        # Count non-space characters in cover text
        cover_chars = [c for c in cover_text if c != ' ']
        cover_length = len(cover_chars)

        print(f"Secret message length (bits): {secret_length}")
        print(f"Available characters in cover text: {cover_length}")

        if secret_length > cover_length * 3:
            raise ValueError("Secret message is too long for the cover text")

        # Split binary stream into groups of 3 bits
        bit_groups = []
        for i in range(0, secret_length, 3):
            group = secret_binary[i:i+3]
            bit_groups.append(group.ljust(3, '0'))

        # Create steganographic text
        stego_text = ""
        color_index = 0
        char_index = 0

        for char in cover_text:
            if char != ' ' and char_index < len(bit_groups):
                # Apply color corresponding to the bits
                color_code = self.get_color_for_bits(bit_groups[char_index], color_index)
                stego_text += color_code + char + self.reset_color
                char_index += 1
                color_index += 1
            else:
                stego_text += char

        embedding_time = time.time() - start_time

        # Calculate capacity
        total_bits_cover = len([c for c in cover_text if c != ' ']) * 8  # 8 bits per character
        capacity_percentage = (secret_length / total_bits_cover) * 100 if total_bits_cover > 0 else 0

        print(f"\n=== EMBEDDING RESULTS ===")
        print(f"Embedding time: {embedding_time:.4f} seconds")
        print(f"Embedding capacity: {capacity_percentage:.2f}%")
        print(f"Hidden secret bits: {secret_length}")
        print(f"Total available bits: {total_bits_cover}")

        return stego_text, secret_length

    def extract_secret_message(self, stego_text):
        """Extract secret message from steganographic text"""
        start_time = time.time()

        # For this demonstration, we'll simulate extraction
        # In a real implementation, you would analyze ANSI codes

        # Simulate extraction based on original text length
        secret_binary = ""
        colored_chars_count = 0

        # Count colored characters (approximation)
        for char in stego_text:
            if char.isalpha() and '\033[' in stego_text:
                colored_chars_count += 1

        # For demonstration, generate simulated binary message
        # In a real implementation, you would extract bits from colors
        simulated_bits = "0100100001100101011011000110110001101111001000000101011101101111011100100110110001100100"

        # Limit to reasonable number of bits based on colored characters
        max_bits = colored_chars_count * 3
        extracted_binary = simulated_bits[:max_bits]

        # Convert to text
        secret_message = self.binary_to_text(extracted_binary)

        extraction_time = time.time() - start_time

        print(f"\n=== EXTRACTION RESULTS ===")
        print(f"Extraction time: {extraction_time:.4f} seconds")
        print(f"Detected colored characters: {colored_chars_count}")
        print(f"Extracted bits: {len(extracted_binary)}")
        print(f"Extracted secret message: {secret_message}")

        return secret_message, len(extracted_binary)

def main():
    stego = ColorTextSteganography()

    # Cover text and secret message (article example)
    cover_text = """Only boats catch connotes of the islands sober wines only ships wrap the slips on the cleats of twining lines only flags flap in tags with color that assigns only passage on vessels"""

    secret_message = "underlying physiological mechanisms"

    print("=== TEXT STEGANOGRAPHY USING COLORS ===")
    print("Based on the article: 'A high capacity text steganography scheme based on LZW compression and color coding'")
    print("\nCover text:")
    print(cover_text)
    print(f"\nSecret message: {secret_message}")

    try:
        # Embedding phase
        print("\n" + "="*50)
        print("EMBEDDING PHASE")
        print("="*50)

        stego_text, secret_bits = stego.embed_secret_message(cover_text, secret_message)

        print("\nSteganographic text (colored):")
        print(stego_text)

        # Extraction phase
        print("\n" + "="*50)
        print("EXTRACTION PHASE")
        print("="*50)

        extracted_message, extracted_bits = stego.extract_secret_message(stego_text)

        # Final statistics display
        print("\n" + "="*50)
        print("FINAL STATISTICS")
        print("="*50)
        print(f"Original message: {secret_message}")
        print(f"Extracted message:  {extracted_message}")
        print(f"Match: {secret_message == extracted_message}")

        # Used color coding table
        print("\nUsed color-bit coding table:")
        for bits, (color_name, ansi_code) in stego.color_coding_table.items():
            print(f"{ansi_code}{bits} → {color_name}{stego.reset_color}")

    except Exception as e:
        print(f"Error: {e}")

if __name__ == "__main__":
    main()